In [ ]:
trainDist= ['RatF00_2021_07_16_10_47_23', 'RatF00_2021_07_16_16_33_15', 'RatF00_2021_07_17_11_22_30', 'RatF00_2021_07_17_16_10_48', 
            'RatF01_2021_07_16_11_22_52', 'RatF01_2021_07_16_17_04_58', 'RatF01_2021_07_17_11_54_07', 'RatF01_2021_07_17_16_41_45', 
            'RatF02_2021_07_16_11_59_16', 'RatF02_2021_07_16_17_36_31', 'RatF02_2021_07_17_12_25_20', 'RatF02_2021_07_17_17_12_56', 
            'RatM00_2021_07_16_13_11_18', 'RatM00_2021_07_16_16_47_19', 'RatM00_2021_07_17_11_46_48', 'RatM00_2021_07_17_16_14_00', 
            'RatM01_2021_07_16_13_47_30', 'RatM01_2021_07_16_17_19_18', 'RatM01_2021_07_17_12_18_12', 'RatM01_2021_07_17_16_45_06', 
            'RatM02_2021_07_16_14_23_43', 'RatM02_2021_07_16_17_51_58', 'RatM02_2021_07_17_12_59_51', 'RatM02_2021_07_17_17_16_27']
dist60 =   ['RatF00_2021_07_19_13_12_15', 'RatF00_2021_07_19_15_25_33', 'RatF00_2021_07_21_10_43_27', 'RatF00_2021_07_21_15_19_33', 
            'RatF00_2021_07_26_10_23_57', 'RatF00_2021_07_26_15_17_06', 'RatF00_2021_08_19_13_14_34', 'RatF00_2021_08_19_15_40_01', 
            'RatF01_2021_07_19_11_09_43', 'RatF01_2021_07_19_16_26_55', 'RatF01_2021_07_21_11_45_12', 'RatF01_2021_07_21_16_21_18', 
            'RatF01_2021_07_26_11_25_24', 'RatF01_2021_07_26_16_18_43', 'RatF01_2021_08_19_11_09_37', 'RatF01_2021_08_19_16_41_27',
            'RatF02_2021_07_19_12_11_07', 'RatF02_2021_07_19_17_28_24', 'RatF02_2021_07_21_12_46_49', 'RatF02_2021_07_21_17_22_58', 
            'RatF02_2021_07_26_12_27_04', 'RatF02_2021_07_26_17_20_00', 'RatF02_2021_08_19_12_10_48', 'RatF02_2021_08_19_17_42_46', 
            'RatM00_2021_07_19_10_35_22', 'RatM00_2021_07_19_15_27_18', 'RatM00_2021_07_21_10_45_30', 'RatM00_2021_07_21_15_21_31', 
            'RatM00_2021_07_26_10_25_22', 'RatM00_2021_07_26_15_18_23', 'RatM00_2021_08_19_10_31_37', 'RatM00_2021_08_19_15_40_49', 
            'RatM01_2021_07_19_11_36_24', 'RatM01_2021_07_19_16_28_01', 'RatM01_2021_07_21_11_46_29', 'RatM01_2021_07_21_16_22_30', 
            'RatM01_2021_07_26_11_26_38', 'RatM01_2021_07_26_16_19_55', 'RatM01_2021_08_19_11_32_59', 'RatM01_2021_08_19_16_45_42', 
            'RatM02_2021_07_19_12_37_31', 'RatM02_2021_07_19_17_30_31', 'RatM02_2021_07_21_12_48_07', 'RatM02_2021_07_21_17_25_04', 
            'RatM02_2021_07_26_12_28_45', 'RatM02_2021_07_26_17_21_19', 'RatM02_2021_08_19_12_34_15', 'RatM02_2021_08_19_17_47_18']
dist90 =   ['RatF00_2021_07_18_10_36_33', 'RatF00_2021_07_18_15_09_13', 'RatF00_2021_07_23_10_38_43', 'RatF00_2021_07_23_15_04_12', 
            'RatF00_2021_07_25_10_54_14', 'RatF00_2021_07_25_15_38_20', 'RatF00_2021_08_18_11_11_19', 'RatF00_2021_08_18_16_06_36', 
            'RatF01_2021_07_18_11_37_31', 'RatF01_2021_07_18_16_10_15', 'RatF01_2021_07_23_11_39_33', 'RatF01_2021_07_23_16_05_46', 
            'RatF01_2021_07_25_11_55_52', 'RatF01_2021_07_25_16_39_39', 'RatF01_2021_08_18_12_12_24', 'RatF01_2021_08_18_17_07_36', 
            'RatF02_2021_07_18_12_38_36', 'RatF02_2021_07_18_17_11_30', 'RatF02_2021_07_23_12_40_32', 'RatF02_2021_07_23_17_08_21', 
            'RatF02_2021_07_25_12_57_49', 'RatF02_2021_07_25_17_40_44', 'RatF02_2021_08_18_13_13_41', 'RatF02_2021_08_18_18_08_44', 
            'RatM00_2021_07_18_10_39_28', 'RatM00_2021_07_18_15_11_10', 'RatM00_2021_07_23_10_40_23', 'RatM00_2021_07_23_15_06_05', 
            'RatM00_2021_07_25_10_55_48', 'RatM00_2021_07_25_15_36_11', 'RatM00_2021_08_18_11_11_07', 'RatM00_2021_08_18_16_07_18', 
            'RatM01_2021_07_18_11_40_37', 'RatM01_2021_07_18_16_12_12', 'RatM01_2021_07_23_11_41_25', 'RatM01_2021_07_23_16_07_06', 
            'RatM01_2021_07_25_11_57_05', 'RatM01_2021_07_25_16_37_04', 'RatM01_2021_08_18_12_13_40', 'RatM01_2021_08_18_17_08_37', 
            'RatM02_2021_07_18_12_41_41', 'RatM02_2021_07_18_17_13_11', 'RatM02_2021_07_23_12_42_27', 'RatM02_2021_07_23_17_08_10', 
            'RatM02_2021_07_25_12_59_12', 'RatM02_2021_07_25_17_37_53', 'RatM02_2021_08_18_13_15_09', 'RatM02_2021_08_18_18_10_26']
dist120 =  ['RatF00_2021_07_20_10_09_39', 'RatF00_2021_07_20_15_10_08', 'RatF00_2021_07_22_11_23_48', 'RatF00_2021_07_22_16_11_45', 
            'RatF00_2021_07_24_10_49_55', 'RatF00_2021_07_24_15_28_05', 'RatF00_2021_08_20_10_49_54', 'RatF00_2021_08_20_15_40_51', 
            'RatF01_2021_07_20_11_11_58', 'RatF01_2021_07_20_16_11_04', 'RatF01_2021_07_22_12_25_04', 'RatF01_2021_07_22_17_13_24', 
            'RatF01_2021_07_24_11_54_05', 'RatF01_2021_07_24_16_30_03', 'RatF01_2021_08_20_11_54_26', 'RatF01_2021_08_20_16_42_08', 
            'RatF02_2021_07_20_12_13_07', 'RatF02_2021_07_20_17_12_04', 'RatF02_2021_07_22_13_26_20', 'RatF02_2021_07_22_18_14_16', 
            'RatF02_2021_07_24_12_55_26', 'RatF02_2021_07_24_17_31_25', 'RatF02_2021_08_20_12_55_40', 'RatF02_2021_08_20_17_43_34', 
            'RatM00_2021_07_20_10_11_03', 'RatM00_2021_07_20_15_12_06', 'RatM00_2021_07_22_11_25_16', 'RatM00_2021_07_22_16_13_03', 
            'RatM00_2021_07_24_10_55_12', 'RatM00_2021_07_24_15_29_12', 'RatM00_2021_08_20_10_51_04', 'RatM00_2021_08_20_15_41_33', 
            'RatM01_2021_07_20_11_13_08', 'RatM01_2021_07_20_16_13_12', 'RatM01_2021_07_22_12_26_37', 'RatM01_2021_07_22_17_14_48', 
            'RatM01_2021_07_24_11_56_06', 'RatM01_2021_07_24_16_31_33', 'RatM01_2021_08_20_11_55_30', 'RatM01_2021_08_20_16_42_59', 
            'RatM02_2021_07_20_12_14_33', 'RatM02_2021_07_20_17_14_10', 'RatM02_2021_07_22_13_27_46', 'RatM02_2021_07_22_18_15_36', 
            'RatM02_2021_07_24_12_56_56', 'RatM02_2021_07_24_17_32_55', 'RatM02_2021_08_20_12_56_49', 'RatM02_2021_08_20_17_44_34']
TMtrain =  ['RatF00_2021_07_30_10_32_07', 'RatF00_2021_07_30_15_32_10', 'RatF00_2021_07_31_10_40_17', 'RatF00_2021_07_31_15_24_58', 
            'RatF00_2021_08_01_10_49_06', 'RatF00_2021_08_01_15_49_09', 'RatF00_2021_08_02_10_30_32', 'RatF00_2021_08_02_15_59_06', 
            'RatF01_2021_07_30_11_33_07', 'RatF01_2021_07_30_16_33_00', 'RatF01_2021_07_31_11_41_14', 'RatF01_2021_07_31_16_25_57', 
            'RatF01_2021_08_01_11_50_10', 'RatF01_2021_08_01_16_50_07', 'RatF01_2021_08_02_11_31_27', 'RatF01_2021_08_02_17_00_00', 
            'RatF02_2021_07_30_12_34_04', 'RatF02_2021_07_30_17_34_15', 'RatF02_2021_07_31_12_42_14', 'RatF02_2021_07_31_17_27_02', 
            'RatF02_2021_08_01_12_51_09', 'RatF02_2021_08_01_17_51_10', 'RatF02_2021_08_02_12_32_40', 'RatF02_2021_08_02_18_00_51', 
            'RatM00_2021_07_30_10_35_44', 'RatM00_2021_07_30_15_33_46', 'RatM00_2021_07_31_10_41_10', 'RatM00_2021_07_31_15_25_47', 
            'RatM00_2021_08_01_10_50_09', 'RatM00_2021_08_01_15_50_07', 'RatM00_2021_08_02_10_31_53', 'RatM00_2021_08_02_16_00_01', 
            'RatM01_2021_07_30_11_42_29', 'RatM01_2021_07_30_16_34_47', 'RatM01_2021_07_31_11_42_28', 'RatM01_2021_07_31_16_27_07', 
            'RatM01_2021_08_01_11_51_28', 'RatM01_2021_08_01_16_51_16', 'RatM01_2021_08_02_11_32_52', 'RatM01_2021_08_02_17_01_04', 
            'RatM02_2021_07_30_12_43_35', 'RatM02_2021_07_30_17_35_44', 'RatM02_2021_07_31_12_43_40', 'RatM02_2021_07_31_17_28_15', 
            'RatM02_2021_08_01_12_52_18', 'RatM02_2021_08_01_17_52_29', 'RatM02_2021_08_02_12_33_55', 'RatM02_2021_08_02_18_02_09']
TMrev20 =  ['RatF00_2021_08_07_10_45_13', 'RatF00_2021_08_07_15_26_07', 'RatF00_2021_08_11_10_41_10', 'RatF00_2021_08_11_15_33_54', 'RatF00_2021_08_15_10_52_18', 'RatF00_2021_08_15_15_49_08', 
            'RatF01_2021_08_07_11_46_04', 'RatF01_2021_08_07_16_26_48', 'RatF01_2021_08_11_11_42_10', 'RatF01_2021_08_11_16_35_02', 'RatF01_2021_08_15_11_53_04', 'RatF01_2021_08_15_16_51_02', 
            'RatF02_2021_08_07_12_48_58', 'RatF02_2021_08_07_17_28_04', 'RatF02_2021_08_11_12_43_53', 'RatF02_2021_08_11_17_36_16', 'RatF02_2021_08_15_12_53_48', 'RatF02_2021_08_15_17_51_39', 
            'RatM00_2021_08_06_10_36_23', 'RatM00_2021_08_06_15_32_55', 'RatM00_2021_08_12_10_37_48', 'RatM00_2021_08_12_15_39_32', 'RatM00_2021_08_14_10_37_14', 'RatM00_2021_08_14_15_32_22', 
            'RatM01_2021_08_06_11_37_52', 'RatM01_2021_08_06_16_34_19', 'RatM01_2021_08_12_11_38_44', 'RatM01_2021_08_12_16_46_52', 'RatM01_2021_08_14_11_38_54', 'RatM01_2021_08_14_16_34_17', 
            'RatM02_2021_08_06_12_38_50', 'RatM02_2021_08_06_17_35_15', 'RatM02_2021_08_12_12_40_31', 'RatM02_2021_08_12_17_48_03', 'RatM02_2021_08_14_12_39_46', 'RatM02_2021_08_14_17_35_16']
TMrev10 =  ['RatF00_2021_08_05_10_29_33', 'RatF00_2021_08_05_15_29_54', 'RatF00_2021_08_09_10_42_32', 'RatF00_2021_08_09_15_48_50', 'RatF00_2021_08_17_10_19_46', 'RatF00_2021_08_17_15_29_43', 
            'RatF01_2021_08_05_11_30_21', 'RatF01_2021_08_05_16_30_40', 'RatF01_2021_08_09_11_43_28', 'RatF01_2021_08_09_16_49_38', 'RatF01_2021_08_17_11_20_34', 'RatF01_2021_08_17_16_30_55', 
            'RatF02_2021_08_05_12_31_10', 'RatF02_2021_08_05_17_31_27', 'RatF02_2021_08_09_12_44_26', 'RatF02_2021_08_09_17_50_20', 'RatF02_2021_08_17_12_21_55', 'RatF02_2021_08_17_17_32_14', 
            'RatM00_2021_08_04_10_07_13', 'RatM00_2021_08_04_15_44_22', 'RatM00_2021_08_10_10_41_37', 'RatM00_2021_08_10_15_42_34', 'RatM00_2021_08_16_10_35_13', 'RatM00_2021_08_16_15_41_30', 
            'RatM01_2021_08_04_11_09_09', 'RatM01_2021_08_04_16_45_12', 'RatM01_2021_08_10_11_42_29', 'RatM01_2021_08_10_16_43_47', 'RatM01_2021_08_16_11_36_24', 'RatM01_2021_08_16_16_42_42', 
            'RatM02_2021_08_04_12_10_25', 'RatM02_2021_08_04_17_46_12', 'RatM02_2021_08_10_17_53_24',                               'RatM02_2021_08_16_12_38_05', 'RatM02_2021_08_16_17_43_34']
TMrev2 =   ['RatF00_2021_08_03_10_23_20', 'RatF00_2021_08_03_15_41_26', 'RatF00_2021_08_13_10_27_32', 'RatF00_2021_08_13_15_44_15', 
            'RatF01_2021_08_03_11_24_05', 'RatF01_2021_08_03_16_43_24', 'RatF01_2021_08_13_11_30_45', 'RatF01_2021_08_13_16_45_13', 
            'RatF02_2021_08_03_12_25_11', 'RatF02_2021_08_03_17_44_16', 'RatF02_2021_08_13_12_31_33', 'RatF02_2021_08_13_17_46_17', 
            'RatM00_2021_08_08_11_18_51', 'RatM00_2021_08_08_15_31_01', 
            'RatM01_2021_08_08_12_19_45', 'RatM01_2021_08_08_16_32_01', 
            'RatM02_2021_08_08_13_21_04', 'RatM02_2021_08_08_17_33_10']
TM2 =      ['RatF00_2021_08_08_11_17_57', 'RatF00_2021_08_08_15_30_16', 
            'RatF01_2021_08_08_12_18_43', 'RatF01_2021_08_08_16_30_58', 
            'RatF02_2021_08_08_13_20_05', 'RatF02_2021_08_08_17_32_08', 
            'RatM00_2021_08_03_10_24_37', 'RatM00_2021_08_03_15_42_34', 'RatM00_2021_08_13_10_28_37', 'RatM00_2021_08_13_15_45_06', 
            'RatM01_2021_08_03_11_25_30', 'RatM01_2021_08_03_16_44_21', 'RatM01_2021_08_13_11_31_58', 'RatM01_2021_08_13_16_46_17', 
            'RatM02_2021_08_03_12_26_20', 'RatM02_2021_08_03_17_45_22', 'RatM02_2021_08_13_12_32_47', 'RatM02_2021_08_13_17_47_30']
TM10 =     ['RatF00_2021_08_04_10_05_18', 'RatF00_2021_08_04_15_43_24', 'RatF00_2021_08_10_10_40_12', 'RatF00_2021_08_10_15_41_42', 'RatF00_2021_08_16_10_34_26', 'RatF00_2021_08_16_15_40_53', 
            'RatF01_2021_08_04_11_07_56', 'RatF01_2021_08_04_16_44_07', 'RatF01_2021_08_10_11_41_20', 'RatF01_2021_08_10_16_42_38', 'RatF01_2021_08_16_11_35_22', 'RatF01_2021_08_16_16_41_43', 
            'RatF02_2021_08_04_12_09_17', 'RatF02_2021_08_04_17_44_51', 'RatF02_2021_08_10_12_42_33', 'RatF02_2021_08_10_17_52_03', 'RatF02_2021_08_16_12_36_57', 'RatF02_2021_08_16_17_42_33', 
            'RatM00_2021_08_05_10_30_59', 'RatM00_2021_08_05_15_30_39', 'RatM00_2021_08_09_10_43_27', 'RatM00_2021_08_09_15_49_35', 'RatM00_2021_08_17_10_20_33', 'RatM00_2021_08_17_15_30_55', 
            'RatM01_2021_08_05_11_31_50', 'RatM01_2021_08_05_16_31_37', 'RatM01_2021_08_09_11_44_31', 'RatM01_2021_08_09_16_50_42', 'RatM01_2021_08_17_11_21_34', 'RatM01_2021_08_17_16_32_05', 
            'RatM02_2021_08_05_12_33_53', 'RatM02_2021_08_05_17_32_26', 'RatM02_2021_08_09_12_45_40', 'RatM02_2021_08_09_17_51_24', 'RatM02_2021_08_17_12_23_24', 'RatM02_2021_08_17_17_33_17']
TM20 =     ['RatF00_2021_08_06_10_35_27', 'RatF00_2021_08_06_15_32_02', 'RatF00_2021_08_12_10_36_41', 'RatF00_2021_08_12_15_45_21', 'RatF00_2021_08_14_10_36_26', 'RatF00_2021_08_14_15_31_37', 
            'RatF01_2021_08_06_11_36_20', 'RatF01_2021_08_06_16_33_11', 'RatF01_2021_08_12_11_37_39', 'RatF01_2021_08_12_16_46_09', 'RatF01_2021_08_14_11_37_37', 'RatF01_2021_08_14_16_33_08', 
            'RatF02_2021_08_06_12_37_37', 'RatF02_2021_08_06_17_34_04', 'RatF02_2021_08_12_12_39_21', 'RatF02_2021_08_12_17_47_05', 'RatF02_2021_08_14_12_38_35', 'RatF02_2021_08_14_17_34_04', 
            'RatM00_2021_08_07_10_46_02', 'RatM00_2021_08_07_15_26_57', 'RatM00_2021_08_11_10_42_10', 'RatM00_2021_08_11_15_35_45', 'RatM00_2021_08_15_10_53_09', 'RatM00_2021_08_15_15_50_06', 
            'RatM01_2021_08_07_11_47_13', 'RatM01_2021_08_07_16_27_46', 'RatM01_2021_08_11_11_43_21', 'RatM01_2021_08_11_16_39_45', 'RatM01_2021_08_15_11_54_13', 'RatM01_2021_08_15_16_52_06', 
            'RatM02_2021_08_07_12_50_26', 'RatM02_2021_08_07_17_29_16', 'RatM02_2021_08_11_12_45_01', 'RatM02_2021_08_11_17_40_25', 'RatM02_2021_08_15_12_55_02', 'RatM02_2021_08_15_17_52_46']

postVacsTrain = ['RatF00_2021_09_27_16_04_43', 'RatF01_2021_09_27_17_05_39', 'RatF02_2021_09_27_18_06_33', 
                 'RatM00_2021_09_27_16_05_59', 'RatM01_2021_09_27_17_06_53', 'RatM02_2021_09_27_18_08_08']
postVacs60 =    ['RatF00_2021_09_29_10_43_05', 'RatF00_2021_09_29_15_41_27', 
                 'RatF01_2021_09_29_11_45_23', 'RatF01_2021_09_29_16_42_54', 
                 'RatF02_2021_09_29_12_46_34', 'RatF02_2021_09_29_17_45_08', 
                 'RatM00_2021_09_29_10_40_46', 'RatM00_2021_09_29_15_42_25', 
                 'RatM01_2021_09_29_11_46_35', 'RatM01_2021_09_29_16_44_55', 
                 'RatM02_2021_09_29_12_47_43', 'RatM02_2021_09_29_17_46_14']
postVacs60BIS = ['RatF00_2021_10_02_10_55_50', 'RatF00_2021_10_02_15_53_59', 
                 'RatF01_2021_10_02_11_57_25', 'RatF01_2021_10_02_16_55_25', 
                 'RatF02_2021_10_02_12_58_33', 'RatF02_2021_10_02_17_56_20', 
                 'RatM00_2021_10_02_10_57_06', 'RatM00_2021_10_02_15_54_54', 
                 'RatM01_2021_10_02_11_59_01', 'RatM01_2021_10_02_16_56_39', 
                 'RatM02_2021_10_02_12_59_46', 'RatM02_2021_10_02_17_57_29']
postVacs90_1 =  ['RatF00_2021_09_28_10_39_10', 'RatF00_2021_09_28_15_36_14', 
                 'RatF01_2021_09_28_11_40_07', 'RatF01_2021_09_28_16_41_40', 
                 'RatF02_2021_09_28_12_41_05', 'RatF02_2021_09_28_17_42_33', 
                 'RatM00_2021_09_28_10_40_16', 'RatM00_2021_09_28_15_36_58', 
                 'RatM01_2021_09_28_11_41_16', 'RatM01_2021_09_28_16_42_45', 
                 'RatM02_2021_09_28_12_42_13', 'RatM02_2021_09_28_17_43_52']
postVacs90_2 =  ['RatF00_2021_10_01_10_38_51', 'RatF00_2021_10_01_15_44_46', 
                 'RatF01_2021_10_01_11_40_00', 'RatF01_2021_10_01_16_45_33', 
                 'RatF02_2021_10_01_12_41_03', 'RatF02_2021_10_01_17_46_50', 
                 'RatM00_2021_10_01_10_39_37', 'RatM00_2021_10_01_15_45_34', 
                 'RatM01_2021_10_01_11_41_14', 'RatM01_2021_10_01_16_46_35', 
                 'RatM02_2021_10_01_12_42_08', 'RatM02_2021_10_01_17_48_04']
postVacs120 =   ['RatF00_2021_09_30_10_11_50', 'RatF00_2021_09_30_15_30_05', 
                 'RatF01_2021_09_30_11_12_53', 'RatF01_2021_09_30_16_30_47', 
                 'RatF02_2021_09_30_12_13_39', 'RatF02_2021_09_30_17_33_37', 
                 'RatM00_2021_09_30_10_12_31', 'RatM00_2021_09_30_15_30_47', 
                 'RatM01_2021_09_30_11_13_57', 'RatM01_2021_09_30_16_31_44', 
                 'RatM02_2021_09_30_12_15_02', 'RatM02_2021_09_30_17_34_43']

#animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
#for animal in animalList:
    #for session in sorted(matchsession(animal, dist60+dist90+dist120)):
    #for session in sorted(matchsession(animal, postVacs)):
    #for session in sorted(matchsession(animal, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20)):
        #print(animal, session)
    #for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        #if params[animal, session]['treadmillDist'] == 60:            print("'"+session+"', ")
        #if params[animal, session]['treadmillDist'] == 90:            print("'"+session+"', ")
        #if params[animal, session]['treadmillDist'] == 120:           print("'"+session+"', ")
        #if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:            print("'"+session+"', ")
        #if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:            print("'"+session+"', ")
        #if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:            print("'"+session+"', ")
        #if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:            print("'"+session+"', ")
        #if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:            print("'"+session+"', ")
        #if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:            print("'"+session+"', ")]

In [ ]:
# OC 
# https://epubs.siam.org/doi/pdf/10.1137/1.9781611975758.10
# https://arxiv.org/pdf/2003.00142.pdf
# https://www.yoptimization.com/goddard_benchmark
# https://link.springer.com/content/pdf/10.1007/BF00198466.pdf
# https://journals.plos.org/plosbiology/article/file?id=10.1371/journal.pbio.2001323&type=printable
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1488877/pdf/nihms10892.pdf

# park
# https://onlinelibrary.wiley.com/doi/abs/10.1002/ana.410310117

# Traj + other
# https://www.sciencedirect.com/science/article/abs/pii/S0957417419304415
# https://arxiv.org/pdf/1802.06971.pdf

# https://link.springer.com/content/pdf/10.1007/s004220050521.pdf
# https://www.curiousjason.com/talks/Analysis_of_arm_movement_data_part_2.pdf
# https://www.frontiersin.org/articles/10.3389/fneur.2018.00897/full
# https://www.semanticscholar.org/paper/Trajectory-Preprocessing-Lee-Krumm/75060b36204b046f1e93455f4884f5b4c8ebe0ea
# https://stackoverflow.com/questions/15085918/clustering-for-trajectories
# https://arxiv.org/pdf/2012.00464.pdf
# https://ascelibrary.org/doi/pdf/10.1061/%28ASCE%29CP.1943-5487.0000528
# https://www2.cs.arizona.edu/~sankar/1212.0935.pdf
# https://www.fstaals.net/publications/trajectories/central2017/central_eurocg2015_local.pdf
# https://link.springer.com/content/pdf/10.1007/s00453-012-9654-2.pdf
# https://stackoverflow.com/questions/65341351/how-to-average-multiple-trajectories-in-r
# https://medium.com/isiway-tech/gps-trajectories-clustering-in-python-2f5874204a53
# https://books.google.fr/books?id=Lk20DAAAQBAJ&pg=PA65&lpg=PA65&dq=python+median+of++trajectories&source=bl&ots=5YPLv0HeEH&sig=ACfU3U2EXzgGlCiBXMDVIxa0L7QvxjaX2g&hl=fr&sa=X&ved=2ahUKEwjDwsC7tOP0AhUuzIUKHVLuDr4Q6AF6BAgWEAM#v=onepage&q=python%20median%20of%20%20trajectories&f=false
# https://www.nature.com/articles/s41467-020-14766-3.pdf
# https://github.com/t0mk/seg
# https://arxiv.org/pdf/1802.06971.pdf
# https://hal.archives-ouvertes.fr/hal-02110086/document
# https://www.sciencedirect.com/science/article/abs/pii/0167945784900058
# https://reader.elsevier.com/reader/sd/pii/S0031320320305331?token=5DBFE25A8FF735059BD72398479335BBAE514D8872F9B4FB7B984864DC62B0D4DC5E9FAA759F3530D24720FFF2A2B013&originRegion=eu-west-1&originCreation=20211214171059

# Import all the libraries that we will need 

In [ ]:
import copy
import csv
import dabest
import datetime
import fnmatch
import glob
from IPython.display import Image, display, clear_output
import itertools
from itertools import groupby
import matplotlib.cbook
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing
import numpy as np
import os
import pandas as pd
import pickle
import platform
from pylab import rcParams
import re
import scipy
from scipy import stats
from scipy.ndimage import gaussian_filter as smooth
from scipy.signal import find_peaks
import statistics
import sys
import time
import warnings
matplotlib.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
startTimeNotebook = datetime.datetime.now()

# Define data path.

In [ ]:
###Set the notebook screen ratio for jupyter
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:85% !important; }</style>"))

#### Define folder with data 
if platform.system()=='Linux':
    root="/home/david/Desktop/allcleanDATA20210820/123456_DATAclean"#testDist3Days"#2TM/20210622"#root="/home/david/Desktop/DATARUN_ANALYSIS/test0727"#new/test1203"#DATARUN ANALYSIS/all distance" #all distances" #DATARUN ANALYSIS/test0820
    savePath="/home/david/Desktop/Save"
else: #mac: if platform.system()=='Darwin':
    root="/Users/tom/Desktop/allcleanDATA20210820/123456_DATAclean"#AET-2/test1203"#Desktop/DATARUN_ANALYSIS/test0826"#" #current : test0826
    savePath="/Users/tom/Desktop/Save"#os.path.join(root,"ALLRATS","Fig")
print("Path to data is: %s"%root)

retval = os.getcwd()
print("Current working directory: %s" % retval)#print("Notebook Path: ", notebookPath)
print("Save Path: ", savePath)   

lickBug = []#'RatF01 RatF01_2021_07_21_16_21_18'
notfixed = ['RatM02 RatM02_2021_07_18_17_13_11', 'RatF00 RatF00_2021_08_19_13_14_34', 'RatF00 RatF00_2021_08_19_15_40_01', 'RatF01 RatF01_2021_07_21_16_21_18', 'RatF00 RatF00_2021_07_21_10_43_27', 'RatF00 RatF00_2021_07_21_15_19_33']
F00lostTRACKlick = ['RatF00 RatF00_2021_08_20_10_49_54', 'RatF00 RatF00_2021_08_20_15_40_51']
buggedRatSessions = ['RatF00 RatF00_2021_07_19_13_12_15']
boundariesBug = [["RatF00 RatF00_2021_07_19_15_25_33", [40, 20]], ["RatF01 RatF01_2021_07_19_11_09_43", [40,20]], ['RatM02 RatM02_2021_08_19_12_34_15', [35, 17]], ['RatM02 RatM02_2021_08_19_12_34_15', [35, 17]], ['RatM02 RatM02_2021_08_19_12_34_15', [35, 17]], ['RatF00 RatF00_2021_08_18_11_11_19', [75, 22]], ['RatF02 RatF02_2021_08_18_18_08_44', [75, 20]], ['RatF02 RatF02_2021_08_19_17_42_46', [40, 18]], ['RatF02 RatF02_2021_07_18_17_11_30', [75, 23]], ['RatF02 RatF02_2021_07_21_12_46_49', [43, 21]], ['RatF02 RatF02_2021_07_23_17_08_21', [75, 25]], ['RatF02 RatF02_2021_07_26_12_27_04', [40, 22]], ['RatF00 RatF00_2021_07_20_10_09_39', [100, 20]], ['RatF00 RatF00_2021_07_21_10_43_27', [40, 20]], ['RatF00 RatF00_2021_07_21_15_19_33', [40, 20]], ['RatF00 RatF00_2021_07_23_10_38_43', [75, 25]], ['RatF00 RatF00_2021_07_23_15_04_12', [75, 25]], ['RatF00 RatF00_2021_07_24_10_49_55', [100, 25]], ['RatF00 RatF00_2021_07_25_10_54_14', [75, 25]], ['RatF00 RatF00_2021_07_25_15_38_20', [75, 25]], ['RatF00 RatF00_2021_07_26_15_17_06', [45, 22]], ]
runstaysepbug = ['RatF00 RatF00_2021_07_18_10_36_33', 'RatM01 RatM01_2021_07_18_11_40_37', 'RatM01 RatM01_2021_07_18_16_12_12', 'RatM01 RatM01_2021_07_19_16_28_01', 'RatM00 RatM00_2021_08_19_15_40_49', 'RatF01 RatF01_2021_08_19_11_09_37', 'RatM02 RatM02_2021_07_21_12_48_07', 'RatM02 RatM02_2021_07_21_17_25_04', 'RatM02 RatM02_2021_07_25_17_37_53', 'RatM02 RatM02_2021_07_26_12_28_45', 'RatM02 RatM02_2021_07_26_17_21_19', 'RatM01 RatM01_2021_07_21_11_46_29', 'RatM01 RatM01_2021_07_26_11_26_38', 'RatM01 RatM01_2021_07_26_16_19_55', 'RatM00 RatM00_2021_07_21_10_45_30', 'RatM00 RatM00_2021_07_26_10_25_22', 'RatM00 RatM00_2021_07_26_15_18_23', 'RatF02 RatF02_2021_07_20_17_12_04', 'RatF00 RatF00_2021_07_20_10_09_39', 'RatF00 RatF00_2021_07_21_10_43_27', ] 

# Define functions

In [ ]:
###------------------------------------------------------------------------------------------------------------------
### Utility/low_level computation functions()
###------------------------------------------------------------------------------------------------------------------

# conversion
def inch2cm(value): return value / 2.54
def cm2inch(value): return value * 2.54

#new px to cm conversion. To correct camera lens distorsion (pixels in the center of the treadmill are more precise than the ones located at the extremities), I directly applied a filter in LabView, and conversion should be uniform now, 11 px is equal to 1 cm at every point of the treadmill.
def datapx2cm(list):
    array = []
    for pos in list:
        if pos == 0:
            array.append(pos)
        elif pos > 0 and pos < 1300:
            array.append(pos/11)
        else: 
            array.append(pos)
            print("might have error in position", pos)
    return array         

#function to split lists --> used to split the raw X position array into smaller arrays (runs and stays). Later in the code we modify the array and change some values to 0, which will be used as cutting points.
def split_a_list_at_zeros(List):
    return [list(g) for k, g in groupby(List, key=lambda x:x!=0) if k]

#function to open and read from the .position files using pandas, specify the path of the file to open, the column that you want to extract from, and the extension of the file
def read_csv_pandas(path, Col=None, header=None):
    #verify that the file exists
    if not os.path.exists(path):
        print("No file %s"%path)
        return []
    try: #open the file
        csvData=pd.read_csv(path, header=header, delim_whitespace=True, low_memory=False)
    except ValueError:
        print("%s not valid (usually empty)"%path)
        return []
        #verify that the column that we specified is not empty, and return the values
    if Col is not None:
        return csvData.values[:, Col[0]] #np.insert(csvData.values[:, Col[0]], 0, 0.0)
    else:
        return csvData

#cuts session in bins
def bin_session(animal, session, data_to_cut, data_template, bins):
    output={}
    bincount = 0
    for timebin in bins:
        if timebin[0] == 0:
            start_of_bin = 0
        else:
            start_of_bin = int(np.where(data_template[animal, session] == timebin[0])[0])+1
        end_of_bin = int(np.where(data_template[animal, session] == timebin[1])[0])+1
        output[bincount] = data_to_cut[animal, session][start_of_bin : end_of_bin]
        bincount += 1 
    return output

#function to read the parameters for each rat for the session in the behav.param file. Specify the name of the parameter that you want to get from the file and optionally the value type that you want. File path is not an option, maybe change that. Dataindex is in case you don't only want the last value in line, so you can choose which value you want using its index --maybe add the option to choose a range of values. 
def read_params(animal, session, paramName, dataindex = -1, valueType=str):
    #define path of the file
    behav = root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".behav_param"
    #check if it exists
    if not os.path.exists(behav):
        print("No file %s" % behav)  
    #check if it is not empty
    #if os.stat(behav).st_size == 0:
        #print("File empty %s" % behav)
    with open(behav, "r") as f:
        #scan the file for a specific parameter, if the name of the parameter is here, get the value 
        for line in f:
            if valueType is str:
                if paramName in line:
                    #get the last value of the line [-1], values are separated with _blanks_ with the .split() function
                    return int(line.split()[dataindex])  
            if valueType is float: 
                if paramName in line:
                    return float(line.split()[dataindex])

#save data as pickle 
def save_as_pickle(data, animal, session, name):
    sessionPath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session
    folderPath = os.path.join(sessionPath, "Analysis")
    if not os.path.exists(folderPath):
        os.mkdir(folderPath)
    filePath = os.path.join(folderPath, name)
    pickle.dump(data, open(filePath, "wb" ))

#load data that has been pickled
def get_from_pickle(animal, session, name): # add pickle name to pick the data
        sessionPath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session
        analysisPath = os.path.join(sessionPath, "Analysis")
        picklePath = os.path.join(analysisPath, name) 
        # if not re.do, and there is a pickle, try to read it
        if os.path.exists(picklePath):
            try:
                data = pickle.load(open(picklePath, "rb"))
                return data
            except:
                print("error")
                pass

#save animal plot as png 
def save_sessionplot_as_png(animal, session, filename, dpi = 'figure', transparent = True, background = 'auto'):
    sessionPath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session
    folderPath = os.path.join(sessionPath, "Figures")
    if not os.path.exists(folderPath):
        os.mkdir(folderPath)
    filePath = os.path.join(folderPath, filename)
    plt.savefig(filePath, dpi=dpi, transparent=transparent, facecolor = background, edgecolor = background)

#save plot as png 
def save_plot_as_png(filename, dpi = 'figure', transparent = True, background = 'auto'):
    folderPath = os.path.join(os.getcwd(), "Figures")
    if not os.path.exists(folderPath):
        os.mkdir(folderPath)
    filePath = os.path.join(folderPath, filename)
    plt.savefig(filePath, dpi=dpi, transparent=transparent, facecolor = background, edgecolor = background)

#only display one legend when there is duplicates (e.g. we don't want one label per run)
def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

#small utility function to compute moving average, used to see the eventual acquisition bugs
def movinavg(interval, window_size):
    if window_size != 0:
        window = np.ones(int(window_size))/float(window_size)
    else:
        print("Error: Window size == 0")
    return np.convolve(interval, window, 'same')

#same with median, used to compute moving threshold for lick detection
def movinmedian(interval, window_size):
    if window_size != 0: window = int(window_size)
    else: print("Error: Window size == 0")
    val = pd.Series(interval)
    return val.rolling(window).median()

def reversemovinmedian(interval, window_size):
    if window_size != 0: window =int(window_size)
    else: print("Error: Window size == 0")
    val = pd.Series(interval[::-1])
    return list(reversed(val.rolling(window).median()))

#function to compute speed array based on position and time arrays            
def compute_speed(dataPos, dataTime): #speed only computed along X axis. Compute along X AND Y axis?
    rawdata_speed = {}
    deltaXPos = (np.diff(dataPos))
    deltaTime = (np.diff(dataTime))
    rawdata_speed = np.divide(deltaXPos, deltaTime)
    rawdata_speed = np.append(rawdata_speed, 0)
    return rawdata_speed.astype('float32')# type of the array has to be modified to float32 (what it should be anyways, or we'll have problems doing some operations on this array), i don't know why but it's set up as 'O' type object by default. #stackexchange says that it might be linked to arrays not having the same size, or wrong dat aformat, but I've not yet seen anything strange. Converting works for now, so let's go with that. 

#group bin data by reward%
def poolByReward(data, proba, blocks, rewardproba):
    output = []
    for i in range(0, len(blocks)):
        if rewardproba[i] == proba:
            if len(data) == 1:
                output.append(data[0][i])
            if len(data) == 2: #usually for data like dataLeft+dataRight
                output.append(data[0][i]+data[1][i])
            if len(data) > 2:
                print("too much data, not intendend")
    return output

#util func to print in a given color
def colorprint(text, color, backgroundcolor = None):#print text with custom colors (text+background) colorprint("texxxxt", (R, G, B), (R, G, B)), with R, G, B [0-1]
    if isinstance(text, str) and isinstance(color, tuple):
        if  any(v > 1 or v < 0 for v in color): print("RGB must be [0-1] not [0-255]")
        else :
            if backgroundcolor == None:
                return "\033[38;2;{0};{1};{2}m{3}".format(int(color[0]*255), int(color[1]*255), int(color[2]*255), text) #takes RGB values [0:255]
            elif backgroundcolor != None:
                return "\033[38;2;{0};{1};{2}m\033[48;2;{3};{4};{5}m{6}".format(int(color[0]*255), int(color[1]*255), int(color[2]*255), int(backgroundcolor[0]*255), int(backgroundcolor[1]*255), int(backgroundcolor[2]*255), text) #takes RGB values [0:255]
    else: print("error: check input type(")

#util funct to print a progress bar
def update_progress(progress):
    animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
    barLength = 50

    def status_update(progress):
        status = ""
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "Error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Something's WRONG...\r\n"
        if progress < 1 and progress >= 0:
            status = "Computing..."
        if progress >= 0.999:
            progress = 1
            status = 'Done ✓ \r'
        return status
        
    #text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "~~~(___C'>" + "-"*(barLength-block), int(round(progress*100)), progressstatus(progress, session))
    text = ("\r" + animalList[0] +"{0}".format( " "*int(round(barLength * progress[0])) + "          __QQ" + " "*(barLength-int(round(barLength * progress[0]))))+ " "*180 + "Progress: [{0}] {1}% {2}".format( " "*int(round(barLength * progress[0])) + " ~~~(_)_\">" + " "*(barLength-int(round(barLength * progress[0]))), int(round(progress[0]*100)), status_update(progress[0]))+
            "\n\n" + animalList[1] +"{0}".format( " "*int(round(barLength * progress[1])) + "          __QQ" + " "*(barLength-int(round(barLength * progress[1]))))+ " "*180 + "Progress: [{0}] {1}% {2}".format( " "*int(round(barLength * progress[1])) + " ~~~(_)_\">" + " "*(barLength-int(round(barLength * progress[1]))), int(round(progress[1]*100)), status_update(progress[1]))+
            "\n\n" + animalList[2] +"{0}".format( " "*int(round(barLength * progress[2])) + "          __QQ" + " "*(barLength-int(round(barLength * progress[2]))))+ " "*180 + "Progress: [{0}] {1}% {2}".format( " "*int(round(barLength * progress[2])) + " ~~~(_)_\">" + " "*(barLength-int(round(barLength * progress[2]))), int(round(progress[2]*100)), status_update(progress[2]))+
            "\n\n" + animalList[3] +"{0}".format( " "*int(round(barLength * progress[3])) + "          __QQ" + " "*(barLength-int(round(barLength * progress[3]))))+ " "*180 + "Progress: [{0}] {1}% {2}".format( " "*int(round(barLength * progress[3])) + " ~~~(_)_\">" + " "*(barLength-int(round(barLength * progress[3]))), int(round(progress[3]*100)), status_update(progress[3]))+
            "\n\n" + animalList[4] +"{0}".format( " "*int(round(barLength * progress[4])) + "          __QQ" + " "*(barLength-int(round(barLength * progress[4]))))+ " "*180 + "Progress: [{0}] {1}% {2}".format( " "*int(round(barLength * progress[4])) + " ~~~(_)_\">" + " "*(barLength-int(round(barLength * progress[4]))), int(round(progress[4]*100)), status_update(progress[4]))+
            "\n\n" + animalList[5] +"{0}".format( " "*int(round(barLength * progress[5])) + "          __QQ" + " "*(barLength-int(round(barLength * progress[5]))))+ " "*180 + "Progress: [{0}] {1}% {2}".format( " "*int(round(barLength * progress[5])) + " ~~~(_)_\">" + " "*(barLength-int(round(barLength * progress[5]))), int(round(progress[5]*100)), status_update(progress[5]))+"\n\n")
    sys.stdout.write(text)
    sys.stdout.flush()

#select specified session among all the session in datapath
def matchsession(animal, sessionlist): 
    list = [session for session in sessionlist if animal == session[0:6]]
    return(list)

###------------------------------------------------------------------------------------------------------------------
### INDIVIDUAL FIGURES
###------------------------------------------------------------------------------------------------------------------

# This function plots the base trajectory of the rat. Parameters are time : time data, position : X position data, lickL/R, lick data, maxminstep for x and y axis, color and marker of the plot, width of the axis, and x y labels
def plot_BASEtrajectory(time, position, lickLeft, lickRight, maxminstep, maxminstep2, color = [], marker = [], linewidth = [], xyLabels=["N","Bins"]):   
    #select data that we want to plot
    plt.plot(time, position, color=color[0], marker=marker[0], linewidth = linewidth[0])#easy time againt position of the animal
    plt.plot(time, [None if x == 0 else x for x in lickLeft], color=color[1], marker=marker[1], markersize = marker[2])#lick data, plot position in which the animal licked, NoLick = we don't care so None
    plt.plot(time, [None if x == 0 else x for x in lickRight], color=color[1], marker=marker[1], markersize = marker[2])
    #configure plot
    traj = plt.gca()
    #configure ax xy span
    traj.set_xlim(maxminstep[0]-maxminstep[2],maxminstep[1]+maxminstep[2])
    traj.set_ylim(maxminstep2[0]-maxminstep2[2],maxminstep2[1]+maxminstep2[2])
    #configure xy style
    traj.set_xlabel(xyLabels[1],fontsize=12,labelpad=0)
    traj.set_ylabel(xyLabels[0],fontsize=12,labelpad=-1)
    traj.xaxis.set_ticks_position('bottom')
    traj.yaxis.set_ticks_position('left')
    traj.get_xaxis().set_tick_params(direction='out',pad=2)
    traj.get_yaxis().set_tick_params(direction='out',pad=2)
    #remove unnecessary axes
    traj.spines['top'].set_color("none")
    traj.spines['right'].set_color("none")
    return traj

def plot_BASEtrajectoryV2(animal, session, time, goodPos, badPos, lickL, lickR, rewardProbaBlock, blocks, barplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):
    ax1 = plt.gca()
    for i in range(0, len(blocks)):
        plt.axvspan(blocks[i][0], blocks[i][1], color='grey', alpha=rewardProbaBlock[i]/250, label = "%reward: " + str(rewardProbaBlock[i]) if (i == 0 or i == 1) else "")
    plt.plot(time, goodPos, label = "run", color="dodgerblue", linewidth = 1)
    plt.plot(time, badPos, label = "wait", color="orange", linewidth = 1)
    plt.plot(time, [None if x == 0 else x for x in lickL], color= "b", marker="o", markersize = 1)#lick data, plot position in which the animal licked, NoLick = we don't care so None
    plt.plot(time, [None if x == 0 else x for x in lickR], color= "b", marker="o", markersize = 1)
    
    ax1.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
    ax1.set_ylabel(xyLabels[2], fontsize = xyLabels[6])
    ax1.set_xlim([barplotaxes[0], barplotaxes[1]])
    ax1.set_ylim([barplotaxes[0], barplotaxes[3]+30])
    ax1.spines['bottom'].set_linewidth(linewidth[0])
    ax1.spines['left'].set_linewidth(linewidth[0])
    ax1.spines['top'].set_color("none")
    ax1.spines['right'].set_color("none")
    ax1.tick_params(width=2, labelsize = xyLabels[7])
    #x_ticks = np.arange(1800, 3300, 300)
    #ax1.set_xticks(x_ticks)
    #ax1.set_xticklabels([int(val / 60) for val in ax1.get_xticks().tolist()])
    return ax1         

#plot old boundaries run stay
def plot_peak(data, animal, session, leftBoundaryPeak, rightBoundaryPeak, kde, maxminstep, maxminstep2, marker = [], xyLabels=["N","Bins"]):
    #fig, ax = plt.subplots(figsize=(3,6))
    bins = np.arange(120)
    xx = np.linspace(0, 120, 120)
    xline1 = [leftBoundaryPeak, leftBoundaryPeak]
    xline2 = [rightBoundaryPeak, rightBoundaryPeak]
    border = 5
    yline = [0, 0.01]
    peak = plt.gca()
    if platform.system()== 'Darwin':
        peak.hist(data, normed=True, bins=bins, alpha=0.3, orientation='horizontal') #bugged on linux, working on mac
    #plot kde + boundaries
    peak.plot(kde(xx), xx, color = 'r')
    peak.plot(yline, xline1, ":", color='k')
    peak.plot(yline, xline2, ":", color='k')
    peak.plot(yline, [xline1[0]+border, xline1[0]+border], ":", color='k', alpha = 0.5)
    peak.plot(yline, [xline2[0]-border, xline2[0]-border], ":", color='k', alpha = 0.5)
    #configure plot
    peak.set_xlim(maxminstep[0]-maxminstep[2],maxminstep[1]+maxminstep[2])
    peak.set_ylim(maxminstep2[0]-maxminstep2[2],maxminstep2[1]+maxminstep2[2])
    peak.set_xlabel(xyLabels[1],fontsize=12,labelpad=0)
    peak.set_ylabel(xyLabels[0],fontsize=12,labelpad=-1)
    peak.spines['top'].set_color("none")
    peak.spines['left'].set_color("none")
    peak.spines['right'].set_color("none")
    peak.yaxis.set_ticks_position('left')
    peak.xaxis.set_ticks_position('bottom')
    peak.get_xaxis().set_tick_params(direction='out',pad=2)
    peak.get_yaxis().set_tick_params(direction='out',pad=2)
    peak.axes.get_yaxis().set_visible(False)
    return peak

#function to plot the tracks of each runs and stays
def plot_tracks(animal, session, posdataRight, timedataRight, bounds, xylim, color, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1): 
    tracksplot = plt.gca()
    tracksplot.set_title(title[0], fontsize = title[1], pad=50)
    for i, j in zip(posdataRight, timedataRight):
        plt.plot(np.subtract(j, j[0]), i, color = color[0], linewidth=0.3, label="Good Item" if i == posdataRight[0] else "")
    tracksplot.set_title(title[0], fontsize = title[1])
    tracksplot.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    tracksplot.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    tracksplot.set_xlim([xylim[0], xylim[1]])
    tracksplot.set_ylim([xylim[2], xylim[3]])
    tracksplot.spines['top'].set_color("none")
    tracksplot.spines['right'].set_color("none")
    xline1 = [bounds[0], bounds[0]]
    xline2 = [bounds[1], bounds[1]]
    yline = [0,20]
    plt.plot(yline, xline1, ":", color='k')
    plt.plot(yline, xline2, ":", color='k')
    tracksplot.legend()
    return tracksplot

#function to plot the cumulative distribution of the run speeds and stay times        
def cumul_plot(dataRight, dataLeft, barplotaxes, maxminstepbin, scatterplotaxes, legend, color, xyLabels=["","","",""], title=[None], linewidth = 1):      
    cumul = plt.gca()
    custom_legend = [Line2D([0], [0], color = color[0], lw=legend[3]), Line2D([0], [0], color = color[1], lw=legend[2])]
    plt.hist(dataRight, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(dataRight)/float(len(dataRight)), color = color[0], histtype = 'step', cumulative = True, linewidth = legend[3])
    plt.hist(dataLeft, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(dataLeft)/float(len(dataLeft)), color =color[1], histtype = 'step', cumulative = True, linewidth = legend[2])
    cumul.set_title(title[0], fontsize = title[1], pad=50)
    cumul.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    cumul.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    cumul.set_xlim([barplotaxes[0], barplotaxes[1]])
    cumul.set_ylim([barplotaxes[2], barplotaxes[3]])
    cumul.spines['bottom'].set_linewidth(linewidth[0])
    cumul.spines['left'].set_linewidth(linewidth[0])
    cumul.spines['top'].set_color("none")
    cumul.spines['right'].set_color("none")
    cumul.tick_params(width=2, labelsize = xyLabels[2]) 
    #plt.tight_layout(pad=0.5)
    cumul.legend(custom_legend, [legend[0], legend[1]], bbox_to_anchor = (0., 1.02, 1., .102), loc='lower left', mode="expand", borderaxespad=0., frameon = False)
    return cumul

#function to plot the scatter data of run times and stay times
def distribution_plot(dataRight, dataLeft, barplotaxes, maxminstepbin, scatterplotaxes, legend, color, xyLabels=["","","",""], title=[None], linewidth = 1):
    distr = plt.gca()
    plt.scatter(np.random.normal(1, 0.05, len(dataRight)), dataRight, s=20, color = color[0], marker = "$\u25ba$", label=legend[0])
    plt.scatter(np.random.normal(2, 0.05, len(dataLeft)), dataLeft, s=20, color = color[1], marker = "$\u25c4$", label=legend[1])

    plt.scatter(1.2, np.mean(dataRight), s=25, color = color[0])
    plt.scatter(2.2, np.mean(dataLeft), s=25, color = color[1])
    plt.boxplot(dataRight, positions = [1.35])
    plt.boxplot(dataLeft, positions = [2.35])
    distr.set_xlabel(xyLabels[1], fontsize = xyLabels[4])
    distr.set_ylabel(xyLabels[0], fontsize = xyLabels[4])
    distr.set_title(title[0], fontsize = title[1], pad=50)
    distr.set_xlim([scatterplotaxes[0], scatterplotaxes[1]])
    distr.set_ylim([scatterplotaxes[2], scatterplotaxes[3]])
    distr.set_xticks([1,2])
    distr.set_xticklabels([xyLabels[2],xyLabels[3]], fontsize = xyLabels[5])
    distr.spines['bottom'].set_linewidth(linewidth[0])
    distr.spines['left'].set_linewidth(linewidth[0])
    distr.spines['top'].set_color("none")
    distr.spines['right'].set_color("none")
    distr.tick_params(width=2, labelsize = xyLabels[5])
    handles, labels = distr.get_legend_handles_labels()
    distr.legend()
    #distr.legend([handles[0], handles[1],], [legend[0], legend[2], legend[1], legend[3]], bbox_to_anchor = (0., 1.02, 1., .102), loc='lower left', ncol=2, mode="expand", borderaxespad=0., frameon = False)
    return distr

#plot a blank plot to have some space
def blank_plot():
    blk = plt.gca()
    blk.axis('off') 

#plot rat speed along run
def plot_speed(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    for i, j in zip(posdataRight, timedataRight):
        time = np.subtract(j, j[0])
        iabs =  [abs(ele) for ele in i]
        plt.plot(np.subtract(j, j[0]), iabs, color='g' , linewidth=0.3, label="Good Item" if i == posdataRight[0] else "")
        if len(np.where(i == max(i))[0]) == 1:
            maxspeed = max(iabs)
            maxspeedtime = np.where(iabs == maxspeed)[0]
            plt.scatter(time[maxspeedtime], maxspeed, color ='darkgreen', s=20)
        else: print(max(i))
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    xline1 = [bounds[0], bounds[0]]
    xline2 = [bounds[1], bounds[1]]
    yline = [0,20]
    plt.plot(yline, xline1, ":", color='k')
    plt.plot(yline, xline2, ":", color='k')
    ax.legend()
    return ax

#plot per block
def plot_figBin(data, rewardProbaBlock, blocks, barplotaxes, color, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):
    warnings.simplefilter("ignore", category=RuntimeWarning)
    binplot = plt.gca()
    for i in range(0, len(blocks)):
        plt.axvspan(blocks[i][0]/60, blocks[i][1]/60, color='grey', alpha=rewardProbaBlock[i]/250, label = "%reward: " + str(rewardProbaBlock[i]) if (i == 0 or i == 1) else "")
        plt.scatter(np.full((1, 1), (blocks[i][1] + blocks[i][0])/120), data[i], s=50, color = color[0])
        for i in range(0, len(blocks)-1):
            plt.plot(((blocks[i][1] + blocks[i][0])/120, (blocks[i+1][1] + blocks[i+1][0])/120), (data[i], data[i+1]), color = color[0], linewidth = 1)
    binplot.set_title(title[0], fontsize = title[1])
    binplot.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    binplot.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    binplot.set_xlim([barplotaxes[0], barplotaxes[1]])
    binplot.set_ylim([barplotaxes[2], barplotaxes[3]])
    binplot.spines['bottom'].set_linewidth(linewidth[0])
    binplot.spines['left'].set_linewidth(linewidth[0])
    binplot.spines['top'].set_color("none")
    binplot.spines['right'].set_color("none")
    binplot.tick_params(width=2, labelsize = xyLabels[3])
    return binplot

#plot block per %reward
def plot_figBinMean(ax, dataLeft, dataRight, color, ylim):
    warnings.simplefilter("ignore", category=RuntimeWarning)
    two_groups_unpaired = dabest.load(pd.DataFrame(dict([(k,pd.Series(v)) for k,v in {"10%\nreward" : dataLeft, "90%\nreward" : dataRight}.items()])), idx=("10%\nreward", "90%\nreward"), resamples=5000)
    dabestplot = two_groups_unpaired.mean_diff.plot(ax = ax, raw_marker_size=7, es_marker_size=5, custom_palette= {"10%\nreward" : color[0], "90%\nreward" : color[0]}, group_summaries='mean_sd', group_summary_kwargs={'lw':3, 'alpha':0.8}, reflines_kwargs= {'linestyle':'dashed', 'linewidth':0.8, 'color' : 'black'}, swarm_ylim=(ylim), swarm_label = "", contrast_label = "mean difference", halfviolin_alpha=0.5, violinplot_kwargs={'widths':0.5});
    ax.axvspan(-0.5, 0.5, color='grey', alpha = 10/250) #when plot is ax, swarm is ax, contrast is ax.contrast_axes..
    ax.axvspan( 0.5, 1.5, color='grey', alpha = 90/250)
    return ax


###------------------------------------------------------------------------------------------------------------------
### DATA PROCESSING FUNCTIONS
###------------------------------------------------------------------------------------------------------------------

# Old function to compute start of run and end of run boundaries
def extract_boundaries(data, animal, session, dist, height = None):
    # animals lick in the extremities, so they spend more time there, so probability of them being there is more important than the probability of being in the middle of the apparatus. we compute the two average positions of these resting points. We defined a run as the trajectory of the animal between the resting points. So we have to find these resting points. In later stages of the experiments the start/end of runs is defined based on the speed of the animals. 
    # function params : data is X position array for the session that we analyse, height = parameter to define a limit to the probability of detecting a place as significantly more probable than another.
    #We use a KDE (Kernel Density Estimate) to find said places. See testhitopeak.ipynb 2nd method for histogram. histogram is coded in next cell between """ """, but does not work on linux
    kde = stats.gaussian_kde(data)
    #compute KDE = get the position probability curve and compute peaks of the curve
    peak_pos, peak_height = [], []
    nb_samples = 120 #played a bit with the values, this works (number of data bins, we chose 1 per cm, also tested 10 bins per cm)
    samples = np.linspace(0, 120, nb_samples)
    probs = kde.evaluate(samples)
    maxima_index = find_peaks(probs, height) 
    peak_pos = maxima_index[0]
    peak_height = maxima_index[1]["peak_heights"]
    #print("values", peak_pos, peak_height)
    # if there is more than two peaks (e.g. an animal decides to stay in the middle of the treadmill), only keep the two biggest peaks (should be the extremities) and remove the extra peak/s if there is one or more
    peak_posLeft, peak_heightLeft, peak_posRight, peak_heightRight = [], [], [], []
    for i, j in zip(peak_pos, peak_height):
        if i < dist/2:
            peak_posLeft.append(i)
            peak_heightLeft.append(j)
        if i > dist/2:
            peak_posRight.append(i)
            peak_heightRight.append(j)
    leftBoundaryPeak = peak_posLeft[np.argmax(peak_heightLeft)]
    rightBoundaryPeak = peak_posRight[np.argmax(peak_heightRight)]
    #print("computing bounds", animal, leftBoundaryPeak, rightBoundaryPeak) 
    return leftBoundaryPeak, rightBoundaryPeak, kde

#convert scale, convert i=0:120 to 60:-60 which correspnds to the speed to the right (0 to 60) and to the left (0 to -60)
def convert_scale(number): 
    old_min=0
    old_max=120
    new_max=-60
    new_min=60
    return int(((number- old_min) / (old_max - old_min)) * (new_max - new_min) + new_min)

# compute mask to separate runs and stays based on speed
def filterspeed(animal, session, dataPos, dataSpeed, dataTime, threshold, dist):
    #dissociate runs from non runs, we want a cut off based on animal, speed. How to define this speed? If we plot the speed of the animal in function of the X position in the apparatus, so we can see that there is some blobs of speeds close to 0 and near the extremities of the treadmill, these are the ones that we want to define as non running speeds. With this function we want to compute the area of these points of data (higher density, this technique might not work when animals are not properly trained) in order to differentiate them.
    middle = dist/2                    
    xmin, xmax = 0, 120 #specify the x and y range of the window that we want to analyse
    ymin, ymax =-60, 60
    position = np.array(dataPos, dtype = float) #data needs to be transformed to float perform the KDE
    speed = np.array(dataSpeed, dtype = float)
    time = np.array(dataTime, dtype = float)
    X, Y = np.mgrid[xmin:xmax:120j, ymin:ymax:120j]#create 2D grid to compute KDE
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = np.vstack([position, speed])
    kernel = stats.gaussian_kde(values)#compute KDE, this gives us an estimation of the point density
    Z = np.reshape(kernel(positions).T, X.shape)
    #Using the KDE that we just computed, we select the zones that have a density above a certain threshold (after testing 0.0001 seems to work well), which roughly corresponds to the set of data that we want to extract.
    # We loop through the 2D array, if the datapoint is > threshold we get the line (speed limit) and the row (X position in cm). This gives us the speed limit for each part of the 
    # treadmill, so basically a zone delimited with speed limits (speed limits can be different in different points of the zone).
    i, j = [], [] #i is the set of speeds (lines) for which we will perform operations, j is the set of positions (rows) for each speed for which we will perform operations
    for line in range(0, len(np.rot90(Z))):
        if len(np.where(np.rot90(Z)[line]> threshold)[0]) > 1:
            i.append(convert_scale(line))
            j.append(np.where(np.rot90(Z)[line] > threshold)[0])
    #create a mask using the zone computed before and combine them. We have two zones (left and right), so we perform the steps on each side, first part is on the left.
    rawMask = np.array([])
    for line, pos in zip(i, j): #pos is the array of positions for which the speed of the animal is under the speed limit. "11 [ 7  8  9 10 11]" for instance here the speed limit is 11 cm/s, and is attained between 7 and 11cm on the treadmill. "10 [  4   5   6   7   8   9  10  11  12  13 105 106 107 108 109 110]" when we decrease the speed limit, here 10, we here have 2 zones, one between 4 and 13cm and another between 105 and 110cm. we continue through these speed values (roughly from 11 to 0, then from 0 to -10 in this example)
        if pos[pos<middle].size:
            low = pos[pos<middle][0]#first value of the array, explained above (7)
            high = pos[pos<middle][-1]#last value of the array (11)
            a = np.ma.masked_less(position, high)#take everything left of the rightmost point, if the value is ok == True
            b = np.ma.masked_greater(position,low)#take everything right of the leftmost point
            c = np.ma.masked_less(speed, line+0.5)#take everything below the high point
            d = np.ma.masked_greater(speed,line-0.5)#take everything above the low point
            mask = np.logical_and(a.mask,b.mask)#first combination for all the rows (=Xposition), keep the intersection of mask a AND b, so keep all between the leftmost and rightmost points
            mask2 = np.logical_and(c.mask, d.mask)#second combination for all the lines (=speed), keep the intersection of mask c AND d, so keep all between speed+0.5:speed-0.5
            combiLeft = np.logical_and(mask, mask2)# combine the first and the second mask, so we only keep the intersection of the two masks, intersection is TRUE, the rest is FALSE
            if not rawMask.size:#must do that for the first iteration so it's not empty
                rawMask = combiLeft
            else : rawMask = np.logical_xor(combiLeft, rawMask) # merge the newly computed mask with the previously computed masks. We use XOR so that the TRUE values of the new mask are added to the complete mask. Same step left and right just add to the existing full mask wether the new mask is on the left or the right. 
        #same as above for the right part
        if pos[pos>middle].size:
            low = pos[pos>middle][0]
            high = pos[pos>middle][-1]
            a = np.ma.masked_less(position, high)
            b = np.ma.masked_greater(position,low)
            c = np.ma.masked_less(speed, line+0.5)
            d = np.ma.masked_greater(speed,line-0.5)
            mask = np.logical_and(a.mask,b.mask)
            mask2 = np.logical_and(c.mask, d.mask)
            combiRight = np.logical_and(mask, mask2)
            if not rawMask.size:
                rawMask = combiRight
            else : rawMask = np.logical_xor(combiRight, rawMask)
    return ~rawMask

# Mask smoothing, what it does is if we have a small part of running in either sides between parts of not running, we say that this is not running and modify the mask. So we have to set up all possible cases and generate an appropriate response, but in all cases encountered these problems were only in the waiting times and not in running. 
def removeSplits_Mask(inputMask, inputPos, animal, session, dist):
    correctedMask = [list(val) for key, val in groupby(inputMask[animal, session], lambda x: x == True)]
    splitPos = []
    middle = (dist)/2   
    count = [0, 0, 0, 0, 0, 0]
    start, end = 0, 0
    for elem in correctedMask:
        start = end
        end = start + len(elem)
        splitPos.append(inputPos[animal,session][start:end])
    for m, p in zip(correctedMask, splitPos):
        if p[0] < middle and p[-1] < middle:
            if m[0] == False: pass #print("in L")
            if m[0] == True: #print("bug")
                correctedMask[count[5]] = [False for val in m]
                count[0] += 1
            count[5] += 1 
            #print(m, p, "all left")  
        elif p[0] > middle and p[-1] > middle:
            if m[0] == False: pass #print("in R")
            if m[0] == True: #print("bug")
                correctedMask[count[5]] = [False for val in m]
                count[1] += 1
            count[5] += 1 
            #print(m, p, "all right")
        elif p[0] < middle and p[-1] > middle:
            if m[0] == True: pass #print("runLR")
            if m[0] == False:
                #print("bug")
                correctedMask[count[5]] = [True for val in m]
                count[2] += 1
            count[5] += 1 
            #print(m, p, "left right")
        elif p[0] > middle and p[-1] < middle:
            if m[0] == True: pass #print("runRL")
            if m[0] == False: #print("bug")
                correctedMask[count[5]] = [True for val in m]
                count[3] += 1
            count[5] += 1 
            #print(m, p, "right left")
        else: #print(m,p,"bbb")
            count[4] += 1 
            count[5] += 1
    #print(count)
    return np.concatenate(correctedMask)

# separate runs/stays * left/right + other variables into dicts
def extract_runSpeedBin(dataPos, dataSpeed, dataTime, dataLickR, dataLickL, mask, animal, session, blocks, boundary, treadmillspeed):
    runs = {}
    stays = {}
    runs[animal, session] = {}
    stays[animal, session] = {}
    position, speed, time, goodPos, badPos, goodSpeed, badSpeed, goodTime, badTime = ({bin : [] for bin in range(0, len(blocks))} for i in range(9))
    speedRunToRight, speedRunToLeft, XtrackRunToRight, XtrackRunToLeft, timeRunToRight, timeRunToLeft, timeStayInRight, timeStayInLeft, XtrackStayInRight, XtrackStayInLeft, TtrackStayInRight, TtrackStayInLeft, instantSpeedRight, instantSpeedLeft, maxSpeedRight, maxSpeedLeft, whenmaxSpeedRight, whenmaxSpeedLeft, wheremaxSpeedRight, wheremaxSpeedLeft, lick_arrivalRight, lick_drinkingRight, lick_waitRight, lick_arrivalLeft, lick_drinkingLeft, lick_waitLeft = ({bin : [] for bin in range(0, len(blocks))} for i in range(26))
    
    for i in range(0, len(blocks)):
        position[i] = np.array(dataPos[animal, session][i], dtype = float)
        speed[i] = np.array(dataSpeed[animal, session][i], dtype = float)
        time[i] = np.array(dataTime[animal, session][i], dtype = float)

        goodPos[i] = [val[0] if val[1] == False  else None for val in [[i, j] for i, j in zip(position[i], mask[animal, session][i])]]
        badPos[i]  = [val[0] if val[1] == True else None for val in [[i, j] for i, j in zip(position[i], mask[animal, session][i])]]
        goodSpeed[i] = [val[0] if val[1] == False  else None for val in [[i, j] for i, j in zip(speed[i], mask[animal, session][i])]]
        badSpeed[i]  = [val[0] if val[1] == True else None for val in [[i, j] for i, j in zip(speed[i], mask[animal, session][i])]]
        goodTime[i] = [val[0] if val[1] == False  else None for val in [[i, j] for i, j in zip(time[i], mask[animal, session][i])]]
        badTime[i]  = [val[0] if val[1] == True else None for val in [[i, j] for i, j in zip(time[i], mask[animal, session][i])]]

        stays[animal, session][i] = [[e[0], e[1], e[2], e[3], e[4]] if [e[0], e[1], e[2]] != [None, None, None] else 0 for e in [[i, j, k, l, m] for i, j, k, l, m in zip(goodPos[i], goodSpeed[i], goodTime[i], dataLickR[animal, session][i], dataLickL[animal, session][i])]]
        runs[animal, session][i] = [[e[0], e[1], e[2]] if [e[0], e[1], e[2]] != [None, None, None] else 0 for e in [[i, j, k] for i, j, k, in zip(badPos[i], badSpeed[i], badTime[i])]]

        for run in split_a_list_at_zeros(runs[animal, session][i]):
            #calculate distance run as the distance between first and last value
            distanceRun = abs(run[0][0]-run[-1][0])
            #calculate time as sum of time interval between frames
            totaltimeRun = []
            xTrackRun = []
            instantSpeed = []
            maxSpeed = []
            for item in run:
                xTrackRun.append(item[0])
                instantSpeed.append(abs(item[1])) #if no abs() here > messes with maxspeed, if-TMspeed > not clean platform + TM changes direction with rat
                totaltimeRun.append(item[2])
            if np.sum(np.diff(totaltimeRun)) != 0:
                speedRun = distanceRun/np.sum(np.diff(totaltimeRun)) - treadmillspeed[i]
                maxSpeed = max(instantSpeed) - treadmillspeed[i]
                wheremaxSpeed = xTrackRun[np.argmax(instantSpeed)] - xTrackRun[0] if xTrackRun[0] < xTrackRun[np.argmax(instantSpeed)] else xTrackRun[0] - xTrackRun[np.argmax(instantSpeed)]
                whenmaxSpeed = np.sum(np.diff(totaltimeRun[0:np.argmax(instantSpeed)])) #totaltimeRun[np.argmax(instantSpeed)]-
                #check if the subsplit starts on the left or the right -> determine if the animal is running left or right
                if run[0][0] < ((boundary[0]+boundary[1])/2):
                    #check if the subsplit is ending on the other side -> determine if this is a run 
                    if run[-1][0] > ((boundary[0]+boundary[1])/2):
                        speedRunToRight[i].append(speedRun)
                        XtrackRunToRight[i].append(xTrackRun)
                        timeRunToRight[i].append(totaltimeRun)
                        instantSpeedRight[i].append(instantSpeed)
                        maxSpeedRight[i].append(maxSpeed)
                        wheremaxSpeedRight[i].append(wheremaxSpeed)
                        whenmaxSpeedRight[i].append(whenmaxSpeed)
                #same thing for the runs that go to the other side
                elif run[0][0]>((boundary[0]+boundary[1])/2):
                    if run[-1][0]<((boundary[0]+boundary[1])/2):
                        speedRunToLeft[i].append(speedRun)
                        XtrackRunToLeft[i].append(xTrackRun)
                        timeRunToLeft[i].append(totaltimeRun)
                        instantSpeedLeft[i].append(instantSpeed)
                        maxSpeedLeft[i].append(maxSpeed)
                        wheremaxSpeedLeft[i].append(wheremaxSpeed)
                        whenmaxSpeedLeft[i].append(whenmaxSpeed)

        for stay in split_a_list_at_zeros(stays[animal, session][i]):
            tInZone = []
            xTrackStay = []
            lickR = []
            lickL = []
            for item in stay:
                xTrackStay.append(item[0])
                tInZone.append(item[2])
                lickR.append(item[3])
                lickL.append(item[4])
            totaltimeStay = np.sum(np.diff(tInZone))
            #first identify if the subsplit created is on the left or right by comparing to the middle
            if stay[0][0] > ((boundary[0]+boundary[1]) / 2):
                #if hasLick == True:
                if not all(v == 0 for v in lickR):
                    pre = []
                    drink = []
                    post = []
                    for t, l in zip(tInZone[0:np.min(np.nonzero(lickR))], lickR[0:np.min(np.nonzero(lickR))]): pre.append(t)
                    for t, l in zip(tInZone[np.min(np.nonzero(lickR)):np.max(np.nonzero(lickR))], lickR[np.min(np.nonzero(lickR)):np.max(np.nonzero(lickR))]): drink.append(t)
                    for t, l in zip(tInZone[np.max(np.nonzero(lickR)):-1], lickR[np.max(np.nonzero(lickR)):-1]): post.append(t)
                    #drink <- dig in that later on to have more info on lick (lick rate, number of licks, etc.)
                    lick_arrivalRight[i].append(np.sum(np.diff(pre)))
                    lick_drinkingRight[i].append(np.sum(np.diff(drink)))
                    lick_waitRight[i].append(np.sum(np.diff(post)))
                timeStayInRight[i].append(totaltimeStay)
                XtrackStayInRight[i].append(xTrackStay)
                TtrackStayInRight[i].append(tInZone)
        
            elif stay[0][0] < ((boundary[0] + boundary[1]) / 2):
                #if hasLick == True:
                if not all(v == 0 for v in lickL):
                    preL = []
                    drinkL = []
                    postL = []
                    for t, l in zip(tInZone[0:np.min(np.nonzero(lickL))], lickR[0:np.min(np.nonzero(lickL))]): preL.append(t)
                    for t, l in zip(tInZone[np.min(np.nonzero(lickL)):np.max(np.nonzero(lickL))], lickL[np.min(np.nonzero(lickL)):np.max(np.nonzero(lickL))]): drinkL.append(t)
                    for t, l in zip(tInZone[np.max(np.nonzero(lickL)):-1], lickL[np.max(np.nonzero(lickL)):-1]): postL.append(t)
                    lick_arrivalLeft[i].append(np.sum(np.diff(preL)))
                    lick_drinkingLeft[i].append(np.sum(np.diff(drinkL)))
                    lick_waitLeft[i].append(np.sum(np.diff(postL)))
                timeStayInLeft[i].append(totaltimeStay)
                XtrackStayInLeft[i].append(xTrackStay)
                TtrackStayInLeft[i].append(tInZone)

    return speedRunToRight, speedRunToLeft, XtrackRunToRight, XtrackRunToLeft, timeRunToRight, timeRunToLeft, timeStayInRight, timeStayInLeft, XtrackStayInRight, XtrackStayInLeft, TtrackStayInRight, TtrackStayInLeft, instantSpeedRight, instantSpeedLeft, maxSpeedRight, maxSpeedLeft, whenmaxSpeedRight, whenmaxSpeedLeft, wheremaxSpeedRight, wheremaxSpeedLeft, lick_arrivalRight, lick_drinkingRight, lick_waitRight, lick_arrivalLeft, lick_drinkingLeft, lick_waitLeft
    
# due to the way blocks are computed, some runs may have started in block[n] and ended in block [n+1], this function appends the end of the run to the previous block. See reCutBins.
def fixSplittedRunsMask(animal, session, input_Binmask, blocks):
    output_Binmask = copy.deepcopy(input_Binmask)
    for i in range(1, len(blocks)): #print(input_Binmask[i-1][-1], input_Binmask[i][0])
        if not all(v == False for v in output_Binmask[i]): #if animal did not do any run (only one stay along the whole block) don't do the operation
            if output_Binmask[i-1][-1] == True and output_Binmask[i][0] == True: #print(i, "case1")
                #print(session, i-1, i) #uncomment to see whether/which bins have had fixes. 
                while output_Binmask[i][0] == True:
                    output_Binmask[i-1] = np.append(output_Binmask[i-1], output_Binmask[i][0])
                    output_Binmask[i]   = np.delete(output_Binmask[i], 0)
            if output_Binmask[i-1][-1] == False and output_Binmask[i][0] == False: #print(i, "case2")
                #print(session, i-1, i)
                while output_Binmask[i][0] == False:
                    output_Binmask[i-1] = np.append(output_Binmask[i-1], output_Binmask[i][0])
                    output_Binmask[i]   = np.delete(output_Binmask[i], 0)
    return output_Binmask

# following fixSplittedRunsMask, this function re cuts the bins of a variable at the same Length as the fixed binMask we just computed. 
def reCutBins(data_to_cut, data_template):
    output={}
    start_of_bin = 0
    for i, bin in enumerate(data_template):
        end_of_bin = start_of_bin + len(data_template[i])
        output[i] = data_to_cut[start_of_bin : end_of_bin]
        start_of_bin = end_of_bin
    return output

#function to stitch together all the bins of a variable to form the full session variable.
def stitch(input):
    dataSession = copy.deepcopy(input)
    for i, data in enumerate(input):
        dataSession[i] = list(itertools.chain.from_iterable(list(data.values())))
    return dataSession
 

### LOAD DATA
#Data is loaded in one dictionary per variable (time, position, etc.) and have [animal, session] as keys, so we have a set of data for each session of each rat
#This cell loads all the behavioral data from the files created with labview
#data in .position is as follows (as of feb 2021): 
#[0:2]: Step of the experiment, e.g. "Session 1 ON". Takes 3 columns because of the spaces.
#[3]: Time at which the LV iteration has been done. Depends on the acquisition frequency, clock adds 0.04s at each iteration. 
#[4]: POSITION DATA : Position on the X axis (length of the treadmill) of the animal's centroid. This position is in pixels, it needs to be converted into cm using the datapxtocm function. The conversion (11px = 1cm) has been determined in LabView as a result of a calibration to counteract the lens distorsion generated with the camera.
#[5]: Same for the position on the Y axis. 
#[6]: LICK DATA : Left hand side of the treadmill. True/1 if there is a lick False/0 if not.
#[7]: Same but Right side
#[8]: SOLENOID DATA : Same idea as just above, Left Side, but determines if the valve has been opened (might be usefull in %to get reward protocol)
#[9]: Same but Right Side valve.
#[10]: BUFFER DATA : #ID of the frame that has been analyzed by LabView in this iteration, used to check if there is acquisition bugs, difference between two consecutive frames should be == 1.
#[11]: EDITED TRACKING : True/False or 1/0 that tells wether the frame has been edited in LabView. When the animal goes too far into the sides, or when the animal rears, the tracking loses the position of the animal and places the centroid precisely at the center of the tracking area. When this happens I coded LabView to maintain the position of the centroid in the last position in which the rat was detected and resume adequate tracking when available. TLDR, eliminates the jitter when tracking is not working properly. 

### LOAD PARAMETERS
#Parameters are loaded from behav_param file for each [animal, session] in a Dict
#sessionDuration : duration of the session 
#acqPer : number of acquisition per second in labview // not used anymore (acqPer -> delta between Time value of two consecutive frames)
#waterLeft : (weight of the measuring cup + 10 drops of water - weight of the cup) --> Weight of 10 drops on the Left, in grams. divide by 10 to have weight of a drop, multiply by 1000 to have value in µL  
#waterRight : idem on right  
#weight : Weight of each rat just before the session 
#lastWeightadlib : Weight of each rat when the water bottles are removed
#lastDayadlib : Last day with ad lib water
#daysSinceadlib : compute the number of days have elapsed since "lastDayadlib"
#boundaries : initialise the boundary array, that is computed later
#bloc duration data
#probability of getting reward on each side for eeach bloc
#treadmill speed for each block

In [ ]:
def processData(arr, ID, sessionIN, index, redoCompute = False, redoFig = False, printFigs = False, redoMask=False):
    index = index
    animal = ID 
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### INIT
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # initialise all Var dicts
    params, rat_markers, water = {}, {}, {}
    extractTime, extractPositionX, extractPositionY, extractLickLeft, extractLickRight, framebuffer, solenoid_ON_Left, solenoid_ON_Right, cameraEdit = ({} for i in range(9)) 
    rawTime, rawPositionX, rawPositionY, rawLickLeftX, rawLickRightX, rawLickLeftY, rawLickRightY, smoothMask, rawSpeed = ({} for i in range(9)) 
    binPositionX, binPositionY, binTime, binLickLeftX, binLickRightX, binSolenoid_ON_Left, binSolenoid_ON_Right = ({} for i in range(7))
    leftBoundaryPeak, rightBoundaryPeak, kde = {}, {}, {}
    smoothMask, rawMask, binSpeed, binMask = {}, {}, {}, {}
    goodPos, badPos, goodSpeed, badSpeed = {}, {}, {}, {}
    speedRunToRight,    speedRunToLeft,    XtrackRunToRight,    XtrackRunToLeft,    timeRunToRight,    timeRunToLeft,    timeStayInRight,    timeStayInLeft,    XtrackStayInRight,    XtrackStayInLeft,    TtrackStayInRight,    TtrackStayInLeft,    instantSpeedRight,    instantSpeedLeft,    maxSpeedRight,    maxSpeedLeft,    whenmaxSpeedRight,    whenmaxSpeedLeft,    wheremaxSpeedRight,    wheremaxSpeedLeft,    lick_arrivalRight,    lick_drinkingRight,    lick_waitRight,    lick_arrivalLeft,    lick_drinkingLeft,    lick_waitLeft    = ({} for i in range(26))
    speedRunToRightBin, speedRunToLeftBin, XtrackRunToRightBin, XtrackRunToLeftBin, timeRunToRightBin, timeRunToLeftBin, timeStayInRightBin, timeStayInLeftBin, XtrackStayInRightBin, XtrackStayInLeftBin, TtrackStayInRightBin, TtrackStayInLeftBin, instantSpeedRightBin, instantSpeedLeftBin, maxSpeedRightBin, maxSpeedLeftBin, whenmaxSpeedRightBin, whenmaxSpeedLeftBin, wheremaxSpeedRightBin, wheremaxSpeedLeftBin, lick_arrivalRightBin, lick_drinkingRightBin, lick_waitRightBin, lick_arrivalLeftBin, lick_drinkingLeftBin, lick_waitLeftBin = ({} for i in range(26))
    nb_runs_to_rightBin, nb_runs_to_leftBin, nb_runsBin, total_trials = {}, {}, {}, {}
    nb_rewardBlockLeft, nb_rewardBlockRight, nbWaterLeft, nbWaterRight, totalWater, totalDistance =({} for i in range(6))

    palette = [(0.55, 0.0, 0.0),  (0.8, 0.36, 0.36),   (1.0, 0.27, 0.0),   (0.0, 0.39, 0.0),    (0.13, 0.55, 0.13),   (0.2, 0.8, 0.2)]### we use RGB [0-1] not [0-255]. See www.colorhexa.com for conversion #old#palette = ['darkred', 'indianred', 'orangered', 'darkgreen', 'forestgreen', 'limegreen']
    if fnmatch.fnmatch(animal, 'RatF*'): rat_markers[animal]=[palette[index], "$\u2640$"]
    elif fnmatch.fnmatch(animal, 'RatM*'): rat_markers[animal]=[palette[index], "$\u2642$"]
    elif fnmatch.fnmatch(animal, 'Rat00*'): rat_markers[animal]=[palette[index], "$\u2426$"]
    else: print("error, this is not a rat you got here")

    if sessionIN != []: sessionList = sessionIN
    else: sessionList = sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")])
    arr[index] = 0
    time.sleep(0.1*(index+1))
    for sessionindex, session in enumerate(sessionList):
        clear_output(wait=True)
        update_progress(arr[:])
        figPath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session+os.sep+"Figures" +os.sep+"recapFIG%s.png"%session
        if redoCompute == True:

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### LOAD AND PREPROCESS DATA
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

            #extract/compute parameters from behav_params and create a parameter dictionnary for each rat and each session
            #change of behav_param format 07/2020 -> labview ok 27/07/2020 before nOk #format behavparam ? #catchup manual up to 27/07
            params[animal, session] = { "sessionDuration" : read_params(animal, session, "sessionDuration"),
                                        "acqPer" : read_params(animal, session, "acqPer"),
                                        "waterLeft" : round((read_params(animal, session, "waterLeft", valueType = float) - read_params(animal, session, "cupWeight", valueType = float))/10*1000, 2),
                                        "waterRight" : round((read_params(animal, session, "waterRight", valueType = float) - read_params(animal, session, "cupWeight", valueType = float))/10*1000, 2), 
                                        "treadmillDist" : read_params(animal, session, "treadmillSize"),      
                                        "weight" : read_params(animal, session, "ratWeight"), 
                                        "lastWeightadlib" : read_params(animal, session, "ratWeightadlib"),
                                        "lastDayadlib" : read_params(animal, session, "lastDayadlib"),
                                        "lickthresholdLeft" : read_params(animal, session, "lickthresholdLeft"), #added in Labview 2021/07/06. Now uses the custom lickthreshold for each side. Useful when lickdata baseline drifts and value is directly changed in LV. Only one session might be bugged, so this parameter is session specific. Before, the default value (300) was used and modified manually during the analysis. 
                                        "lickthresholdRight" : read_params(animal, session, "lickthresholdRight"),
                                        "realEnd" : str(read_params(animal, session, "ClockStop"))} 

            #initialize boundaries to be computed later using the KDE function
            params[animal, session]["boundaries"] = []

            #compute number of days elapsed between experiment day and removal of the water bottle
            lastDayadlib   = str(datetime.datetime.strptime(str(read_params(animal, session, "lastDayadlib")), "%Y%m%d").date())
            stringmatch    = re.search(r'\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}', session)
            experimentDay  = str(datetime.datetime.strptime(stringmatch.group(), '%Y_%m_%d_%H_%M_%S'))
            daysSinceadlib = datetime.date(int(experimentDay[0:4]), int(experimentDay[5:7]), int(experimentDay[8:10])) - datetime.date(int(lastDayadlib[0:4]), int(lastDayadlib[5:7]), int(lastDayadlib[8:10]))
            params[animal, session]["daysSinceadLib"] = daysSinceadlib.days

            #compute IRL elapsed session time
            if params[animal, session]['realEnd'] != 'None':
                startExpe = datetime.time(int(experimentDay[11:13]), int(experimentDay[14:16]), int(experimentDay[17:19]))
                endExpe   = datetime.time(hour = int(params[animal, session]['realEnd'][0:2]), minute = int(params[animal, session]['realEnd'][2:4]), second = int(params[animal, session]['realEnd'][4:6]))
                params[animal, session]["realSessionDuration"] = datetime.datetime.combine(datetime.date(1, 1, 1), endExpe) - datetime.datetime.combine(datetime.date(1, 1, 1), startExpe)
            else: params[animal, session]["realSessionDuration"] = None

            #determine block duration set based on the block timing defined in labview. 1 block in labview is comprised of a ON period and a OFF period. Max 12 blocks in LabView (12 On + 12 Off)*repeat.
            blocklist = []# raw blocks from LabView -> 1 block (ON+OFF) + etc
            for blockN in range(1,13): #13? or more ? Max 12 blocks, coded in LabView...
                #add block if  block >0 seconds then get data from file. 
                #Data from behav_params as follows: Block N°: // ON block Duration // OFF block duration // Repeat block // % reward ON // % reward OFF // Treadmill speed.
                if read_params(animal, session, "Block "+ str(blockN), dataindex =  -6, valueType = str) != 0:
                    blocklist.append([read_params(animal, session, "Block "+ str(blockN), dataindex =  -6, valueType = str), read_params(animal, session, "Block "+ str(blockN), dataindex =  -5, valueType = str), 
                                      read_params(animal, session, "Block "+ str(blockN), dataindex =  -4, valueType = str), read_params(animal, session, "Block "+ str(blockN), dataindex =  -3, valueType = str), 
                                      read_params(animal, session, "Block "+ str(blockN), dataindex =  -2, valueType = str), read_params(animal, session, "Block "+ str(blockN), dataindex =  -1, valueType = str), blockN])
            
            #create an array [start_block, end_block] for each block using the values we have just read -> 1 block ON + 1 bloc OFF + etc.
            timecount, blockON_start, blockON_end, blockOFF_start, blockOFF_end = 0, 0, 0, 0, 0
            blocks = [] #blocks that we are going to use in the data processing. 1 block ON + 1 bloc OFF + etc.
            rewardP_ON = [] #probability of getting the reward in each ON phase 
            rewardP_OFF = [] #same for OFF
            treadmillSpeed = [] #treadmill speed for each block (ON + OFF blocks not differenciated for now)
            rewardProbaBlock = []
            for block in blocklist:
                for repeat in range(0, block[2]): #in essence blocks = [a, b], [b, c], [c, d], ...
                    blockON_start = timecount
                    timecount += block[0]
                    blockON_end = timecount
                    blockOFF_start = timecount
                    timecount += block[1]
                    blockOFF_end = timecount
                    blocks.append([blockON_start, blockON_end])
                    if blockOFF_start - blockOFF_end != 0:
                        blocks.append([blockOFF_start, blockOFF_end])
                    rewardP_ON.append(block[3])
                    rewardP_OFF.append(block[4])
                    rewardProbaBlock.extend(block[3:5])
                    treadmillSpeed.append(block[5])
                    treadmillSpeed.append(block[5])
            params[animal, session]["blocks"], params[animal, session]["rewardP_ON"], params[animal, session]["rewardP_OFF"], params[animal, session]["treadmillSpeed"], params[animal, session]['rewardProbaBlock'] = blocks, rewardP_ON, rewardP_OFF, treadmillSpeed, rewardProbaBlock

            #Extract data for each .position file generated from LabView
            #Data loaded : time array, position of the animal X and Y axis, Licks to the left and to the right, and frame number
            extractTime[animal, session]      = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[3])#old format = 5
            extractPositionX[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[4])#old format = 6
            extractPositionY[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[5])
            extractLickLeft[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[6])
            extractLickRight[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[7])
            solenoid_ON_Left[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[8])
            solenoid_ON_Right[animal, session]= read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[9])
            framebuffer[animal, session]      = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[10])
            cameraEdit[animal, session]       = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[11])

            #Cut leftover data at the end of the session (e.g. session is 1800s long, data goes up to 1820s because session has not been stopped properly/stopped manually, so we remove the extra 20s)
            rawTime[animal, session]          = extractTime[animal, session][extractTime[animal, session]      <= params[animal, session]["sessionDuration"]]
            rawPositionX[animal, session]     = extractPositionX[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            rawPositionY[animal, session]     = extractPositionY[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            rawLickLeftX[animal, session]     = extractLickLeft[animal, session][extractTime[animal, session]  <= params[animal, session]["sessionDuration"]]
            rawLickLeftY[animal, session]     = extractLickLeft[animal, session][extractTime[animal, session]  <= params[animal, session]["sessionDuration"]]# not needed, check
            rawLickRightX[animal, session]    = extractLickRight[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            rawLickRightY[animal, session]    = extractLickRight[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]# not needed, check
            solenoid_ON_Left[animal, session] = solenoid_ON_Left[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            solenoid_ON_Right[animal, session]= solenoid_ON_Right[animal, session][extractTime[animal, session]<= params[animal, session]["sessionDuration"]]# not needed, check
            
            #convert data from px to cm
            rawPositionX[animal, session], rawPositionY[animal, session] = datapx2cm(rawPositionX[animal, session]), datapx2cm(rawPositionY[animal, session])
            rawSpeed[animal, session]  = compute_speed(rawPositionX[animal, session], rawTime[animal, session])
            smoothMask[animal,session] = np.array([True])

#######################################################################################
            #smoothing
            smoothPos, smoothSpeed = True, True
            sigmaPos, sigmaSpeed = 2, 2 #seems to work, less: not smoothed enough, more: too smoothed, not sure how to objectively compute an optimal value.
            if smoothPos == True:
                if smoothSpeed == True:
                    rawPositionX[animal, session]  = smooth(rawPositionX[animal, session], sigmaPos)
                    rawSpeed[animal, session]      = smooth(compute_speed(rawPositionX[animal, session], rawTime[animal, session]), sigmaSpeed)
                else: rawPositionX[animal, session]= smooth(rawPositionX[animal, session], sigmaPos)
######################################################################################

            # Load lick data -- Licks == measure of conductance at the reward port. Conductance is ____ and when lick, increase of conductance so ___|_|___, we define it as a lick if it is above a threshold. But baseline value can randomly increase like this ___----, so baseline can be above threshold, so false detections. -> compute moving median to get the moving baseline (median, this way we eliminate the peaks in the calculation of the baseline) and then compare with threshold. __|_|__---|---|----
            window = 200
            if params[animal, session]["lickthresholdLeft"] == None: params[animal, session]["lickthresholdLeft"] = 300
            if params[animal, session]["lickthresholdRight"]== None: params[animal, session]["lickthresholdRight"] = 300
            rawLickLeftX[animal, session]  = [k if i-j >= params[animal, session]["lickthresholdLeft"]  else 0 for i, j, k in zip(rawLickLeftX[animal, session],  movinmedian(rawLickLeftX[animal, session],  window), rawPositionX[animal, session])]
            rawLickRightX[animal, session] = [k if i-j >= params[animal, session]["lickthresholdRight"] else 0 for i, j, k in zip(rawLickRightX[animal, session], movinmedian(rawLickRightX[animal, session], window), rawPositionX[animal, session])]
            
            # Specify if a session has lick data problems, so we don't discard the whole session (keep the run behavior, remove lick data)     
            if   all(v == 0 for v in rawLickLeftX[animal, session]):  params[animal, session]["hasLick"] = False
            elif all(v == 0 for v in rawLickRightX[animal, session]): params[animal, session]["hasLick"] = False       
            elif animal + " " + session in lickBug: params[animal, session]["hasLick"] = False
            else: params[animal, session]["hasLick"] = True

            # Water data. Drop size and volume rewarded. Compute drop size for each reward port. Determine if drops are equal, or which one is bigger. Assign properties (e.g. line width for plots) accordingly.
            limitWater_diff = 5
            watL = round(params[animal, session]["waterLeft"], 1) #print(round(params[animal, session]["waterLeft"], 1), "µL/drop")
            watR = round(params[animal, session]["waterRight"], 1) #print(round(params[animal, session]["waterRight"], 1), "µL/drop")
            if watL-(watL*limitWater_diff/100) <= watR <= watL+(watL*limitWater_diff/100): water[animal, session] = ["Same Reward Size", "Same Reward Size", 2, 2] #print(session, "::", watL, watR, "     same L-R") #print(watL-(watL*limitWater_diff/100)) #print(watL+(watL*limitWater_diff/100))
            elif watL < watR: water[animal, session] = ["Small Reward", "Big Reward", 1, 5]#print(session, "::", watL, watR, "     bigR")
            elif watL > watR: water[animal, session] = ["Big Reward", "Small Reward", 5, 1] #print(session, "::", watL, watR, "     bigL")
            else: print(session, "error, bypass? Y/N")

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### DATA PROCESSING
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------      
    
            # Compute boundaries
            border = 5 #define arbitrary border
            leftBoundaryPeak[animal, session], rightBoundaryPeak[animal, session], kde[animal, session] = extract_boundaries(rawPositionX[animal, session], animal, session, params[animal, session]['treadmillDist'], height = 0.001)
            for s in boundariesBug: 
                if animal + " " + session == s[0]: 
                    params[animal, session]["boundaries"] = s[1]
                    break
                else: 
                    params[animal, session]["boundaries"] = [rightBoundaryPeak[animal, session] - border, leftBoundaryPeak[animal, session] + border]

            # Compute or pickle run/stay mask
            maskpicklePath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session+os.sep+"Analysis"+os.sep+"mask.p"
            if os.path.exists(maskpicklePath) and (not redoMask): 
                binMask[animal, session] = get_from_pickle(animal, session, name="mask.p")
            else: 
                if animal + " " + session in runstaysepbug: septhreshold = 0.0004
                else: septhreshold = 0.0002
                rawMask[animal,session]     = filterspeed(animal, session, rawPositionX[animal, session], rawSpeed[animal, session], rawTime[animal, session], septhreshold, params[animal, session]["treadmillDist"])#threshold 0.0004 seems to work ok for all TM distances. lower the thresh the bigger the wait blob zone taken, which caused problems in 60cm configuration.
                smoothMask[animal, session] = removeSplits_Mask(rawMask, rawPositionX, animal, session, params[animal, session]["treadmillDist"])
                binMask[animal,session]     = fixSplittedRunsMask(animal, session, bin_session(animal, session, smoothMask, rawTime, blocks), blocks)
            smoothMask[animal, session] = stitch([binMask[animal, session]])[0]
            goodPos[animal,session]     = [val[0] if val[1] == True  else None for val in [[i, j] for i, j in zip(rawPositionX[animal,session], smoothMask[animal,session])]]
            badPos[animal,session]      = [val[0] if val[1] == False else None for val in [[i, j] for i, j in zip(rawPositionX[animal,session], smoothMask[animal,session])]]
            goodSpeed[animal,session]   = [val[0] if val[1] == True  else None for val in [[i, j] for i, j in zip(rawSpeed[animal,session], smoothMask[animal,session])]]
            badSpeed[animal,session]    = [val[0] if val[1] == False else None for val in [[i, j] for i, j in zip(rawSpeed[animal,session], smoothMask[animal,session])]]
            binSpeed[animal, session]             = reCutBins(rawSpeed[animal, session], binMask[animal, session]) 
            binTime[animal, session]              = reCutBins(rawTime[animal, session], binMask[animal, session]) 
            binPositionX[animal, session]         = reCutBins(rawPositionX[animal, session], binMask[animal, session])
            binPositionY[animal, session]         = reCutBins(rawPositionY[animal, session], binMask[animal, session])
            binLickLeftX[animal, session]         = reCutBins(rawLickLeftX[animal, session], binMask[animal, session])
            binLickRightX[animal, session]        = reCutBins(rawLickRightX[animal, session], binMask[animal, session])
            binSolenoid_ON_Left[animal, session]  = reCutBins(solenoid_ON_Left[animal, session], binMask[animal, session])
            binSolenoid_ON_Right[animal, session] = reCutBins(solenoid_ON_Right[animal, session], binMask[animal, session]) 


            # Extract all variables.
            speedRunToRightBin[animal, session], speedRunToLeftBin[animal, session], XtrackRunToRightBin[animal, session], XtrackRunToLeftBin[animal, session], timeRunToRightBin[animal, session], timeRunToLeftBin[animal, session], timeStayInRightBin[animal, session], timeStayInLeftBin[animal, session], XtrackStayInRightBin[animal, session], XtrackStayInLeftBin[animal, session], TtrackStayInRightBin[animal, session], TtrackStayInLeftBin[animal, session], instantSpeedRightBin[animal, session], instantSpeedLeftBin[animal, session], maxSpeedRightBin[animal, session], maxSpeedLeftBin[animal, session], whenmaxSpeedRightBin[animal, session], whenmaxSpeedLeftBin[animal, session], wheremaxSpeedRightBin[animal, session], wheremaxSpeedLeftBin[animal, session], lick_arrivalRightBin[animal, session], lick_drinkingRightBin[animal, session], lick_waitRightBin[animal, session], lick_arrivalLeftBin[animal, session], lick_drinkingLeftBin[animal, session], lick_waitLeftBin[animal, session] = extract_runSpeedBin(binPositionX, binSpeed, binTime, binLickRightX, binLickLeftX, binMask, animal, session, params[animal, session]['blocks'], params[animal, session]["boundaries"],  params[animal, session]["treadmillSpeed"])
            speedRunToRight[animal, session],    speedRunToLeft[animal, session],    XtrackRunToRight[animal, session],    XtrackRunToLeft[animal, session],    timeRunToRight[animal, session],    timeRunToLeft[animal, session],    timeStayInRight[animal, session],    timeStayInLeft[animal, session],    XtrackStayInRight[animal, session],    XtrackStayInLeft[animal, session],    TtrackStayInRight[animal, session],    TtrackStayInLeft[animal, session],    instantSpeedRight[animal, session],    instantSpeedLeft[animal, session],    maxSpeedRight[animal, session],    maxSpeedLeft[animal, session],    whenmaxSpeedRight[animal, session],    whenmaxSpeedLeft[animal, session],    wheremaxSpeedRight[animal, session],    wheremaxSpeedLeft[animal, session],    lick_arrivalRight[animal, session],    lick_drinkingRight[animal, session],    lick_waitRight[animal, session],    lick_arrivalLeft[animal, session],    lick_drinkingLeft[animal, session],    lick_waitLeft[animal, session] = stitch([speedRunToRightBin[animal, session], speedRunToLeftBin[animal, session], XtrackRunToRightBin[animal, session], XtrackRunToLeftBin[animal, session], timeRunToRightBin[animal, session], timeRunToLeftBin[animal, session], timeStayInRightBin[animal, session], timeStayInLeftBin[animal, session], XtrackStayInRightBin[animal, session], XtrackStayInLeftBin[animal, session], TtrackStayInRightBin[animal, session], TtrackStayInLeftBin[animal, session], instantSpeedRightBin[animal, session], instantSpeedLeftBin[animal, session], maxSpeedRightBin[animal, session], maxSpeedLeftBin[animal, session], whenmaxSpeedRightBin[animal, session], whenmaxSpeedLeftBin[animal, session], wheremaxSpeedRightBin[animal, session], wheremaxSpeedLeftBin[animal, session], lick_arrivalRightBin[animal, session], lick_drinkingRightBin[animal, session],lick_waitRightBin[animal, session], lick_arrivalLeftBin[animal, session], lick_drinkingLeftBin[animal, session], lick_waitLeftBin[animal, session]])

            nb_runs_to_rightBin[animal, session], nb_runs_to_leftBin[animal, session], nb_runsBin[animal, session], total_trials[animal, session] = {}, {}, {}, 0
            for i in range (0, len(params[animal, session]['blocks'])):
                nb_runs_to_rightBin[animal, session][i] = len(speedRunToRightBin[animal, session][i])
                nb_runs_to_leftBin[animal, session][i] = len(speedRunToLeftBin[animal, session][i])
                nb_runsBin[animal, session][i] = len(speedRunToRightBin[animal, session][i]) + len(speedRunToLeftBin[animal, session][i])
                total_trials[animal, session] = total_trials[animal, session] + nb_runsBin[animal, session][i] 

            nb_rewardBlockLeft[animal, session], nb_rewardBlockRight[animal, session],nbWaterLeft[animal, session], nbWaterRight[animal, session] = {}, {}, 0, 0
            for i in range(0, len(params[animal, session]['blocks'])):
                nb_rewardBlockLeft[animal, session][i] = sum([1 if t[0] <= params[animal, session]['rewardProbaBlock'][i] else 0 for t in split_a_list_at_zeros(binSolenoid_ON_Left[animal, session][i])]) #split a list because in data file we have %open written along valve opening time duration (same value multiple time), so we only take the first one, verify >threshold, ...
                nb_rewardBlockRight[animal, session][i] = sum([1 if t[0] <= params[animal, session]['rewardProbaBlock'][i] else 0 for t in split_a_list_at_zeros(binSolenoid_ON_Right[animal, session][i])]) #print(i+1, nb_rewardBlockLeft[animal, session][i], nb_rewardBlockRight[animal, session][i])
            nbWaterLeft[animal, session] = sum(nb_rewardBlockLeft[animal, session].values())
            nbWaterRight[animal, session] = sum(nb_rewardBlockRight[animal, session].values())
            totalWater[animal, session] = round((nbWaterLeft[animal, session] * params[animal, session]["waterLeft"] + nbWaterRight[animal, session] * params[animal, session]["waterRight"])/1000, 2), 'mL'             #totalWater[animal, session] = nbWaterLeft[animal, session] * params[animal, session]["waterLeft"], "+", nbWaterRight[animal, session] * params[animal, session]["waterRight"]
            
            # compute total X distance moved during the session for each rat. maybe compute XY. 
            totalDistance[animal, session] = sum(abs(np.diff(rawPositionX[animal, session])))/100

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### FIGURES
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
        if os.path.exists(figPath) and (not redoFig):
            if printFigs == True: display(Image(filename=figPath))
        else:
            if redoCompute == False:
                print(session, " Error, you need to recompute everything to generate Fig.")
            else:
                # Plot figure
                fig = plt.figure(constrained_layout = False, figsize=(32, 42))
                fig.suptitle(session, y=0.9, fontsize = 24)
                gs = fig.add_gridspec(75, 75)
                ax00 = fig.add_subplot(gs[0:7, 0:4])
                ax00 = plot_peak(rawPositionX[animal, session], animal, session, leftBoundaryPeak[animal, session], rightBoundaryPeak[animal, session], kde[animal, session],[0.05, 0, 0],[0,120,0],  marker = [""], xyLabels=["Position (cm)", "%"])
                ax01 = fig.add_subplot(gs[0:7, 5:75])
                ax01 = plot_BASEtrajectoryV2(animal, session, rawTime[animal, session], goodPos[animal, session], badPos[animal, session], rawLickLeftX[animal, session], rawLickRightX[animal, session], params[animal, session]['rewardProbaBlock'], params[animal, session]['blocks'], barplotaxes = [0, params[animal, session]['sessionDuration'], 50, 90, 0, 22, 10],  xyLabels=["Time (min)","", "Position (cm)", "", "", "", 14, 12], title=[session, "", "", "", 16], linewidth = [1.5])
                plt.plot([0, params[animal, session]['sessionDuration']], [params[animal, session]["boundaries"][0], params[animal, session]["boundaries"][0]], ":", color='k', alpha = 0.5)
                plt.plot([0, params[animal, session]['sessionDuration']], [params[animal, session]["boundaries"][1], params[animal, session]["boundaries"][1]], ":", color='k', alpha = 0.5)

                gs00 = gs[8:13, 0:75].subgridspec(2,75)
                ax11 = fig.add_subplot(gs00[0, 5:75])
                ax12 = fig.add_subplot(gs00[1, 0:75])
                ax11.plot(rawTime[animal,session], goodSpeed[animal,session], color = 'dodgerblue')
                ax11.plot(rawTime[animal,session], badSpeed[animal, session], color = 'orange')
                ax11.set_xlabel('time (s)')
                ax11.set_ylabel('speed (cm/s)')
                ax11.set_xlim(0, 3600)
                ax11.set_ylim(-200,200)
                ax11.spines['top'].set_color("none")
                ax11.spines['right'].set_color("none")
                ax11.spines['left'].set_color("none")
                ax11.spines['bottom'].set_color("none")
                ax12.scatter(rawPositionX[animal,session], goodSpeed[animal,session], color = 'dodgerblue', s=0.5)
                ax12.scatter(rawPositionX[animal,session], badSpeed[animal,session], color = 'orange', s=0.5)
                ax12.set_xlabel('position (cm)')
                ax12.set_ylabel('speed (cm/s)')
                ax12.set_xlim(0,130)
                ax12.set_ylim(-150,150)
                ax12.spines['top'].set_color("none")
                ax12.spines['right'].set_color("none")
                ax12.spines['left'].set_color("none")
                ax12.spines['bottom'].set_color("none")
                yline = [0, 120]
                xline = [0,0]
                ax12.plot(yline, xline, ":", color='k')
                
                ax20 = fig.add_subplot(gs[17:22, 0:10])
                ax20 = plot_tracks(animal, session, XtrackRunToRight[animal, session], timeRunToRight[animal, session], params[animal, session]["boundaries"], xylim = [-0.1, 2, 0, 120], color=['paleturquoise', 'tomato'],  xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5])
                ax21 = fig.add_subplot(gs[17:22, 15:25])       
                ax21 = plot_tracks(animal, session, XtrackRunToLeft[animal, session], timeRunToLeft[animal, session], params[animal, session]["boundaries"], xylim = [-0.1, 2, 0, 120], color = ['darkcyan', 'darkred'], xyLabels=["Time (s)","", 14], title=["Tracking run to Left", 16], linewidth = [1.5])
                ax20 = fig.add_subplot(gs[17:22, 30:40])
                ax20 = cumul_plot(speedRunToRight[animal, session], speedRunToLeft[animal, session], barplotaxes = [0, 120, 0, 1], maxminstepbin = [0,120,1], scatterplotaxes = [0, 0, 0, 0], color = ['paleturquoise', 'darkcyan', 'tomato', 'darkred'], xyLabels=["Speed cm/s","Cumulative Frequency Run Speed", 14, 12], title=["Cumulative Plot Good Run Speed", 16], linewidth = [1.5], legend = ["To Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "To Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax21 = fig.add_subplot(gs[17:22, 45:55])       
                ax21 = distribution_plot(speedRunToRight[animal, session], speedRunToLeft[animal, session], barplotaxes = [0, 0, 0, 0], maxminstepbin = [0, 120, 1], scatterplotaxes = [0.5, 2.5, 0, 120], color = ['paleturquoise', 'darkcyan', 'tomato', 'darkred'], xyLabels=["Speed (cm/s)", "Direction of run", "To Right" + "\n" + water[animal, session][1], "To Left" + "\n" + water[animal, session][0], 14, 12], title=["Distribution of All Run Speed", 16], linewidth = [1.5], legend = ["To Right: Good Runs ", "To Left: Good Runs", "To Right: Bad Runs", "To Left: Bad Runs"])
                
                gs23 = gs[15:22, 60:75].subgridspec(5,2)
                ax231 = fig.add_subplot(gs23[0:2, 0:2])
                if len(framebuffer[animal, session]) != 0:
                    ax231.set_title("NbBug/TotFrames: %s/%s = %.2f" %(sum(np.diff(framebuffer[animal, session])-1), len(framebuffer[animal, session]), sum(np.diff(framebuffer[animal, session])-1)/len(framebuffer[animal, session])), fontsize = 16)
                ax231.scatter(list(range(1, len(framebuffer[animal, session]))), [ x-1 for x in np.diff(framebuffer[animal, session])], s=5)
                ax231.set_xlabel("frame index")
                ax231.set_ylabel("dFrame -1 (0 is ok)")
                ax232 = fig.add_subplot(gs23[3:5, 0:2])
                ax232.set_title(params[animal, session]["realSessionDuration"], fontsize = 16)
                ax232.plot(np.diff(rawTime[animal, session]), label = "data")
                ax232.plot(movinavg(np.diff(rawTime[animal, session]), 100), label="moving average")
                ax232.set_xlim(0, len(np.diff(rawTime[animal, session])))
                ax232.set_ylim(0, 0.1)
                ax232.set_xlabel("frame index")
                ax232.set_ylabel("time per frame (s)")
                
                ax30 = fig.add_subplot(gs[25:30, 0:10])
                ax30 = cumul_plot(maxSpeedRight[animal, session], maxSpeedLeft[animal, session], barplotaxes = [0, 200, 0, 1], maxminstepbin = [0,200,1], scatterplotaxes = [0.5, 2.5, 0, 100], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Speed cm/s","Cumulative Frequency MAX Run Speed", 14, 12], title=["Cumulative Plot MAX Run Speed", 16], linewidth = [1.5], legend = ["To Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "To Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax31 = fig.add_subplot(gs[25:30, 15:25])
                ax31 = distribution_plot(maxSpeedRight[animal, session], maxSpeedLeft[animal, session], barplotaxes = [0, 100, 0, 1], maxminstepbin = [0, 100, 1], scatterplotaxes = [0.5, 2.5, 0, 200], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Speed (cm/s)", "Direction of run", "To Right" + "\n" + water[animal, session][1], "To Left" + "\n" + water[animal, session][0], 14, 12], title = ["Distribution of MAX Run Speed", 16], linewidth = [1.5], legend = ["To Right: Good Runs", "To Left: Good Runs", "To Right: Bad Runs", "To Left: Bad Runs"])
                ax32 = fig.add_subplot(gs[25:30, 30:40])
                ax32 = plot_speed(animal, session, instantSpeedRight[animal, session], timeRunToRight[animal, session], [0,0], xylim = [-0.1, 4, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["To Right" + "\n" + "To " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", 12], linewidth = [1.5])
                ax33 = fig.add_subplot(gs[25:30, 45:55])
                ax33 = plot_speed(animal, session, instantSpeedLeft[animal, session], timeRunToLeft[animal, session], [0,0], xylim = [-0.1, 4, 0, 200], xyLabels=["Time (s)","", 14], title=["To Left" + "\n" + "To " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", 12], linewidth = [1.5])
                ax34 = fig.add_subplot(gs[25:30, 60:70])
                ax34 = plot_speed(animal, session, instantSpeedRight[animal, session] + instantSpeedLeft[animal, session], timeRunToRight[animal, session] + timeRunToLeft[animal, session], [0,0], xylim = [-0.1, 4, 0, 200], xyLabels=["Time (s)","", 14], title=["Speed" + "\n" + " To left and to right", 12], linewidth = [0.5])
                
                ax40 = fig.add_subplot(gs[35:40, 0:8])
                ax40 = cumul_plot(maxSpeedRight[animal, session], maxSpeedLeft[animal, session], barplotaxes = [0, 250, 0, 1], maxminstepbin = [0, 250, 1], scatterplotaxes = [0, 0, 0, 0], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Speed cm/s","Cumulative Frequency MAX Run Speed", 14, 12], title=["CumulPlt MAXrunSpeed <TreadmillCorrected>", 16], linewidth = [1.5], legend = ["To Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "To Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax41 = fig.add_subplot(gs[35:40, 12:23])
                ax41 = distribution_plot(maxSpeedRight[animal, session], maxSpeedLeft[animal, session], barplotaxes = [0, 0, 0, 0], maxminstepbin = [0, 0, 0], scatterplotaxes = [0.5, 2.5, 0, 250], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Speed (cm/s)", "Direction of run", "To Right" + "\n" + water[animal, session][1], "To Left" + "\n" + water[animal, session][0], 14, 12], title = ["Distr. of MAXrunSpeed <TreadmillCorrected>", 16], linewidth = [1.5], legend = ["To Right: Good Runs", "To Left: Good Runs", "To Right: Bad Runs", "To Left: Bad Runs"])
                ax42 = fig.add_subplot(gs[35:40, 26:34]) #where maxspeed
                ax42 = cumul_plot(wheremaxSpeedRight[animal, session], wheremaxSpeedLeft[animal, session], barplotaxes = [0, 120, 0, 1], maxminstepbin = [0, 120, 1], scatterplotaxes = [0, 0, 0, 0], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Position maxSpeed reached (cm)", "Cumulative Frequency MAX runSpeed Position", 14, 12], title=["CumulPlt MAXrunSpeed \nPosition from start of run", 16], linewidth = [1.5], legend = ["To Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "To Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax43 = fig.add_subplot(gs[35:40, 38:49])
                ax43 = distribution_plot(wheremaxSpeedRight[animal, session], wheremaxSpeedLeft[animal, session], barplotaxes = [0, 0, 0, 0], maxminstepbin = [0, 0, 0], scatterplotaxes = [0.5, 2.5, 0, 120], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["X Position (cm)", "Direction of run", "To Right" + "\n" + water[animal, session][1], "To Left" + "\n" + water[animal, session][0], 14, 12], title = ["Distr. MAXrunSpeed \nPosition from start of run", 16], linewidth = [1.5], legend = ["To Right: Good Runs", "To Left: Good Runs", "To Right: Bad Runs", "To Left: Bad Runs"])
                ax44 = fig.add_subplot(gs[35:40, 52:60]) #when maxspeed
                ax44 = cumul_plot(whenmaxSpeedRight[animal, session], whenmaxSpeedLeft[animal, session], barplotaxes = [0, 2.5, 0, 1], maxminstepbin = [0, 2.5, 0.04], scatterplotaxes = [0, 0, 0, 0], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Time MAX runSpeed reached (s)","Cumulative Frequency", 14, 12], title=["CumulPlt Time of \nMAXrunSpeed from start of run", 16], linewidth = [1.5], legend = ["To Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "To Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax45 = fig.add_subplot(gs[35:40, 64:75])
                ax45 = distribution_plot(whenmaxSpeedRight[animal, session], whenmaxSpeedLeft[animal, session], barplotaxes = [0, 0, 0, 0], maxminstepbin = [0, 0, 0], scatterplotaxes = [0.5, 2.5, 0, 2.5], color=['lightgreen', 'darkgreen', 'tomato', 'darkred'], xyLabels=["Time MAX runSpeed reached (s)", "Direction of run", "To Right" + "\n" + water[animal, session][1], "To Left" + "\n" + water[animal, session][0], 14, 12], title = ["Distr. Time of MAXrunSpeed \nfrom start of run", 16], linewidth = [1.5], legend = ["To Right: Good Runs", "To Left: Good Runs", "To Right: Bad Runs", "To Left: Bad Runs"])

                ax50 = fig.add_subplot(gs[45:50, 0:10])  
                ax50 = plot_tracks(animal, session, XtrackStayInRight[animal, session], TtrackStayInRight[animal, session], params[animal, session]["boundaries"], xylim = [-1, 10, params[animal, session]['treadmillDist']-40, params[animal, session]['treadmillDist']], color=['moccasin', 'tomato'], xyLabels=["Time (s)","X Position (cm)", 14, 12], title=["Tracking in Right", 16], linewidth = [1.5])
                ax51 = fig.add_subplot(gs[45:50, 15:25])
                ax51 = plot_tracks(animal, session, XtrackStayInLeft[animal, session], TtrackStayInLeft[animal, session], params[animal, session]["boundaries"], xylim = [-1, 10, 0, 40], color = ['darkorange', 'darkred'], xyLabels=["Time (s)","", 14, 12], title=["Tracking in Left", 16], linewidth = [1.5])
                ax52 = fig.add_subplot(gs[45:50, 30:40])  
                ax52 = cumul_plot(timeStayInRight[animal, session], timeStayInLeft[animal, session], barplotaxes = [0, 15, 0, 1], maxminstepbin = [0, 15, 0.1], scatterplotaxes = [0, 0, 0, 0], color = ['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time in zone (s)","Cumulative Frequency Time In Zone", 14, 12], title=["Cumulative Plot Good Time In Zone", 16], linewidth = [1.5], legend = ["In Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "In Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax53 = fig.add_subplot(gs[45:50, 45:60])
                ax53 = distribution_plot(timeStayInRight[animal, session], timeStayInLeft[animal, session], barplotaxes = [0, 0, 0, 0], maxminstepbin = [0,30,1], scatterplotaxes = [0.5, 2.5, 0, 30], color = ['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time in zone (s)", "Zone", "In Right" + "\n" + water[animal, session][1], "In Left" + "\n" + water[animal, session][0], 14, 12], title=["Distribution of All Time In Zone", 16], linewidth = [1.5], legend = ["To Right: Good Runs", "To Left: Good Runs", "To Right: Bad Runs", "To Left: Bad Runs"]) 
                
                ax60 = fig.add_subplot(gs[55:60, 0:8])
                ax60 = cumul_plot(lick_arrivalRight[animal, session], lick_arrivalLeft[animal, session], barplotaxes = [0, 2, 0, 1], maxminstepbin = [0,2,0.1], scatterplotaxes = [0.5, 2.5, 0, 100], color=['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time (s)","Cumulative Frequency", 14, 12], title=["Cumulative Plot preDrink Time", 16], linewidth = [1.5], legend = ["In Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "In Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax61 = fig.add_subplot(gs[55:60, 12:23])
                ax61 = distribution_plot(lick_arrivalRight[animal, session], lick_arrivalLeft[animal, session], barplotaxes = [0, 100, 0, 1], maxminstepbin = [0, 100, 1], scatterplotaxes = [0.5, 2.5, 0, 2], color=['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time (s)", "Zone", "In Right" + "\n" + water[animal, session][1], "In Left" + "\n" + water[animal, session][0], 14, 12], title=["Distribution preDrink Time", 16], linewidth = [1.5], legend = ["In Right", "In Left", " ", " "])
                ax62 = fig.add_subplot(gs[55:60, 26:34])
                ax62 = cumul_plot(lick_drinkingRight[animal, session], lick_drinkingLeft[animal, session], barplotaxes = [0, 4, 0, 1], maxminstepbin = [0,4,0.1], scatterplotaxes = [0.5, 2.5, 0, 100], color=['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time (s)","Cumulative Frequency", 14, 12], title=["Cumulative Plot Drink Time", 16], linewidth = [1.5], legend = ["In Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "In Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax63 = fig.add_subplot(gs[55:60, 38:49])
                ax63 = distribution_plot(lick_drinkingRight[animal, session], lick_drinkingLeft[animal, session], barplotaxes = [0, 100, 0, 1], maxminstepbin = [0, 100, 1], scatterplotaxes = [0.5, 2.5, 0, 4], color=['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time (s)", "Zone", "In Right" + "\n" + water[animal, session][1], "In Left" + "\n" + water[animal, session][0], 14, 12], title = ["Distribution of Drink Time", 16], linewidth = [1.5], legend = ["In Right", "In Left", " ", " "])
                ax64 = fig.add_subplot(gs[55:60, 52:60])
                ax64 = cumul_plot(lick_waitRight[animal, session], lick_waitLeft[animal, session], barplotaxes = [0, 10, 0, 1], maxminstepbin = [0,10,0.1], scatterplotaxes = [0.5, 2.5, 0, 100], color=['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time (s)","Cumulative Frequency", 14, 12], title=["Cumulative Plot postDrink Time", 16], linewidth = [1.5], legend = ["In Right: " + water[animal, session][1] + " " + str(params[animal, session]["waterRight"]) + "µL/drop", "In Left:  " + water[animal, session][0] + " " + str(params[animal, session]["waterLeft"]) + "µL/drop", water[animal, session][2], water[animal, session][3]])
                ax65 = fig.add_subplot(gs[55:60, 64:75])
                ax65 = distribution_plot(lick_waitRight[animal, session], lick_waitLeft[animal, session], barplotaxes = [0, 100, 0, 1], maxminstepbin = [0, 100, 1], scatterplotaxes = [0.5, 2.5, 0, 10], color=['moccasin', 'darkorange', 'tomato', 'darkred'], xyLabels=["Time (s)", "Zone", "In Right" + "\n" + water[animal, session][1], "In Left" + "\n" + water[animal, session][0], 14, 12], title = ["Distribution of postDrink Time", 16], linewidth = [1.5], legend = ["In Right", "In Left", " ", " "])
                
                if len(params[animal, session]['blocks']) > 1:
                    ax70 = fig.add_subplot(gs[63:70, 0:9])
                    ax70 = plot_figBin([nb_runsBin[animal, session][i]/(int((blocks[i][1]-blocks[i][0])/60)) for i in range(0, len(blocks))], params[animal, session]['rewardProbaBlock'], params[animal, session]['blocks'], barplotaxes = [0, params[animal, session]['sessionDuration']/60, 0, 25], color = ['k'], xyLabels=["Time (min)", "\u0023 runs / min", 14, 12], title=["", 16], linewidth = [1.5]);
                    ax72 = fig.add_subplot(gs[63:70, 20:29])
                    ax72 = plot_figBin([np.mean(speedRunToLeftBin[animal, session][i] + speedRunToRightBin[animal, session][i]) for i in range(0, len(blocks))], params[animal, session]['rewardProbaBlock'], params[animal, session]['blocks'], barplotaxes = [0, params[animal, session]['sessionDuration']/60, 0, 100], color = ['dodgerblue'], xyLabels=["Time (min)","Avg. run speed (cm/s)", 14, 12], title=["", 16], linewidth = [1.5]);
                    ax74 = fig.add_subplot(gs[63:70, 40:49])
                    ax74 = plot_figBin([np.mean(maxSpeedRightBin[animal, session][i] + maxSpeedLeftBin[animal, session][i]) for i in range(0, len(blocks))], params[animal, session]['rewardProbaBlock'], params[animal, session]['blocks'], barplotaxes = [0, params[animal, session]['sessionDuration']/60, 0, 150], color = ['red'], xyLabels=["Time (min)", "Average max speed (cm/s)", 14, 12], title=["", 16], linewidth = [1.5]);
                    ax76 = fig.add_subplot(gs[63:70, 60:69])
                    ax76 = plot_figBin([np.mean(timeStayInLeftBin[animal, session][i] + timeStayInRightBin[animal, session][i]) for i in range(0, len(blocks))], params[animal, session]['rewardProbaBlock'], params[animal, session]['blocks'], barplotaxes = [0, params[animal, session]['sessionDuration']/60, 0, 25], color = ['orange'], xyLabels=["Time (min)","Avg. time in sides (s)", 14, 12], title=["", 16], linewidth = [1.5]);

                    ax71 = fig.add_subplot(gs[63:70, 10:15])
                    ax71 = plot_figBinMean(ax71, [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([nb_runsBin[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([nb_runsBin[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))], color = ['k'], ylim = (0, 25))
                    ax73 = fig.add_subplot(gs[63:70, 30:35])
                    ax73 = plot_figBinMean(ax73, [np.mean(i) for i in poolByReward([speedRunToRightBin[animal, session], speedRunToLeftBin[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])], [np.mean(i) for i in poolByReward([speedRunToRightBin[animal, session], speedRunToLeftBin[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])], color = ['dodgerblue'], ylim = (0, 100))
                    ax75 = fig.add_subplot(gs[63:70, 50:55])
                    ax75 = plot_figBinMean(ax75, [np.mean(i) for i in poolByReward([maxSpeedRightBin[animal, session], maxSpeedLeftBin[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])], [np.mean(i) for i in poolByReward([maxSpeedRightBin[animal, session], maxSpeedLeftBin[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])], color = ['red'], ylim = (0, 150))
                    ax77 = fig.add_subplot(gs[63:70, 70:75])
                    ax77 = plot_figBinMean(ax77, [np.mean(i) for i in poolByReward([timeStayInRightBin[animal, session], timeStayInLeftBin[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])], [np.mean(i) for i in poolByReward([timeStayInRightBin[animal, session], timeStayInLeftBin[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])], color = ['orange'], ylim = (0, 25))
                    
                %config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
                ax80 = fig.add_subplot(gs[73:74, 0:60])
                ax80.spines['top'].set_color("none")
                ax80.spines['right'].set_color("none")
                ax80.spines['left'].set_color("none")
                ax80.spines['bottom'].set_color("none")
                ax80.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) 
                ax80.tick_params(axis='y', which='both', left=False, right=False, labelleft=False) 
                text = ("sessionDuration: {0} | acqPer: {1} | waterLeft: {2} | waterRight: {3} | treadmillDist: {4} | weight: {5} | lastWeightadlib: {6} | lastDayadlib: {7} | lickthresholdLeft: {8} | lickthresholdRight: {9} | realEnd: {10} | boundaries: {11} | daysSinceadLib: {12} \n realSessionDuration: {13} | blocks: {14} | \n rewardP_ON: {15} | rewardP_OFF: {16} | treadmillSpeed: {17} | rewardProbaBlock: {18} | hasLick: {19}").format(params[animal,session]['sessionDuration'], params[animal,session]['acqPer'], params[animal,session]['waterLeft'], params[animal,session]['waterRight'], params[animal,session]['treadmillDist'], params[animal,session]['weight'], params[animal,session]['lastWeightadlib'], params[animal,session]['lastDayadlib'], params[animal,session]['lickthresholdLeft'], params[animal,session]['lickthresholdRight'], params[animal,session]['realEnd'], params[animal,session]['boundaries'], params[animal,session]['daysSinceadLib'],params[animal,session]['realSessionDuration'], params[animal,session]['blocks'], params[animal,session]['rewardP_ON'], params[animal,session]['rewardP_OFF'], params[animal,session]['treadmillSpeed'], params[animal,session]['rewardProbaBlock'], params[animal,session]['hasLick'])
                ax80 = plt.text(0,0, str(text), wrap = True)

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### SAVE + PICKLE
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        if redoCompute == True:
            save_sessionplot_as_png(animal, session, 'recapFIG%s.png'%session, dpi='figure', transparent = False, background = 'w')
            save_as_pickle(params[animal, session],     animal, session, "params.p")
            save_as_pickle(binMask[animal, session], animal, session, "mask.p")
            save_as_pickle(nb_runsBin[animal, session], animal, session, "nbRuns.p")
            save_as_pickle([totalDistance[animal, session], totalWater[animal, session], total_trials[animal, session]], animal, session, "misc.p")

            save_as_pickle([speedRunToLeftBin[animal, session], speedRunToRightBin[animal, session]], animal, session, "avgSpeed.p")
            save_as_pickle([[[np.sum(np.diff(j)) for j in timeRunToLeftBin[animal, session][i] ]for i in range(0, len(params[animal, session]['blocks']))],
            [[np.sum(np.diff(j)) for j in timeRunToRightBin[animal, session][i] ]for i in range(0, len(params[animal, session]['blocks']))]] , animal, session, "timeRun.p")
            save_as_pickle([maxSpeedLeftBin[animal, session], maxSpeedRightBin[animal, session]], animal, session, "maxSpeed.p")
            save_as_pickle([timeStayInLeftBin[animal, session], timeStayInRightBin[animal, session]], animal, session, "timeinZone.p")
            save_as_pickle([XtrackRunToLeftBin[animal, session], XtrackRunToRightBin[animal, session]], animal, session, "trackPos.p")
            save_as_pickle([instantSpeedLeftBin[animal, session], instantSpeedRightBin[animal, session]], animal, session, "trackSpeed.p")
            save_as_pickle([timeRunToLeftBin[animal, session], timeRunToRightBin[animal, session]], animal, session, "trackTime.p")

            #lick_arrivalRightBin[animal, session], lick_drinkingRightBin[animal, session], lick_waitRightBin[animal, session], lick_arrivalLeftBin[animal, session], lick_drinkingLeftBin[animal, session], lick_waitLeftBin[animal, session]
            if printFigs == False:
                plt.close('all')

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### FLUSH
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

            # Delete all data for this session
            params, rat_markers, water = {}, {}, {}
            extractTime, extractPositionX, extractPositionY, extractLickLeft, extractLickRight, framebuffer, solenoid_ON_Left, solenoid_ON_Right, cameraEdit = ({} for i in range(9)) 
            rawTime, rawPositionX, rawPositionY, rawLickLeftX, rawLickRightX, rawLickLeftY, rawLickRightY, smoothMask, rawSpeed = ({} for i in range(9)) 
            binPositionX, binPositionY, binTime, binLickLeftX, binLickRightX, binSolenoid_ON_Left, binSolenoid_ON_Right = ({} for i in range(7))
            leftBoundaryPeak, rightBoundaryPeak, kde = {}, {}, {}
            smoothMask, rawMask, binSpeed, binMask = {}, {}, {}, {}
            goodPos, badPos, goodSpeed, badSpeed = {}, {}, {}, {}
            speedRunToRight,    speedRunToLeft,    XtrackRunToRight,    XtrackRunToLeft,    timeRunToRight,    timeRunToLeft,    timeStayInRight,    timeStayInLeft,    XtrackStayInRight,    XtrackStayInLeft,    TtrackStayInRight,    TtrackStayInLeft,    instantSpeedRight,    instantSpeedLeft,    maxSpeedRight,    maxSpeedLeft,    whenmaxSpeedRight,    whenmaxSpeedLeft,    wheremaxSpeedRight,    wheremaxSpeedLeft,    lick_arrivalRight,    lick_drinkingRight,    lick_waitRight,    lick_arrivalLeft,    lick_drinkingLeft,    lick_waitLeft    = ({} for i in range(26))
            speedRunToRightBin, speedRunToLeftBin, XtrackRunToRightBin, XtrackRunToLeftBin, timeRunToRightBin, timeRunToLeftBin, timeStayInRightBin, timeStayInLeftBin, XtrackStayInRightBin, XtrackStayInLeftBin, TtrackStayInRightBin, TtrackStayInLeftBin, instantSpeedRightBin, instantSpeedLeftBin, maxSpeedRightBin, maxSpeedLeftBin, whenmaxSpeedRightBin, whenmaxSpeedLeftBin, wheremaxSpeedRightBin, wheremaxSpeedLeftBin, lick_arrivalRightBin, lick_drinkingRightBin, lick_waitRightBin, lick_arrivalLeftBin, lick_drinkingLeftBin, lick_waitLeftBin = ({} for i in range(26))
            nb_runs_to_rightBin, nb_runs_to_leftBin, nb_runsBin, total_trials = {}, {}, {}, {}
        
        arr[index] += (1/len(sessionList))
        clear_output(wait=True)
        update_progress(arr[:])


# COMPUTE

In [ ]:
p = {}
from multiprocessing import Array
animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
#for animal in animalList:# animal, index, redoFig, printFigs, redoMask
#    processData(           animal, index, True,    False,     False,  )
progress = Array('d', [0, 0, 0, 0, 0, 0])
for index, animal in enumerate(animalList):                       # progress, animal, sessions loop through /or[] ,                    index, redo ,redoFigs, printFigs, redoMask
    p[animal] = multiprocessing.Process(target = processData, args=(progress, animal, matchsession(animal, postVacsTrain), index, False, False,    False,     False,  )) #+postVacsTrain + postVacs60 + postVacs60BIS + postVacs90_1 + postVacs90_2 + postVacs120
for index, animal in enumerate(animalList):
    p[animal].start()
for index, animal in enumerate(animalList):
    p[animal].join()
#trainDist + TMtrain + postVacsTrain lastpass 20211015
#dist60 + dist90 + dist120 lastpass 20211117 #removed all/good/bad > everything is in the same variable.
#TMrev20 + TMrev10 + TMrev2 + TM2 + TM10 + TM20 lastpass 20211117 #same
#postVacs60+postVacs60BIS+postVacs90_1+postVacs90_2+postVacs120 lastpass 20211117 #same

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

In [ ]:
endTimeNotebook = datetime.datetime.now()
timeExecutionNotebook = endTimeNotebook-startTimeNotebook
print(timeExecutionNotebook)

# Accross session plots 

In [ ]:
params = {}
smoothMask = {}
nb_runsBin = {}
rat_markers = {}
speedRunToLeftBin, speedRunToRightBin = {}, {}
timeRunToLeftBin, timeRunToRightBin = {}, {}
maxSpeedLeftBin, maxSpeedRightBin = {}, {}
timeStayInLeftBin, timeStayInRightBin = {}, {}
totalDistance, totalWater, total_trials = {}, {}, {}
XtrackRunToLeftBin, XtrackRunToRightBin = {}, {}
instantSpeedLeftBin, instantSpeedRightBin = {}, {}
trackTimeToLeftBin, trackTimeToRightBin = {}, {}
palette = [(0.55, 0.0, 0.0),  (0.8, 0.36, 0.36),   (1.0, 0.27, 0.0),   (0.0, 0.39, 0.0),    (0.13, 0.55, 0.13),   (0.2, 0.8, 0.2)]### we use RGB [0-1] not [0-255]. See www.colorhexa.com for conversion #old#palette = ['darkred', 'indianred', 'orangered', 'darkgreen', 'forestgreen', 'limegreen']

animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
for index, animal in enumerate(animalList):
    if fnmatch.fnmatch(animal, 'RatF*'): rat_markers[animal]=[palette[index], "$\u2640$"]
    elif fnmatch.fnmatch(animal, 'RatM*'): rat_markers[animal]=[palette[index], "$\u2642$"]
    elif fnmatch.fnmatch(animal, 'Rat00*'): rat_markers[animal]=[palette[index], "$\u2426$"]
    else: print("error, this is not a rat you got here")
    
    for session in sorted(matchsession(animal, dist60+dist90+dist120 + TM20+TM10+TM2+TMrev2+TMrev10+TMrev20 +postVacs60+postVacs60BIS+postVacs90_1+postVacs90_2+postVacs120)):
        params[animal, session] = get_from_pickle(animal, session, name="params.p")
        smoothMask[animal, session] = get_from_pickle(animal, session, name="mask.p")
        nb_runsBin[animal, session] = get_from_pickle(animal, session, name="nbRuns.p")
        speedRunToLeftBin[animal, session], speedRunToRightBin[animal, session] = get_from_pickle(animal, session, name="avgSpeed.p")
        maxSpeedLeftBin[animal, session], maxSpeedRightBin[animal, session] = get_from_pickle(animal, session, name="maxSpeed.p")
        timeStayInLeftBin[animal, session], timeStayInRightBin[animal, session] = get_from_pickle(animal, session, name="timeinZone.p")
        totalDistance[animal, session], totalWater[animal, session], total_trials[animal, session] = get_from_pickle(animal, session, name="misc.p")
        timeRunToLeftBin[animal, session], timeRunToRightBin[animal, session] = get_from_pickle(animal, session, name="timeRun.p")
        XtrackRunToLeftBin[animal, session], fig = plt.figure(constrained_layout = False, figsize=(23, 5))
gs = fig.add_gridspec(5, 23)[animal, session] = get_from_pickle(animal, session, "trackPos.p")
        instantSpeedLeftBin[animal, session], instantSpeedRightBin[animal, session] = get_from_pickle(animal, session, "trackSpeed.p")
        trackTimeToLeftBin[animal, session], trackTimeToRightBin[animal, session] = get_from_pickle(animal, session, "trackTime.p")

    for session in sorted(matchsession(animal, trainDist+TMtrain+postVacsTrain)):
        params[animal, session] = get_from_pickle(animal, session, name="params.p")
        smoothMask[animal, session] = get_from_pickle(animal, session, name="mask.p")
        totalDistance[animal, session], totalWater[animal, session], total_trials[animal, session] = get_from_pickle(animal, session, name="misc.p")

#plot rat speed along run
def plot_speed(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    for i, j in zip(posdataRight, timedataRight):
        #print(i, j)
        time = np.subtract(j, j[0])
        iabs =  [abs(ele) for ele in i]
        plt.plot(np.subtract(j, j[0]), iabs, color='g' , linewidth=0.3)
        if len(np.where(i == max(i))[0]) == 1:
            maxspeed = max(iabs)
            maxspeedtime = np.where(iabs == maxspeed)[0]
            plt.scatter(time[maxspeedtime], maxspeed, color ='darkgreen', s=20)
        else: print(max(i))
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    xline1 = [bounds[0], bounds[0]]
    xline2 = [bounds[1], bounds[1]]
    yline = [0,20]
    plt.plot(yline, xline1, ":", color='k')
    plt.plot(yline, xline2, ":", color='k')
    return ax

def plot_trackstest(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    n = len(posdataRight)
    colors = plt.cm.RdYlGn_r(np.linspace(0,1,n)) #green early, red late
    for index, (i, j) in enumerate(zip(posdataRight, timedataRight)):
        tracks = ax.plot(np.subtract(j, j[0]), i, color='g' , linewidth=0.3, label = index if index == 0 or index == n-1 else "")#, label="Good Item" if i == posdataRight[0] else "")
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    xline1 = [bounds[0], bounds[0]]
    xline2 = [bounds[1], bounds[1]]
    yline = [0,20]
    plt.plot(yline, xline1, ":", color='k')
    plt.plot(yline, xline2, ":", color='k')
    return ax    

def plot_median_trajectory2(posdataRight, timedataRight, animal, alpha, color):
    median_plot = plt.gca()
    try : maxL = np.nanmax([len(j) for j in timedataRight])
    except TypeError: maxL = 1
    except ValueError: maxL = 1
    avgPos = np.ones((len(timedataRight), maxL))*np.nan
    for i, p in enumerate(posdataRight): avgPos[i, 0:len(p)] = p
    nanSum = np.sum(np.isnan(avgPos), axis = 0)
    try: end_medTraj = np.where(nanSum > .3 * avgPos.shape[0])[0][0]
    except IndexError: end_medTraj = avgPos.shape[1]
    ci = (np.nanstd(avgPos, axis = 0)[:end_medTraj])/(np.sqrt(np.count_nonzero(~np.isnan(avgPos), axis = 0)[:end_medTraj]))
    plt.plot(np.linspace(0, end_medTraj*0.04, end_medTraj), np.nanmedian(avgPos, axis = 0)[:end_medTraj], color =color, alpha = alpha)#color = 'g' if block == 90 else 'r')
    plt.fill_between(np.linspace(0, end_medTraj*0.04, end_medTraj), np.nanmedian(avgPos, axis = 0)[:end_medTraj]-ci, np.nanmedian(avgPos, axis = 0)[:end_medTraj]+ci, color =color, zorder = 3, alpha = alpha/4) #,'g' if block == 90 else 'r'
    return median_plot




fig = plt.figure(constrained_layout = False, figsize=(5, 15))
ax1 = plt.subplot(311)
ax1 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0, 0],xylim = [-0.1, 2, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
ax1 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], animal, 1, 'k')
aaa = compute_median_trajectory(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])

ax2 = plt.subplot(312)
ax2 = plot_speed(animal, session, instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0,0], xylim = [-0.1, 2, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["To Right" + "µL/drop", 12], linewidth = [1.5])
ax2 = plot_median_trajectory2(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], animal, 1, 'k')
bbb = compute_median_trajectory(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])

#bbb = compute_median_trajectory(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])

In [ ]:
# Median computation
# Modified from: Averaging GPS segments competition 2019. https://doi.org/10.1016/j.patcog.2020.107730
#                T. Karasek, "SEGPUB.IPYNB", Github, 2019. https://gist.github.com/t0mk/eb640963d7d64e14d69016e5a3e93fd6

# # # should be able to squeeze SEM in SampleSet

def median(lst): 
    sortedLst = sorted(lst)
    lstLen = len(lst)
    index = (lstLen - 1) // 2    
    return sortedLst[index] 

def zscore(l):
    if len(np.unique(l)) == 1:
        return np.full(len(l),0.)
    return (np.array(l)  - np.mean(l)) / np.std(l)
    
def disterr(x1,y1, x2, y2):        
    sd = np.array([x1[0]-x2[0],y1[0]-y2[0]])
    ed = np.array([x1[0]-x2[-1],y1[0]-y2[-1]])
    if np.linalg.norm(sd) > np.linalg.norm(ed):
        x2 = np.flip(x2, axis=0)
        y2 = np.flip(y2, axis=0)
        
    offs = np.linspace(0,1,10)
    xrs1, yrs1 = Traj((x1,y1)).getPoints(offs)
    xrs2, yrs2 = Traj((x2,y2)).getPoints(offs)
    return np.sum(np.linalg.norm([xrs1-xrs2, yrs1-yrs2],axis=0))

def rdp(points, epsilon):
    dmax = 0.0
    index = 0
    for i in range(1, len(points) - 1):
        d = point_line_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d
    if dmax >= epsilon:
        results = rdp(points[:index+1], epsilon)[:-1] + rdp(points[index:], epsilon)
    else:
        results = [points[0], points[-1]]
    return results
    
def distance(a, b): 
    return  np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def point_line_distance(point, start, end):
    if (start == end):
        return distance(point, start)
    else:
        n = abs((end[0] - start[0]) * (start[1] - point[1]) - (start[0] - point[0]) * (end[1] - start[1]))
        d = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
        return n / d

class OnlyOnePointError(Exception):
    pass

class SampleSet:
    def __init__(self, ll):
        # ll is list of tuples [x_array,y_array] for every trajectory in sample
        self.trajs = [Traj(l) for l in ll]
        self.xp = None
        self.yp = None
        self.d = None
        self.filtix = None
        self.lenoutix = None
        self.disoutix = None
        self.eps = None

    def getRawAvg(self):
        trajLen = median([len(t.xs) for t in self.trajs])
        offs = np.linspace(0,1,trajLen)
        xm = []
        ym = []
        for t in self.trajs:
            xs, ys = t.getPoints(offs)
            xm.append(xs)
            ym.append(ys)        
        xp, yp = np.median(xm, axis=0), np.median(ym, axis=0)
        #xp, yp = np.mean(xm, axis=0), np.mean(ym, axis=0)
        return xp, yp

    def endpoints(self):
        cs = np.array([[self.trajs[0].xs[0],self.trajs[0].xs[-1]], [self.trajs[0].ys[0],self.trajs[0].ys[-1]]])
        xs = np.hstack([t.xs[0] for t in self.trajs] + [t.xs[-1] for t in self.trajs])
        ys = np.hstack([t.ys[0] for t in self.trajs] + [t.ys[-1] for t in self.trajs])       
        clabs = []
        oldclabs = []
        for j in range(10):
            for i in range(len(xs)):
                ap = np.array([[xs[i]],[ys[i]]])
                dists = np.linalg.norm(ap - cs, axis=0)
                clabs.append(np.argmin(dists))
            #cx = np.array([np.mean(xs[np.where(np.array(clabs)==0)]), np.mean(xs[np.where(np.array(clabs)==1)])])
            #cy = np.array([np.mean(ys[np.where(np.array(clabs)==0)]), np.mean(ys[np.where(np.array(clabs)==1)])])
            if oldclabs == clabs: 
                break
            oldclabs = clabs
            clabs = []
        for i,l in enumerate(clabs[:len(clabs)//2]):
            if l == 1:
                oldT = self.trajs[i]                
                reversedTraj = (np.flip(oldT.xs, axis=0), np.flip(oldT.ys, axis=0))
                self.trajs[i] = Traj(reversedTraj)   

    def zlen(self):
        ls = np.array([t.cuts[-1] for t in self.trajs])
        return zscore(ls)
        
    def getFiltered(self, dismax, lenlim):
        xa, ya = self.getRawAvg()
        d = zscore(np.array([disterr(t.xs, t.ys, xa, ya) for t in self.trajs]))
        l = self.zlen()
        self.lenoutix = np.where((l<lenlim[0])|(l>lenlim[1]))[0]
        lenix = np.where((l>lenlim[0])&(l<lenlim[1]))[0]
        self.disoutix = np.where(d>dismax)[0]
        disix = np.where(d<dismax)[0]
        self.d = d
        self.l = l
        self.filtix = np.intersect1d(lenix,disix)

    def getAvg(self, dismax, lenlim, eps):
        self.eps = eps
        self.endpoints()        
        self.getFiltered(dismax, lenlim)
        atleast = 4
        if len(self.filtix) <= atleast:            
            distrank = np.argsort(self.d)
            self.disoutix = distrank[atleast:]
            self.lenoutix = []
            self.filtix = distrank[:atleast]
        filtered = [self.trajs[i] for i in self.filtix]
        trajLen = median([len(t.xs) for t in filtered])
        offs = np.linspace(0,1,trajLen*10)
        xm = []
        ym = []
        for t in filtered:
            xs, ys = t.getPoints(offs)            
            xm.append(xs)
            ym.append(ys)
        self.xp, self.yp = zip(*rdp(list(zip(np.median(xm, axis=0),np.median(ym, axis=0))), eps))
        #self.xp, self.yp = zip(*rdp(list(zip(np.mean(xm, axis=0),np.mean(ym, axis=0))), eps))
        #self.xp, self.yp = np.mean(xm, axis=0), np.mean(ym, axis=0)
        xp, yp = self.xp,self.yp
        return xp, yp
 
    def pax(self, ax):
        ax.set_xlim(0,2.5)
        ax.set_xticks([])
        ax.set_yticks([])        
        ax.set_ylim(0,130)
        for _, t in enumerate(self.trajs):    
            ax.plot(t.xs,t.ys, c="b", marker="o", markersize=2)
        for n, t in enumerate([self.trajs[i] for i in self.disoutix]):    
            ax.plot(t.xs,t.ys, c="g")
        for n, t in enumerate([self.trajs[i] for i in self.lenoutix]):    
            ax.plot(t.xs,t.ys, c="cyan")
        for n, t in enumerate([self.trajs[i] for i in np.intersect1d(self.lenoutix,self.disoutix)]):    
            ax.plot(t.xs,t.ys, c="magenta")
        if self.xp is not None:
            ax.plot(self.xp,self.yp, marker='D', color='r', linewidth=3)                

class Traj:
    def __init__(self,xsys):
        xs, ys = xsys
        a = np.array(xsys).T
        _, filtered = np.unique(a, return_index=True,axis=0)
        if len(filtered) < 2:
            raise OnlyOnePointError()
        self.xs = np.array(xs)[sorted(filtered)]
        self.ys = np.array(ys)[sorted(filtered)]
        self.xd = np.diff(xs)
        self.yd = np.diff(ys)
        self.dists = np.linalg.norm([self.xd, self.yd],axis=0)
        self.cuts = np.cumsum(self.dists)
        self.d = np.hstack([0,self.cuts])
        
    def getPoints(self, offsets):        
        offdists = offsets * self.cuts[-1]
        ix = np.searchsorted(self.cuts, offdists)        
        offdists -= self.d[ix]
        segoffs = offdists/self.dists[ix]
        x = self.xs[ix] + self.xd[ix]*segoffs
        y = self.ys[ix] + self.yd[ix]*segoffs
        return x,y     

# def compute_median_trajectory(posdataRight, timedataRight):
#     # eps, zmax, lenlim used in outlier detection. Here they are set so they don't exclude any outlier in the median computation. Outlying runs will be//are removed beforehand.
#     eps = 0.001
#     zmax = np.inf
#     lenlim=(-np.inf, np.inf)
#     data = list(zip([t - t[0] for t in timedataRight], posdataRight))

#     ss = SampleSet(data)
#     X, Y = ss.getAvg(zmax, lenlim, eps)

#     # Here median computation warps time (~Dynamic Time Warping) so interpolate to get back to 0.04s timestamps.
#     interpTime = np.linspace(X[0], X[-1], int(X[-1]/0.04)+1) # create time from 0 to median arrival time, evenly spaced 0.04s
#     interpPos = np.interp(interpTime, X, Y) # interpolate the position at interpTime
#     return interpTime, interpPos

def compute_median_trajectory(posdataRight, timedataRight):
    # eps, zmax, lenlim used in outlier detection. Here they are set so they don't exclude any outlier in the median computation. Outlying runs will be//are removed beforehand.
    eps = 0.001
    zmax = np.inf
    lenlim=(-np.inf, np.inf)
    data = list(zip([t - t[0] for t in timedataRight], posdataRight))

    ss = SampleSet(data)
    ss.getAvg(zmax, lenlim, eps) #does not do anything but has to be here to work ???????
    X, Y = ss.getAvg(zmax, lenlim, eps)

    # Here median computation warps time (~Dynamic Time Warping) so interpolate to get back to 0.04s timestamps.
    interpTime = np.linspace(X[0], X[-1], int(X[-1]/0.04)+1) # create time from 0 to median arrival time, evenly spaced 0.04s
    interpPos = np.interp(interpTime, X, Y) # interpolate the position at interpTime
    return interpTime, interpPos


# def compute_median_speed(posdataRight, timedataRight): 
#     eps = 0.001
#     zmax = np.inf
#     lenlim=(-np.inf, np.inf)
#     x1, x2, y1, y2 = [],[],[],[]

#     for i, j in zip(posdataRight, timedataRight):
#         iabs =  [abs(ele) for ele in i]
#         if len(np.where(i == max(i))[0]) == 1:
#             maxspeed = max(iabs)
#             maxspeedtime = np.where(iabs == maxspeed)[0]
#         xx = int(maxspeedtime)
#         xx1 = [_-j[0] for _ in j[:xx]]
#         yy1 = iabs[:xx]
#         xx2 = [_-j[0] for _ in j[xx-1:]]
#         yy2 = iabs[xx-1:]
#         #if maxspeed <= 250 and j[-1] >= 0.5 and i[0]<=100: ##### ugly filter
#         x1.append(xx1)
#         x2.append(xx2)
#         y1.append(yy1)
#         y2.append(yy2)
#         #else: print("deleted one", maxspeed, j[-1])

#     data1 = list(zip(x1, y1))
#     data2 = list(zip(x2, y2))
#     ss1 = SampleSet(data1)
#     ss2 = SampleSet(data2)
#     X1, Y1 = ss1.getAvg(zmax, lenlim, eps)
#     X2, Y2 = ss2.getAvg(zmax, lenlim, eps)

#     interpTime1 = np.linspace(X1[0], X1[-1], int((X1[-1]-X1[0])/0.04)+1) 
#     interpPos1  = np.interp(interpTime1, X1, Y1) 
#     interpTime2 = np.linspace(X2[0], X2[-1], int((X2[-1]-X2[0])/0.04)+1) 
#     interpPos2  = np.interp(interpTime2, X2, Y2) 
    
#     TIME = np.concatenate((interpTime1, interpTime2[1:]))
#     TRAJ = np.concatenate((interpPos1, interpPos2[1:]))
#     return TIME, TRAJ

In [ ]:
def separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, bin):

    if experiment == 'Distance':
        if bin == False:
            data90_60, data90_90, data90_120, data10_60, data10_90, data10_120 = ({} for _ in range(6))
            for animal in animal_list:
                data90_60[animal], data90_90[animal], data90_120[animal], data10_60[animal], data10_90[animal], data10_120[animal] = ([] for _ in range(6))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillDist'] == 60:
                        if datatype == 'nb_runs': 
                            data90_60[animal] = np.append(data90_60[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_60[animal] = np.append(data10_60[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_60[animal] = np.append(data90_60[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_60[animal] = np.append(data10_60[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillDist'] == 90:
                        if datatype == 'nb_runs':  
                            data90_90[animal] = np.append(data90_90[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_90[animal] = np.append(data10_90[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_90[animal] = np.append(data90_90[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_90[animal] = np.append(data10_90[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillDist'] == 120:  
                        if datatype == 'nb_runs': 
                            data90_120[animal] = np.append(data90_120[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_120[animal] = np.append(data10_120[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_120[animal] = np.append(data90_120[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_120[animal] = np.append(data10_120[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
            return data90_60, data90_90, data90_120, data10_60, data10_90, data10_120

        if bin == True:
            data60, data90, data120 = ({} for _ in range(3))
            for animal in animal_list:
                data60[animal], data90[animal], data120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(3))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillDist'] == 60:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data60[animal][i] = np.append(data60[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data60[animal][i] = np.append(data60[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])

                    if params[animal, session]['treadmillDist'] == 90:                    
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data90[animal][i] = np.append(data90[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data90[animal][i] = np.append(data90[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])

                    if params[animal, session]['treadmillDist'] == 120:                     
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data120[animal][i] = np.append(data120[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data120[animal][i] = np.append(data120[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
            return data60, data90, data120

    if experiment == 'TM_ON':
        if bin == False:
            data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = ({} for _ in range(12))
            for animal in animal_list:
                data90_rev20[animal], data90_rev10[animal], data90_rev2[animal], data90_2[animal], data90_10[animal], data90_20[animal], data10_rev20[animal], data10_rev10[animal], data10_rev2[animal], data10_2[animal], data10_10[animal], data10_20[animal] = ([] for _ in range(12))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:
                        if datatype == 'nb_runs': 
                            data90_rev20[animal] = np.append(data90_rev20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev20[animal] = np.append(data10_rev20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev20[animal] = np.append(data90_rev20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev20[animal] = np.append(data10_rev20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
                        if datatype == 'nb_runs': 
                            data90_rev10[animal] = np.append(data90_rev10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev10[animal] = np.append(data10_rev10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev10[animal] = np.append(data90_rev10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev10[animal] = np.append(data10_rev10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:
                        if datatype == 'nb_runs': 
                            data90_rev2[animal] = np.append(data90_rev2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev2[animal] = np.append(data10_rev2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev2[animal] = np.append(data90_rev2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev2[animal] = np.append(data10_rev2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:
                        if datatype == 'nb_runs':
                            data90_2[animal] = np.append(data90_2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_2[animal] = np.append(data10_2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_2[animal] = np.append(data90_2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_2[animal] = np.append(data10_2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:
                        if datatype == 'nb_runs':
                            data90_10[animal] = np.append(data90_10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_10[animal] = np.append(data10_10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_10[animal] = np.append(data90_10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_10[animal] = np.append(data10_10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:
                        if datatype == 'nb_runs': 
                            data90_20[animal] = np.append(data90_20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_20[animal] = np.append(data10_20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_20[animal] = np.append(data90_20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_20[animal] = np.append(data10_20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
            return data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20

        if bin == True:
            datarev20, datarev10, datarev2, data2, data10, data20 = ({} for _ in range(6))
            for animal in animal_list:
                datarev20[animal], datarev10[animal], datarev2[animal], data2[animal], data10[animal], data20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev20[animal][i] = np.append(datarev20[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): datarev20[animal][i] = np.append(datarev20[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev10[animal][i] = np.append(datarev10[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): datarev10[animal][i] = np.append(datarev10[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev2[animal][i] = np.append(datarev2[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): datarev2[animal][i] = np.append(datarev2[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data2[animal][i] = np.append(data2[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data2[animal][i] = np.append(data2[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data10[animal][i] = np.append(data10[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data10[animal][i] = np.append(data10[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data20[animal][i] = np.append(data20[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data20[animal][i] = np.append(data20[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
            return datarev20, datarev10, datarev2, data2, data10, data20

def across_session_plot(plot, animal_list, session_list, dataLeft, dataRight, experiment, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype): 
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16)
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12) 
    #if experiment == 'TM_ON': ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.yaxis.set_label_coords(-0.22,0.5)
    ax.yaxis.label.set_color('dodgerblue' if datatype == 'avgrunspeed' else 
                             'red' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')

    if experiment == 'Distance':
        data90_60, data90_90, data90_120, data10_60, data10_90, data10_120 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs': 
                if plot == "90%": ax.plot((60, 90, 120), (np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                    #ax.errorbar((60, 90, 120), (np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])), yerr = (stats.sem(data90_60[animal]), stats.sem(data90_90[animal]), stats.sem(data90_120[animal])), color = rat_markers[animal][0])
                if plot == "10%": ax.plot((60, 90, 120), (np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                    #ax.errorbar((60, 90, 120), (np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal])), yerr = (stats.sem(data10_60[animal]), stats.sem(data10_90[animal]), stats.sem(data10_120[animal])), color = rat_markers[animal][0])
                if plot == "%"  : ax.plot((60, 90, 120), ((np.mean(data10_60[animal])  / np.mean(data90_60[animal])), (np.mean(data10_90[animal])  / np.mean(data90_90[animal])), (np.mean(data10_120[animal]) / np.mean(data90_120[animal]))), marker = 'o', markersize = 6, color = rat_markers[animal][0])  
            else:
                if plot == "90%": ax.plot((60, 90, 120), (np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                    #ax.errorbar((60, 90, 120), (np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist])), yerr = (stats.sem([item for sublist in data90_60[animal]  for item in sublist]),  stats.sem([item for sublist in data90_90[animal]  for item in sublist]), stats.sem([item for sublist in data90_120[animal] for item in sublist])), color = rat_markers[animal][0])
                if plot == "10%": ax.plot((60, 90, 120), (np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                    #ax.errorbar((60, 90, 120), (np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist])), yerr = (stats.sem([item for sublist in data10_60[animal]  for item in sublist]),  stats.sem([item for sublist in data10_90[animal]  for item in sublist]), stats.sem([item for sublist in data10_120[animal] for item in sublist])), color = rat_markers[animal][0])
                if plot == "%"  : ax.plot((60, 90, 120), ((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_60[animal]  for item in sublist])), 
                                                         (np.nanmean([item for sublist in data10_90[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_90[animal]  for item in sublist])), 
                                                         (np.nanmean([item for sublist in data10_120[animal] for item in sublist]) / np.nanmean([item for sublist in data90_120[animal] for item in sublist]))), marker = 'o', markersize = 6, color = rat_markers[animal][0])

    if experiment == 'TM_ON':
        data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs': 
                if plot == "90%": 
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.mean(data90_rev20[animal]), np.mean(data90_rev10[animal]), np.mean(data90_rev2[animal]), 
                                                        np.mean(data90_2[animal]), np.mean(data90_10[animal]), np.mean(data90_20[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "10%": 
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.mean(data10_rev20[animal]), np.mean(data10_rev10[animal]), np.mean(data10_rev2[animal]), 
                                                        np.mean(data10_2[animal]), np.mean(data10_10[animal]), np.mean(data10_20[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "%" : 
                    ax.plot((-20, -10, -2, 2, 10, 20), ((np.mean(data10_rev20[animal]) / np.mean(data90_rev20[animal])), (np.mean(data10_rev10[animal]) / np.mean(data90_rev10[animal])), 
                                                        (np.mean(data10_rev2[animal]) / np.mean(data90_rev2[animal])), (np.mean(data10_2[animal]) / np.mean(data90_2[animal])), 
                                                        (np.mean(data10_10[animal]) / np.mean(data90_10[animal])), (np.mean(data10_20[animal]) / np.mean(data90_20[animal]))), marker = 'o', markersize = 6, color = rat_markers[animal][0])   
            else:
                if plot == "90%":
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.nanmean([item for sublist in data90_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev10[animal] for item in sublist]), 
                                                        np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data90_2[animal]     for item in sublist]), 
                                                        np.nanmean([item for sublist in data90_10[animal]    for item in sublist]), np.nanmean([item for sublist in data90_20[animal]    for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "10%": 
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]), 
                                                        np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data10_2[animal]     for item in sublist]), 
                                                        np.nanmean([item for sublist in data10_10[animal]    for item in sublist]), np.nanmean([item for sublist in data10_20[animal]    for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "%" : 
                    ax.plot((-20, -10, -2, 2, 10, 20), ((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev20[animal] for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev10[animal] for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_2[animal]     for item in sublist]) / np.nanmean([item for sublist in data90_2[animal]     for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_10[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_10[animal]    for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_20[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_20[animal]    for item in sublist]))), marker = 'o', markersize = 6, color = rat_markers[animal][0])
    return ax

def across_Binsession_plot(plot, animal_list, session_list, dataLeft, dataRight, experiment, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype): 
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16)
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12)
    ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.yaxis.label.set_color('dodgerblue' if datatype == 'avgrunspeed' else 
                             'dodgerblue' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')

    if experiment == 'Distance':
        data60, data90, data120 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, True)
        for animal in animal_list:
            if plot == "60" : ax.plot([x+1 for x in data60[animal].keys()], [np.mean(i) for i in data60[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "90" : ax.plot([x+1 for x in data90[animal].keys()], [np.mean(i) for i in data90[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "120": ax.plot([x+1 for x in data120[animal].keys()],[np.mean(i) for i in data120[animal].values()],marker = 'o', markersize = 6, color = rat_markers[animal][0])  
        
    if experiment == 'TM_ON':
        datarev20, datarev10, datarev2, data2, data10, data20 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, True)
        for animal in animal_list:      
            if plot == "-20" : ax.plot([x+1 for x in datarev20[animal].keys()], [np.mean(i) for i in datarev20[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "-10" : ax.plot([x+1 for x in datarev10[animal].keys()], [np.mean(i) for i in datarev10[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "-2"  : ax.plot([x+1 for x in datarev2[animal].keys()],  [np.mean(i) for i in datarev2[animal].values()],  marker = 'o', markersize = 6, color = rat_markers[animal][0])  
            if plot == "2"   : ax.plot([x+1 for x in data2[animal].keys()],     [np.mean(i) for i in data2[animal].values()],     marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "10"  : ax.plot([x+1 for x in data10[animal].keys()],    [np.mean(i) for i in data10[animal].values()],    marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "20"  : ax.plot([x+1 for x in data20[animal].keys()],    [np.mean(i) for i in data20[animal].values()],    marker = 'o', markersize = 6, color = rat_markers[animal][0])  
    return ax

def rats_Binsession_plot(plot, animal_list, session_list, dataLeft, dataRight, experiment, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype): 
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16)
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12)
    ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.yaxis.label.set_color('dodgerblue' if datatype == 'avgrunspeed' else 
                             'dodgerblue' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')

    if experiment == 'Distance':
        data60, data90, data120 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, True)
        animal = plot
        ax.plot([x+1 for x in data60[animal].keys()],  [np.mean(i) for i in data60[animal].values()],  marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 0.5, label = "60")
        ax.plot([x+1 for x in data90[animal].keys()],  [np.mean(i) for i in data90[animal].values()],  marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 1.25, label = "90")
        ax.plot([x+1 for x in data120[animal].keys()], [np.mean(i) for i in data120[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 2, label = "120")  
        legend_without_duplicate_labels(ax)
        
    if experiment == 'TM_ON':
        datarev20, datarev10, datarev2, data2, data10, data20 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, True)
        animal = plot                  
        ax.plot([x+1 for x in datarev20[animal].keys()], [np.mean(i) for i in datarev20[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 2, label ="-20", linestyle=":")
        ax.plot([x+1 for x in datarev10[animal].keys()], [np.mean(i) for i in datarev10[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 1.5, label ="-10", linestyle=":")
        ax.plot([x+1 for x in datarev2[animal].keys()],  [np.mean(i) for i in datarev2[animal].values()],  marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 1, label ="-2", linestyle=":")  
        ax.plot([x+1 for x in data2[animal].keys()],     [np.mean(i) for i in data2[animal].values()],     marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 1, label ="2")
        ax.plot([x+1 for x in data10[animal].keys()],    [np.mean(i) for i in data10[animal].values()],    marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 1.5, label ="10")
        ax.plot([x+1 for x in data20[animal].keys()],    [np.mean(i) for i in data20[animal].values()],    marker = 'o', markersize = 6, color = rat_markers[animal][0], linewidth = 2, label ="20")  
        legend_without_duplicate_labels(ax)
    return ax

def rwd_plot(plot, animal_list, session_list, dataLeft, dataRight, experiment, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype): 
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16)
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12) 
    if experiment == 'TM_ON': ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.axvspan(-35, 50, color='grey', alpha=10/250, label = "%reward: 90%", zorder = 1)
    ax.axvspan(50, 135, color='grey', alpha=90/250, label = "%reward: 10%", zorder = 1)
    ax.yaxis.label.set_color('dodgerblue' if datatype == 'avgrunspeed' else 
                             'dodgerblue' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')
    if experiment == 'Distance':
        _, data90_90, _, _, data10_90, _ = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs': ax.plot((10, 90), (np.mean(data10_90[animal]), np.mean(data90_90[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0], zorder = 2)
            else: ax.plot((10, 90), (np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_90[animal]  for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0], zorder = 2)      
    return ax

def corr_twoExps(plot, animal_list, dataLeft, dataRight, session_list1, session_list2, experiment1, experiment2, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype):
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16, pad = 20, color = 'dodgerblue' if datatype == 'avgrunspeed' else 
                             'dodgerblue' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12) 
    #if experiment == 'TM_ON': ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)

    xdata, ydata = {}, {}
    if experiment1 == 'Distance':
        data90_60, data90_90, data90_120, data10_60, data10_90, data10_120 = separate_data(animal_list, session_list1, dataLeft, dataRight, experiment1, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs':  
                if plot == "90%": xdata[animal] = np.mean((np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])))
                if plot == "10%": xdata[animal] = np.mean((np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal])))
                if plot == "%"  : xdata[animal] = np.mean(((np.mean(data10_60[animal])  / np.mean(data90_60[animal])), (np.mean(data10_90[animal])  / np.mean(data90_90[animal])), (np.mean(data10_120[animal]) / np.mean(data90_120[animal]))))
            else: 
                if plot == "90%": xdata[animal] = np.mean((np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist])))
                if plot == "10%": xdata[animal] = np.mean((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist])))
                if plot == "%"  : xdata[animal] = np.mean(((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_60[animal]  for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_90[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_90[animal]  for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_120[animal] for item in sublist]) / np.nanmean([item for sublist in data90_120[animal] for item in sublist]))))
    if experiment1 == 'TM_ON':
        data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = separate_data(animal_list, session_list1, dataLeft, dataRight, experiment1, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs':
                if plot == "90%": xdata[animal] = np.mean((np.mean(data90_rev20[animal]), np.mean(data90_rev10[animal]), np.mean(data90_rev2[animal]), np.mean(data90_2[animal]), np.mean(data90_10[animal]), np.mean(data90_20[animal])))
                if plot == "10%": xdata[animal] = np.mean((np.mean(data10_rev20[animal]), np.mean(data10_rev10[animal]), np.mean(data10_rev2[animal]), np.mean(data10_2[animal]), np.mean(data10_10[animal]), np.mean(data10_20[animal])))
                if plot == "%"  : xdata[animal] = np.mean(((np.mean(data10_rev20[animal]) / np.mean(data90_rev20[animal])), (np.mean(data10_rev10[animal]) / np.mean(data90_rev10[animal])), 
                                                           (np.mean(data10_rev2[animal]) / np.mean(data90_rev2[animal])), (np.mean(data10_2[animal]) / np.mean(data90_2[animal])), 
                                                           (np.mean(data10_10[animal]) / np.mean(data90_10[animal])), (np.mean(data10_20[animal]) / np.mean(data90_20[animal]))))   
            else:
                if plot == "90%": xdata[animal] = np.mean((np.nanmean([item for sublist in data90_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev10[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data90_2[animal]     for item in sublist]), np.nanmean([item for sublist in data90_10[animal]    for item in sublist]), np.nanmean([item for sublist in data90_20[animal]    for item in sublist])))
                if plot == "10%": xdata[animal] = np.mean((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data10_2[animal]     for item in sublist]), np.nanmean([item for sublist in data10_10[animal]    for item in sublist]), np.nanmean([item for sublist in data10_20[animal]    for item in sublist])))
                if plot == "%"  : xdata[animal] = np.mean(((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev20[animal] for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev10[animal] for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_2[animal]     for item in sublist]) / np.nanmean([item for sublist in data90_2[animal]     for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_10[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_10[animal]    for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_20[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_20[animal]    for item in sublist]))))
    if experiment2 == 'Distance':
        data90_60, data90_90, data90_120, data10_60, data10_90, data10_120 = separate_data(animal_list, session_list2, dataLeft, dataRight, experiment2, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs':  
                if plot == "90%": ydata[animal] = np.mean((np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])))
                if plot == "10%": ydata[animal] = np.mean((np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal])))
                if plot == "%"  : ydata[animal] = np.mean(((np.mean(data10_60[animal])  / np.mean(data90_60[animal])), (np.mean(data10_90[animal])  / np.mean(data90_90[animal])), (np.mean(data10_120[animal]) / np.mean(data90_120[animal]))))
            else: 
                if plot == "90%": ydata[animal] = np.mean((np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist])))
                if plot == "10%": ydata[animal] = np.mean((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist])))
                if plot == "%"  : ydata[animal] = np.mean(((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_60[animal]  for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_90[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_90[animal]  for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_120[animal] for item in sublist]) / np.nanmean([item for sublist in data90_120[animal] for item in sublist]))))
    if experiment2 == 'TM_ON':
        data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = separate_data(animal_list, session_list2, dataLeft, dataRight, experiment2, datatype, False)
        for animal in animal_list:
            if datatype == 'nb_runs':
                if plot == "90%": ydata[animal] = np.mean((np.mean(data90_rev20[animal]), np.mean(data90_rev10[animal]), np.mean(data90_rev2[animal]), np.mean(data90_2[animal]), np.mean(data90_10[animal]), np.mean(data90_20[animal])))
                if plot == "10%": ydata[animal] = np.mean((np.mean(data10_rev20[animal]), np.mean(data10_rev10[animal]), np.mean(data10_rev2[animal]), np.mean(data10_2[animal]), np.mean(data10_10[animal]), np.mean(data10_20[animal])))
                if plot == "%"  : ydata[animal] = np.mean(((np.mean(data10_rev20[animal]) / np.mean(data90_rev20[animal])), (np.mean(data10_rev10[animal]) / np.mean(data90_rev10[animal])), 
                                                           (np.mean(data10_rev2[animal]) / np.mean(data90_rev2[animal])), (np.mean(data10_2[animal]) / np.mean(data90_2[animal])), 
                                                           (np.mean(data10_10[animal]) / np.mean(data90_10[animal])), (np.mean(data10_20[animal]) / np.mean(data90_20[animal]))))   
            else:
                if plot == "90%": ydata[animal] = np.mean((np.nanmean([item for sublist in data90_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev10[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data90_2[animal]     for item in sublist]), np.nanmean([item for sublist in data90_10[animal]    for item in sublist]), np.nanmean([item for sublist in data90_20[animal] for item in sublist])))
                if plot == "10%": ydata[animal] = np.mean((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data10_2[animal]     for item in sublist]), np.nanmean([item for sublist in data10_10[animal]    for item in sublist]), np.nanmean([item for sublist in data10_20[animal] for item in sublist])))
                if plot == "%"  : ydata[animal] = np.mean(((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev20[animal] for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev10[animal] for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_2[animal]     for item in sublist]) / np.nanmean([item for sublist in data90_2[animal]     for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_10[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_10[animal]    for item in sublist])), 
                                                           (np.nanmean([item for sublist in data10_20[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_20[animal]    for item in sublist]))))

    for animal in animal_list:    
        ax.scatter(xdata[animal], ydata[animal], color = rat_markers[animal][0])
    gradient, intercept, r_value, p_value, std_err = stats.linregress(list(xdata.values()), list(ydata.values()))
    ax.plot((0, 200), (0, 200), alpha=0.5, color='k')
    ax.plot(np.linspace(np.min(list(xdata.values())), np.max(list(xdata.values())), 500), gradient * np.linspace(np.min(list(xdata.values())), np.max(list(xdata.values())), 500) + intercept, color = 'k', lw = 2)
    #ax.annotate(("r=%.2f, \np=%.2f" %(r_value, p_value)), (np.max(list(xdata.values())), gradient * np.max(list(xdata.values()))+ intercept))
    return ax

In [ ]:
def compute_median_trajectoryOLD(posdataRight, timedataRight, cutoff):
    try : maxL = np.nanmax([len(j) for j in timedataRight]) # determine maximum lenght of run
    except TypeError: maxL = 1
    except ValueError: maxL = 1
    avgPos = np.ones((len(timedataRight), maxL))*np.nan     # initialise matrix
    for i, p in enumerate(posdataRight): avgPos[i, 0:len(p)] = p    # fill matrix with all run data
    nanSum = np.sum(np.isnan(avgPos), axis = 0)
    try: end_medTraj = np.where(nanSum > cutoff * avgPos.shape[0])[0][0]     # cut median run after CUTOFF missing values == cut median run lenght when CUTOFF% of runs have ended
    except IndexError: end_medTraj = avgPos.shape[1]
    time = np.linspace(0, end_medTraj*0.04, end_medTraj)
    traj = np.nanmedian(avgPos, axis = 0)[:end_medTraj]
    ci = (np.nanstd(avgPos, axis = 0)[:end_medTraj])/(np.sqrt(np.count_nonzero(~np.isnan(avgPos), axis = 0)[:end_medTraj]))
    return time, traj, ci 


def plot_trackstest(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    n = len(posdataRight)
    colors = 'g'*n #plt.cm.RdYlGn_r(np.linspace(0,1,n)) #green early, red late
    for index, (i, j) in enumerate(zip(posdataRight, timedataRight)):
        tracks = ax.plot(np.subtract(j, j[0]), i, color=colors[index] , linewidth=0.2) #[p if 16<p<90 else None for p in i]
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    return ax    

cutoff = 0.3
animal = 'RatF02'
session = matchsession(animal, dist120)[5]
fig = plt.figure(constrained_layout = False, figsize=(5,10))
ax = plt.subplot(211)
ax = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0, 0],xylim = [-0.1, 2, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
time, traj = compute_median_trajectory(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])
ax = plt.plot(time, traj, color= 'k', lw=1.5, alpha=1, label="new")
#ax = plt.fill_between(time, traj-ci, traj+ci, color ='r', zorder = 3, alpha = 1/4) 
for i in np.linspace(0, 100, 11):
    time, traj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], (100-i)/100)
    plt.plot(time, traj, lw=1.5, label="{0} % of data left".format(int(100-i)))#print(time)
    plt.legend()


ax1 = plt.subplot(212)
ax1 = plot_speed(animal, XtrackRunToRightBin[animal, session][0], instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0,0], xylim = [-0.1, 2, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["", 12], linewidth = [1.5])
time2, traj2 = compute_median_trajectory(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])
ax1 = plt.plot(time2, traj2, color= 'k', lw=1.5, alpha=1, label="new")
#ax1 = plt.fill_between(time, traj-ci, traj+ci, color ='r', zorder = 3, alpha = 1/4) 
for i in np.linspace(0, 100, 11):
    time, traj, ci = compute_median_trajectoryOLD(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], (100-i)/100)
    plt.plot(time, traj, lw=1.5, label="{0} % of data left".format(int(100-i)))#print(time)
    plt.legend()


In [ ]:


cutoff = 0.3
animal = 'RatF02'
session = matchsession(animal, dist120)[5]
fig = plt.figure(constrained_layout = False, figsize=(12,12))

ax = plt.subplot(221)
ax = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0, 0],xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
time, traj = compute_median_trajectory(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])
ax = plt.plot(time, traj, color= 'k', lw=1.5, alpha=1, label="new")

ax0 = plt.subplot(222)
ax0 = plot_trackstest(animal, session, XtrackRunToLeftBin[animal, session][0], trackTimeToLeftBin[animal, session][0], [0, 0],xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Left",  16], linewidth = [1.5]) 
time, traj = compute_median_trajectory(XtrackRunToLeftBin[animal, session][0], trackTimeToLeftBin[animal, session][0])
ax0 = plt.plot(time, traj, color= 'k', lw=1.5, alpha=1, label="new")


ax1 = plt.subplot(223)
ax1 = plot_speed(animal, XtrackRunToRightBin[animal, session][0], instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0,0], xylim = [-0.1, 2.5, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["", 12], linewidth = [1.5])
time2, traj2 = compute_median_trajectory(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])
ax1 = plt.plot(time2, traj2, color= 'k', lw=1.5, alpha=1, label="new")



ax2 = plt.subplot(224)
ax2 = plot_speed(animal, XtrackRunToLeftBin[animal, session][0], instantSpeedLeftBin[animal, session][0], trackTimeToLeftBin[animal, session][0], [0,0], xylim = [-0.1, 2.5, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["", 12], linewidth = [1.5])
time2, traj2 = compute_median_trajectory(instantSpeedLeftBin[animal, session][0], trackTimeToLeftBin[animal, session][0])
ax2 = plt.plot(time2, traj2, color= 'k', lw=1.5, alpha=1, label="new")




In [ ]:
def is_list(data):
    if isinstance(data, float):
        return False
    elif isinstance(data, list):
        if len(data) > 1:
            return True
        else:
            #print(data, 'only one')
            return False

def data_to_add(data1, data2):
    if len(data1)>1 and len(data2)>1: 
        return [data1 + data2]
    elif len(data1) > 1 and len(data2) <= 1: 
        print("case01")
        return [data1]
    elif len(data1) <= 1 and len(data2) > 1: 
        print("case10")
        return [data2]
    elif len(data1) <= 1 and len(data2) <= 1: 
        return [data1 + data2]
        #print("case00", data1, data2, [data1], [data2])
        #pass



In [ ]:
nb_runsBin['RatM02',
  'RatM02_2021_08_05_12_33_53']

In [ ]:
" blocks with 0 runs"
res = []
for k, v in nb_runsBin.items(): 
    for kk, vv in v.items(): 
        if vv <= 2: 
            print(k, kk)
            res.append(kk+.5)
plt.hist(res, bins=[i+.5 for i in range(0, 13)])


In [ ]:
" blocks with 0 runs"
res = []
for k, v in nb_runsBin.items(): 
    for kk, vv in v.items(): 
        if vv == 0: 
            print(k, kk)
            res.append(kk+.5)
plt.hist(res, bins=[i+.5 for i in range(0, 13)])


In [ ]:
def separate_datatest(animal_list, session_list, dataLeft, dataRight, experiment, datatype, bin):



    if experiment == 'Distance':
        if bin == False:
            data90_60, data90_90, data90_120, data10_60, data10_90, data10_120 = ({} for _ in range(6))
            for animal in animal_list:
                data90_60[animal], data90_90[animal], data90_120[animal], data10_60[animal], data10_90[animal], data10_120[animal] = ([] for _ in range(6))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillDist'] == 60:
                        if datatype == 'nb_runs': 
                            data90_60[animal] = np.append(data90_60[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_60[animal] = np.append(data10_60[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_60[animal] = np.append(data90_60[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_60[animal] = np.append(data10_60[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillDist'] == 90:
                        if datatype == 'nb_runs':  
                            data90_90[animal] = np.append(data90_90[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_90[animal] = np.append(data10_90[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_90[animal] = np.append(data90_90[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_90[animal] = np.append(data10_90[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillDist'] == 120:  
                        if datatype == 'nb_runs': 
                            data90_120[animal] = np.append(data90_120[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_120[animal] = np.append(data10_120[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_120[animal] = np.append(data90_120[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_120[animal] = np.append(data10_120[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
            return data90_60, data90_90, data90_120, data10_60, data10_90, data10_120

        if bin == True:
            data60, data90, data120 = ({} for _ in range(3))
            for animal in animal_list:
                data60[animal], data90[animal], data120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(3))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillDist'] == 60:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                data60[animal][i] = np.append(data60[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): #data60[animal][i] = np.append(data60[animal][i], data_to_add(dataRight[animal, session][i], dataLeft[animal, session][i]))
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]): #maybe and instead of or///#if i==5 : print(session, i, dataRight[animal, session][i], dataLeft[animal, session][i], is_list(dataRight[animal, session][i]), is_list( dataLeft[animal, session][i]))
                                    data60[animal][i] = np.append(data60[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                                #else: print(session, i, dataRight[animal, session][i], dataLeft[animal, session][i], is_list(dataRight[animal, session][i]), is_list( dataLeft[animal, session][i]))

                    if params[animal, session]['treadmillDist'] == 90:                    
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data90[animal][i] = np.append(data90[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    data90[animal][i] = np.append(data90[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])

                    if params[animal, session]['treadmillDist'] == 120:                     
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data120[animal][i] = np.append(data120[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    data120[animal][i] = np.append(data120[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
            return data60, data90, data120

    if experiment == 'TM_ON':
        if bin == False:
            data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = ({} for _ in range(12))
            for animal in animal_list:
                data90_rev20[animal], data90_rev10[animal], data90_rev2[animal], data90_2[animal], data90_10[animal], data90_20[animal], data10_rev20[animal], data10_rev10[animal], data10_rev2[animal], data10_2[animal], data10_10[animal], data10_20[animal] = ([] for _ in range(12))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:
                        if datatype == 'nb_runs': 
                            data90_rev20[animal] = np.append(data90_rev20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev20[animal] = np.append(data10_rev20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev20[animal] = np.append(data90_rev20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev20[animal] = np.append(data10_rev20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
                        if datatype == 'nb_runs': 
                            data90_rev10[animal] = np.append(data90_rev10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev10[animal] = np.append(data10_rev10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev10[animal] = np.append(data90_rev10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev10[animal] = np.append(data10_rev10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:
                        if datatype == 'nb_runs': 
                            data90_rev2[animal] = np.append(data90_rev2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev2[animal] = np.append(data10_rev2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev2[animal] = np.append(data90_rev2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev2[animal] = np.append(data10_rev2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:
                        if datatype == 'nb_runs':
                            data90_2[animal] = np.append(data90_2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_2[animal] = np.append(data10_2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_2[animal] = np.append(data90_2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_2[animal] = np.append(data10_2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:
                        if datatype == 'nb_runs':
                            data90_10[animal] = np.append(data90_10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_10[animal] = np.append(data10_10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_10[animal] = np.append(data90_10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_10[animal] = np.append(data10_10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:
                        if datatype == 'nb_runs': 
                            data90_20[animal] = np.append(data90_20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_20[animal] = np.append(data10_20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_20[animal] = np.append(data90_20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_20[animal] = np.append(data10_20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
            return data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20

        if bin == True:
            datarev20, datarev10, datarev2, data2, data10, data20 = ({} for _ in range(6))
            for animal in animal_list:
                datarev20[animal], datarev10[animal], datarev2[animal], data2[animal], data10[animal], data20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev20[animal][i] = np.append(datarev20[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    datarev20[animal][i] = np.append(datarev20[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev10[animal][i] = np.append(datarev10[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    datarev10[animal][i] = np.append(datarev10[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev2[animal][i] = np.append(datarev2[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    datarev2[animal][i] = np.append(datarev2[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data2[animal][i] = np.append(data2[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    data2[animal][i] = np.append(data2[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data10[animal][i] = np.append(data10[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    data10[animal][i] = np.append(data10[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data20[animal][i] = np.append(data20[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): 
                                if is_list(dataRight[animal, session][i]) and is_list(dataLeft[animal, session][i]):
                                    data20[animal][i] = np.append(data20[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
            return datarev20, datarev10, datarev2, data2, data10, data20


data60, data90, data120 = separate_datatest(animalList, dist60+dist90+dist120, XtrackRunToRightBin, XtrackRunToRightBin, 'Distance', 'avgrunspeed', True)
datat60, datat90, datat120 = separate_datatest(animalList, dist60+dist90+dist120, trackTimeToRightBin, trackTimeToRightBin, 'Distance', 'avgrunspeed', True)

sdata60, sdata90, sdata120 = separate_datatest(animalList, dist60+dist90+dist120, instantSpeedRightBin, instantSpeedRightBin, 'Distance', 'avgrunspeed', True)
sdatat60, sdatat90, sdatat120 = separate_datatest(animalList, dist60+dist90+dist120, trackTimeToRightBin, trackTimeToRightBin, 'Distance', 'avgrunspeed', True)




datarev20, datarev10, datarev2, data2, data10, data20 = separate_datatest(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, XtrackRunToRightBin, XtrackRunToRightBin, 'TM_ON', 'avgrunspeed', True)
datarevt20, datarevt10, datarevt2, datat2, datat10, datat20 = separate_datatest(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, trackTimeToRightBin, trackTimeToRightBin, 'TM_ON', 'avgrunspeed', True)

sdatarev20, sdatarev10, sdatarev2, sdata2, sdata10, sdata20 = separate_datatest(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, instantSpeedRightBin, instantSpeedRightBin, 'TM_ON', 'avgrunspeed', True)
sdatarevt20, sdatarevt10, sdatarevt2, sdatat2, sdatat10, sdatat20 = separate_datatest(animalList, TM20+TM10+TM2+TMrev2+TMrev10+TMrev20, trackTimeToRightBin, trackTimeToRightBin, 'TM_ON', 'avgrunspeed', True)


In [ ]:
(sdata10['RatM02'][9])

In [ ]:
# for k, l in zip(b, a): plt.plot(k-k[0],l, c='g', lw=0.1)

# ts = list(zip([t - t[0] for t in sdatat90[animal][i]], sdata90[animal][i]))

# def sho(ts):
#     eps = 0.001  #.0755
#     zmax = np.inf #2.13
#     lenlim=(-np.inf, np.inf) #(-1.23,1.8129)
#     ss = SampleSet(ts)
#     ss.getAvg(zmax, lenlim, eps) #get filtered median
#     #print(ss.getAvg(zmax, lenlim, eps))
#     #ss.getRawAvg() #get raw median
#     X, Y = ss.getAvg(zmax, lenlim, eps)
#     #print("xy", X, Y)
#     return X, Y

# def compute_median_trajectory(posdataRight, timedataRight):
#     # eps, zmax, lenlim used in outlier detection. Here they are set so they don't exclude any outlier in the median computation. Outlying runs will be//are removed beforehand.
#     eps = 0.001
#     zmax = np.inf
#     lenlim=(-np.inf, np.inf)
#     data = list(zip([t - t[0] for t in timedataRight], posdataRight))

#     ss = SampleSet(data)
#     ss.getAvg(zmax, lenlim, eps) #does not do anything but has to be here to work ???
#     X, Y = ss.getAvg(zmax, lenlim, eps)

#     # Here median computation warps time (~Dynamic Time Warping) so interpolate to get back to 0.04s timestamps.
#     interpTime = np.linspace(X[0], X[-1], int(X[-1]/0.04)+1) # create time from 0 to median arrival time, evenly spaced 0.04s
#     interpPos = np.interp(interpTime, X, Y) # interpolate the position at interpTime
#     return interpTime, interpPos


# #ss = SampleSet(ts)
# Xc, Yc = sho(ts)
# #print("z", Xc, Yc)
# interpTime = np.linspace(0, Xc[-1], int(Xc[-1]/0.04)+1) # create time from 0 to median arrival time, evenly spaced 0.04s
# interpPos = np.interp(interpTime, Xc, Yc) # interpolate the position at interpTime
# plt.plot(interpTime, interpPos)
# print(interpTime, interpPos)

# time2, traj2 = compute_median_trajectory(sdata90[animal][i], sdatat90[animal][i])
# ax = plt.plot(time2, traj2, color= 'k', lw=3, alpha=1) 
# print(time2, traj2)

In [ ]:
# a =[r for r in sdata90[animal][i] if len(r)<250]
# b =[t for t, r in zip(sdatat90[animal][i], sdata90[animal][i]) if len(r)<250]

In [ ]:
# def cut_long(data1, data2, cut):
#     time = cut/.04
#     print(time)
#     good1 = [r for r in data1 if len(r)<time]
#     good2 = [t for t, r in zip(data2, data1) if len(r)<time]
#     bad1 = [r for r in data1 if len(r)>=time]
#     bad2 = [t for t, r in zip(data2, data1) if len(r)>=time]
#     return good1, good2, bad1, bad2

# #for k, l in zip(datat120[animal][i], data120[animal][i]): plt.plot(k-k[0],l, c='r', lw=0.1)
# good1, good2, bad1, bad2 = cut_long(data120[animal][i], datat120[animal][i], 4)
# for k, l in zip(good2, good1): plt.plot(k-k[0],l, c='g', lw=0.1)
# for k, l in zip(bad2, bad1): plt.plot(k-k[0],l, c='r', lw=0.1)

In [ ]:
# def cut_long(X, Y, cut):
#     time = int(cut/.04)
#     goodY = [r for r in Y if len(r)<time]
#     goodX = [t for t, r in zip(X, Y) if len(r)<time]
#     badY = [r for r in Y if len(r)>=time]
#     badX = [t for t, r in zip(X, Y) if len(r)>=time]
#     return goodX, goodY, badX, badY

# fig = plt.figure(constrained_layout = False, figsize=(30, 20))
# gs = fig.add_gridspec(len(animalList), 12)
# for index, animal in enumerate(animalList):
#     gs00 = gs[index, 0:12].subgridspec(1,12)
#     for i in range(0, 12):
#         axs= fig.add_subplot(gs00[0, i])
#         axs.set_xlim(0, 2.5)

#         goodt60, goodp60, badt60, badp60 = cut_long(datat60[animal][i], data60[animal][i], 4)
#         goodt90, goodp90, badt90, badp90 = cut_long(datat90[animal][i], data90[animal][i], 4)
#         goodt120, goodp120, badt120, badp120 = cut_long(datat120[animal][i], data120[animal][i], 4)


#         for k, l in zip(goodt120, goodp120): plt.plot(k-k[0],l, c='b', lw=0.1)
#         for k, l in zip(goodt90, goodp90): plt.plot(k-k[0],l, c='g', lw=0.1)
#         for k, l in zip(goodt60, goodp60): plt.plot(k-k[0],l, c='c', lw=0.1)

#         for k, l in zip(badt120, badp120): plt.plot(k-k[0],l, c='r', lw=0.1)
#         for k, l in zip(badt90, badp90): plt.plot(k-k[0],l, c='r', lw=0.1)
#         for k, l in zip(badt60, badp60): plt.plot(k-k[0],l, c='r', lw=0.1)
            
#         time, traj = compute_median_trajectory(goodp60, goodt60)
#         ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
#         time, traj = compute_median_trajectory(goodp90, goodt90)
#         ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
#         time, traj = compute_median_trajectory(goodp120, goodt120)
#         ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
#         # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
#         # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:

fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2.5)

        for k, l in zip(datat120[animal][i], data120[animal][i]): plt.plot(k-k[0],l, c='r', lw=0.1)
        for k, l in zip(datat90[animal][i], data90[animal][i]): plt.plot(k-k[0],l, c='g', lw=0.1)
        for k, l in zip(datat60[animal][i], data60[animal][i]): plt.plot(k-k[0],l, c='c', lw=0.1)
            
        time, traj = compute_median_trajectory(data60[animal][i], datat60[animal][i])
        ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
        time, traj = compute_median_trajectory(data90[animal][i], datat90[animal][i])
        ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
        time, traj = compute_median_trajectory(data120[animal][i], datat120[animal][i])
        ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
        # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
        # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2.5)
        axs.set_ylim(0, 150)

        for k, l in zip(sdatat120[animal][i], sdata120[animal][i]): plt.plot(k-k[0],l, c='r', lw=0.1)
        for k, l in zip(sdatat90[animal][i], sdata90[animal][i]): plt.plot(k-k[0],l, c='g', lw=0.1)
        for k, l in zip(sdatat60[animal][i], sdata60[animal][i]): plt.plot(k-k[0],l, c='c', lw=0.1)
            
        time, traj = compute_median_trajectory(sdata60[animal][i], sdatat60[animal][i])
        ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
        time, traj = compute_median_trajectory(sdata90[animal][i], sdatat90[animal][i])
        ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
        time, traj = compute_median_trajectory(sdata120[animal][i], sdatat120[animal][i])
        ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
        # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
        # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:
# def cut_long(X, Y, cut):
#     time = int(cut/.04)
#     goodY = [r for r in Y if len(r)<time]
#     goodX = [t for t, r in zip(X, Y) if len(r)<time]
#     badY = [r for r in Y if len(r)>=time]
#     badX = [t for t, r in zip(X, Y) if len(r)>=time]
#     return goodX, goodY, badX, badY

# fig = plt.figure(constrained_layout = False, figsize=(30, 20))
# gs = fig.add_gridspec(len(animalList), 12)
# for index, animal in enumerate(animalList):
#     gs00 = gs[index, 0:12].subgridspec(1,12)
#     for i in range(0, 12):
#         axs= fig.add_subplot(gs00[0, i])
#         axs.set_xlim(0, 2.5)
#         axs.set_ylim(0, 150)

#         goodt60, goodp60, badt60, badp60 = cut_long(sdatat60[animal][i], sdata60[animal][i], 4)
#         goodt90, goodp90, badt90, badp90 = cut_long(sdatat90[animal][i], sdata90[animal][i], 4)
#         goodt120, goodp120, badt120, badp120 = cut_long(sdatat120[animal][i], sdata120[animal][i], 4)


#         for k, l in zip(goodt120, goodp120): plt.plot(k-k[0],l, c='b', lw=0.1)
#         for k, l in zip(goodt90, goodp90): plt.plot(k-k[0],l, c='g', lw=0.1)
#         for k, l in zip(goodt60, goodp60): plt.plot(k-k[0],l, c='c', lw=0.1)

#         for k, l in zip(badt120, badp120): plt.plot(k-k[0],l, c='r', lw=0.1)
#         for k, l in zip(badt90, badp90): plt.plot(k-k[0],l, c='r', lw=0.1)
#         for k, l in zip(badt60, badp60): plt.plot(k-k[0],l, c='r', lw=0.1)

#         try:            
#             time, traj = compute_median_speed(goodp60, goodt60)
#             ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
#             time, traj = compute_median_speed(goodp90, goodt90)
#             ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
#             time, traj = compute_median_speed(goodp120, goodt120)
#             ax = plt.plot(time, traj, color= 'k', lw=3, alpha=1) 
#         except: ValueError
#         # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
#         # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:

fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for session in matchsession(animal, dist120): # for session in [matchsession(animal, dist120)[0]]:
        for i in range(0, 12):
            axs= fig.add_subplot(gs00[0, i])
            axs.set_xlim(0, 2.5)
            #ax = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], [0, 0],xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=[i,  16], linewidth = [1.5]) 
            if len(XtrackRunToRightBin[animal, session][i]):
                time, traj = compute_median_trajectory(XtrackRunToRightBin[animal, session][i], trackTimeToRightBin[animal, session][i])
                ax = plt.plot(time, traj, color= 'c', lw=1, alpha=1) 
                # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
                # plt.plot(oldtime, oldtraj, lw=1.5, c='r')
    for session in matchsession(animal, dist90): # for session in [matchsession(animal, dist120)[0]]:
        for i in range(0, 12):
            axs= fig.add_subplot(gs00[0, i])
            axs.set_xlim(0, 2.5)
            #ax = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], [0, 0],xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=[i,  16], linewidth = [1.5]) 
            if len(XtrackRunToRightBin[animal, session][i]):
                time, traj = compute_median_trajectory(XtrackRunToRightBin[animal, session][i], trackTimeToRightBin[animal, session][i])
                ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
                # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
                # plt.plot(oldtime, oldtraj, lw=1.5, c='r')
    for session in matchsession(animal, dist60): # for session in [matchsession(animal, dist120)[0]]:
        for i in range(0, 12):
            axs= fig.add_subplot(gs00[0, i])
            axs.set_xlim(0, 2.5)
            #ax = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], [0, 0],xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=[i,  16], linewidth = [1.5]) 
            if len(XtrackRunToRightBin[animal, session][i]):
                time, traj = compute_median_trajectory(XtrackRunToRightBin[animal, session][i], trackTimeToRightBin[animal, session][i])
                ax = plt.plot(time, traj, color= 'r', lw=1, alpha=1) 
                # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
                # plt.plot(oldtime, oldtraj, lw=1.5, c='r')
plt.show()

In [ ]:

fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for session in matchsession(animal, dist120):
        for i in range(0, 12):
            axs= fig.add_subplot(gs00[0, i])
            axs.set_xlim(0, 2.5)
            axs.set_ylim(0, 130)
            #ax = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], [0, 0],xylim = [-0.1, 2.5, 0, 130], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
            if len(XtrackRunToRightBin[animal, session][i]):
                time, traj = compute_median_trajectory(instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i])
                ax = plt.plot(time, traj, color= 'c', lw=1, alpha=1) 
                # oldtime, oldtraj, ci = compute_median_trajectoryOLD(instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], .30)
                # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

    for session in matchsession(animal, dist90):
        for i in range(0, 12):
            axs= fig.add_subplot(gs00[0, i])
            axs.set_xlim(0, 2.5)
            axs.set_ylim(0, 130)
            #ax = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], [0, 0],xylim = [-0.1, 2.5, 0, 130], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
            if len(XtrackRunToRightBin[animal, session][i]):
                time, traj = compute_median_trajectory(instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i])
                ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
                # oldtime, oldtraj, ci = compute_median_trajectoryOLD(instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], .30)
                # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

    for session in matchsession(animal, dist60):
        for i in range(0, 12):
            axs= fig.add_subplot(gs00[0, i])
            axs.set_xlim(0, 2.5)
            axs.set_ylim(0, 130)
            #ax = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], [0, 0],xylim = [-0.1, 2.5, 0, 130], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
            if len(XtrackRunToRightBin[animal, session][i]):
                time, traj = compute_median_trajectory(instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i])
                ax = plt.plot(time, traj, color= 'r', lw=1, alpha=1) 
                # oldtime, oldtraj, ci = compute_median_trajectoryOLD(instantSpeedRightBin[animal, session][i], trackTimeToRightBin[animal, session][i], .30)
                # plt.plot(oldtime, oldtraj, lw=1.5, c='r')
            
                
#plt.show()

In [ ]:

#
# median of all rightward runs, for each rat (each row), each block ( each column), and 3 Dist conditions (1 line per condition)
#

def plot_median_trajectory2(posdataRight, timedataRight, animal, alpha, color):
    median_plot = plt.gca()
    try : maxL = np.nanmax([len(j) for j in timedataRight])
    except TypeError: maxL = 1
    except ValueError: maxL = 1
    avgPos = np.ones((len(timedataRight), maxL))*np.nan
    for i, p in enumerate(posdataRight): avgPos[i, 0:len(p)] = p
    nanSum = np.sum(np.isnan(avgPos), axis = 0)
    try: end_medTraj = np.where(nanSum > .3 * avgPos.shape[0])[0][0]
    except IndexError: end_medTraj = avgPos.shape[1]
    ci = (np.nanstd(avgPos, axis = 0)[:end_medTraj])/(np.sqrt(np.count_nonzero(~np.isnan(avgPos), axis = 0)[:end_medTraj]))
    plt.plot(np.linspace(0, end_medTraj*0.04, end_medTraj), np.nanmedian(avgPos, axis = 0)[:end_medTraj], color =color, alpha = alpha)#color = 'g' if block == 90 else 'r')
    plt.fill_between(np.linspace(0, end_medTraj*0.04, end_medTraj), np.nanmedian(avgPos, axis = 0)[:end_medTraj]-ci, np.nanmedian(avgPos, axis = 0)[:end_medTraj]+ci, color =color, zorder = 3, alpha = alpha/4) #,'g' if block == 90 else 'r'
    return median_plot


median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])


fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2)
        axs = plot_median_trajectory2(median60[animal][i], tmedian60[animal][i], animal, 1, rat_markers[animal][0])    
        axs = plot_median_trajectory2(median90[animal][i], tmedian90[animal][i], animal, 1, rat_markers[animal][0])
        axs = plot_median_trajectory2(median120[animal][i], tmedian120[animal][i], animal, 1, rat_markers[animal][0])    


        time60, traj60 = compute_median_trajectory(median60[animal][i], tmedian60[animal][i])
        axs = plt.plot(time60, traj60, color= 'k', lw=1, alpha=1) 
        time90, traj90 = compute_median_trajectory(median90[animal][i], tmedian90[animal][i])
        axs = plt.plot(time90, traj90, color= 'k', lw=1, alpha=1) 
        time120, traj120 = compute_median_trajectory(median120[animal][i], tmedian120[animal][i])
        axs = plt.plot(time120, traj120, color= 'k', lw=1, alpha=1) 


plt.show()

In [ ]:

median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])


fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2)
        axs.set_ylim(0, 150)
        # axs = plot_median_trajectory2(median60[animal][i], tmedian60[animal][i], animal, 1, rat_markers[animal][0])    
        # axs = plot_median_trajectory2(median90[animal][i], tmedian90[animal][i], animal, 1, rat_markers[animal][0])
        # axs = plot_median_trajectory2(median120[animal][i], tmedian120[animal][i], animal, 1, rat_markers[animal][0])    


        for p, t in zip(median120[animal][i], tmedian120[animal][i]): axs = plt.plot(t-t[0], p, color = 'g', lw=0.2)
        for p, t in zip(median90[animal][i], tmedian90[animal][i]): axs = plt.plot(t-t[0], p, color = 'c', lw=0.2)
        for p, t in zip(median60[animal][i], tmedian60[animal][i]): axs = plt.plot(t-t[0], p, color = 'r', lw=0.2)

        time60, traj60 = compute_median_trajectory(median60[animal][i], tmedian60[animal][i])
        axs = plt.plot(time60, traj60, color= 'k', lw=2, alpha=1) 

        time90, traj90 = compute_median_trajectory(median90[animal][i], tmedian90[animal][i])
        axs = plt.plot(time90, traj90, color= 'k', lw=2, alpha=1) 

        time120, traj120 = compute_median_trajectory(median120[animal][i], tmedian120[animal][i])
        axs = plt.plot(time120, traj120, color= 'k', lw=2, alpha=1) 

    
        
        


plt.show()



In [ ]:

median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])

med60, med90, med120 = {}, {}, {}
tmed60, tmed90, tmed120 = {}, {}, {}
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    med60[animal], med90[animal], med120[animal], tmed60[animal], tmed90[animal], tmed120[animal] = ([]for i in range(6))
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        med60[animal] = np.append(med60[animal], median60[animal][i])
        tmed60[animal] = np.append(tmed60[animal], tmedian60[animal][i])
        med90[animal] = np.append(med90[animal], median90[animal][i])
        tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
        med120[animal] = np.append(med120[animal], median120[animal][i])
        tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])


    axs= fig.add_subplot(gs00[0, 1])
    axs.set_xlim(0, 2)
    axs.set_ylim(0, 130)
    # for i, j in zip(med120[animal], tmed120[animal]):
    #     axs=plt.plot([_-j[0] for _ in j], i, 'r', lw=0.2, alpha=0.1)
    # for i, j in zip(med90[animal], tmed90[animal]):
    #     axs=plt.plot([_-j[0] for _ in j], i, 'c', lw=0.2, alpha=0.1)
    # for i, j in zip(med60[animal], tmed60[animal]):
    #     axs=plt.plot([_-j[0] for _ in j], i, 'g', lw=0.2, alpha=0.1)




    axs = plot_median_trajectory2(med60[animal], tmed60[animal], animal, 1, 'r')    
    axs = plot_median_trajectory2(med90[animal], tmed90[animal], animal, 1, 'r')
    axs = plot_median_trajectory2(med120[animal], tmed120[animal], animal, 1, 'r')    


    time60, traj60 = compute_median_trajectory(med60[animal], tmed60[animal])
    axs = plt.plot(time60, traj60, color= rat_markers[animal][0], lw=2, alpha=1) 
    #print(animal, time60, traj60)


    time90, traj90 = compute_median_trajectory(med90[animal], tmed90[animal])
    axs = plt.plot(time90, traj90, color= rat_markers[animal][0], lw=2, alpha=1) 
    # print(time90, traj90)


    time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
    axs = plt.plot(time120, traj120, color= rat_markers[animal][0], lw=2, alpha=1) 
    # print(time120, traj120)
    
plt.show()
#print(med60[animal],med90[animal], med120[animal])


In [ ]:
median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])

med60, med90, med120 = {}, {}, {}
tmed60, tmed90, tmed120 = {}, {}, {}
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    med60[animal], med90[animal], med120[animal], tmed60[animal], tmed90[animal], tmed120[animal] = ([]for i in range(6))

    for i in range(0, 12):
        med60[animal] = np.append(med60[animal], median60[animal][i])
        tmed60[animal] = np.append(tmed60[animal], tmedian60[animal][i])
        med90[animal] = np.append(med90[animal], median90[animal][i])
        tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
        med120[animal] = np.append(med120[animal], median120[animal][i])
        tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])


    axs= fig.add_subplot(111)
    axs.set_xlim(0, 2)
    axs.set_ylim(0, 130)
    # for i, j in zip(med120[animal], tmed120[animal]):
    #     axs=plt.plot([_-j[0] for _ in j], i, 'r', lw=0.2, alpha=0.1)
    # for i, j in zip(med90[animal], tmed90[animal]):
    #     axs=plt.plot([_-j[0] for _ in j], i, 'c', lw=0.2, alpha=0.1)
    # for i, j in zip(med60[animal], tmed60[animal]):
    #     axs=plt.plot([_-j[0] for _ in j], i, 'g', lw=0.2, alpha=0.1)


    time60, traj60 = compute_median_trajectory(med60[animal], tmed60[animal])
    axs = plt.plot(time60, traj60, color= rat_markers[animal][0], lw=2, alpha=1) 
    #print(animal, time60, traj60)


    time90, traj90 = compute_median_trajectory(med90[animal], tmed90[animal])
    axs = plt.plot(time90, traj90, color= rat_markers[animal][0], lw=2, alpha=1) 
    # print(time90, traj90)


    time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
    axs = plt.plot(time120, traj120, color= rat_markers[animal][0], lw=2, alpha=1) 
    # print(time120, traj120)
    
plt.show()
#print(med60[animal],med90[animal], med120[animal])

In [ ]:
# same with TM_ON experiment

In [ ]:
medianrev20, medianrev10, medianrev2, median2, median10, median20, tmedianrev20, tmedianrev10, tmedianrev2, tmedian2, tmedian10, tmedian20 = ({} for i in range(12))
for animal in animalList:
    medianrev20[animal], medianrev10[animal], medianrev2[animal], median2[animal], median10[animal], median20[animal], tmedianrev20[animal], tmedianrev10[animal], tmedianrev2[animal], tmedian2[animal], tmedian10[animal], tmedian20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(12))
    
    for session in sorted(matchsession(animal, TMrev20)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                medianrev20[animal][i] = np.append(medianrev20[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev20[animal][i] = np.append(tmedianrev20[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TMrev10)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                medianrev10[animal][i] = np.append(medianrev10[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev10[animal][i] = np.append(tmedianrev10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TMrev2)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                medianrev2[animal][i]= np.append(medianrev2[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev2[animal][i]= np.append(tmedianrev2[animal][i], trackTimeToRightBin[animal, session][i])

    for session in sorted(matchsession(animal, TM2)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median2[animal][i] = np.append(median2[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian2[animal][i] = np.append(tmedian2[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TM10)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median10[animal][i] = np.append(median10[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian10[animal][i] = np.append(tmedian10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TM20)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 5:
                median20[animal][i]= np.append(median20[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian20[animal][i]= np.append(tmedian20[animal][i], trackTimeToRightBin[animal, session][i])

fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2.5)

        # for k, l in zip(datarevt20[animal][i], datarev20[animal][i]): plt.plot(k-k[0],l, c='darkred',    lw=0.1)
        # for k, l in zip(datarevt10[animal][i], datarev10[animal][i]): plt.plot(k-k[0],l, c='darkorange', lw=0.1)
        # for k, l in zip(datarevt2[animal][i],  datarev2[animal][i]):  plt.plot(k-k[0],l, c='bisque',     lw=0.1)
        # for k, l in zip(datat2[animal][i],     data2[animal][i]):     plt.plot(k-k[0],l, c='greenyellow',lw=0.1)
        # for k, l in zip(datat10[animal][i],    data10[animal][i]):    plt.plot(k-k[0],l, c='lightgreen', lw=0.1)
        # for k, l in zip(datat20[animal][i],    data20[animal][i]):    plt.plot(k-k[0],l, c='darkgreen',  lw=0.1)

        axs = plot_median_trajectory2(medianrev20[animal][i], tmedianrev20[animal][i], animal, 1, 'darkred')    
        axs = plot_median_trajectory2(medianrev10[animal][i], tmedianrev10[animal][i], animal, 1, 'darkorange')
        axs = plot_median_trajectory2(medianrev2[animal][i], tmedianrev2[animal][i], animal, 1, 'bisque')
        axs = plot_median_trajectory2(median2[animal][i], tmedian2[animal][i], animal, 1, 'greenyellow')    
        axs = plot_median_trajectory2(median10[animal][i], tmedian10[animal][i], animal, 1, 'lightgreen')
        axs = plot_median_trajectory2(median20[animal][i], tmedian20[animal][i], animal, 1, 'darkgreen')

        try:    
            time, traj = compute_median_trajectory(datarev20[animal][i], datarevt20[animal][i])
            ax = plt.plot(time, traj, color= 'r', lw=3, alpha=1) 
            time, traj = compute_median_trajectory(datarev10[animal][i], datarevt10[animal][i])
            ax = plt.plot(time, traj, color= 'r', lw=1.5, alpha=1) 
            time, traj = compute_median_trajectory(datarev2[animal][i], datarevt2[animal][i])
            ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
            time, traj = compute_median_trajectory(data2[animal][i], datat2[animal][i])
            ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
            time, traj = compute_median_trajectory(data10[animal][i], datat10[animal][i])
            ax = plt.plot(time, traj, color= 'g', lw=1.5, alpha=1) 
            time, traj = compute_median_trajectory(data20[animal][i], datat20[animal][i])
            ax = plt.plot(time, traj, color= 'g', lw=3, alpha=1) 
        except: TypeError
        # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
        # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:
medianrev20, medianrev10, medianrev2, median2, median10, median20, tmedianrev20, tmedianrev10, tmedianrev2, tmedian2, tmedian10, tmedian20 = ({} for i in range(12))
for animal in animalList:
    medianrev20[animal], medianrev10[animal], medianrev2[animal], median2[animal], median10[animal], median20[animal], tmedianrev20[animal], tmedianrev10[animal], tmedianrev2[animal], tmedian2[animal], tmedian10[animal], tmedian20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(12))
    
    for session in sorted(matchsession(animal, TMrev20)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                medianrev20[animal][i] = np.append(medianrev20[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev20[animal][i] = np.append(tmedianrev20[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TMrev10)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                medianrev10[animal][i] = np.append(medianrev10[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev10[animal][i] = np.append(tmedianrev10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TMrev2)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                medianrev2[animal][i]= np.append(medianrev2[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev2[animal][i]= np.append(tmedianrev2[animal][i], trackTimeToRightBin[animal, session][i])

    for session in sorted(matchsession(animal, TM2)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median2[animal][i] = np.append(median2[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian2[animal][i] = np.append(tmedian2[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TM10)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median10[animal][i] = np.append(median10[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian10[animal][i] = np.append(tmedian10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TM20)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 5:
                median20[animal][i]= np.append(median20[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian20[animal][i]= np.append(tmedian20[animal][i], trackTimeToRightBin[animal, session][i])

fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2)
        axs = plot_median_trajectory2(medianrev20[animal][i], tmedianrev20[animal][i], animal, 1, 'darkred')    
        axs = plot_median_trajectory2(medianrev10[animal][i], tmedianrev10[animal][i], animal, 1, 'darkorange')
        axs = plot_median_trajectory2(medianrev2[animal][i], tmedianrev2[animal][i], animal, 1, 'bisque')
        axs = plot_median_trajectory2(median2[animal][i], tmedian2[animal][i], animal, 1, 'greenyellow')    
        axs = plot_median_trajectory2(median10[animal][i], tmedian10[animal][i], animal, 1, 'lightgreen')
        axs = plot_median_trajectory2(median20[animal][i], tmedian20[animal][i], animal, 1, 'darkgreen')
plt.show()


In [ ]:
medianrev20, medianrev10, medianrev2, median2, median10, median20, tmedianrev20, tmedianrev10, tmedianrev2, tmedian2, tmedian10, tmedian20 = ({} for i in range(12))


for animal in animalList:
    medianrev20[animal], medianrev10[animal], medianrev2[animal], median2[animal], median10[animal], median20[animal], tmedianrev20[animal], tmedianrev10[animal], tmedianrev2[animal], tmedian2[animal], tmedian10[animal], tmedian20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(12))
    
    for session in sorted(matchsession(animal, TMrev20)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev20[animal][i] = np.append(medianrev20[animal][i], instantSpeedRightBin[animal, session][i])
                tmedianrev20[animal][i] = np.append(tmedianrev20[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TMrev10)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev10[animal][i] = np.append(medianrev10[animal][i], instantSpeedRightBin[animal, session][i])
                tmedianrev10[animal][i] = np.append(tmedianrev10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TMrev2)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev2[animal][i]= np.append(medianrev2[animal][i], instantSpeedRightBin[animal, session][i])
                tmedianrev2[animal][i]= np.append(tmedianrev2[animal][i], trackTimeToRightBin[animal, session][i])

    for session in sorted(matchsession(animal, TM2)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median2[animal][i] = np.append(median2[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian2[animal][i] = np.append(tmedian2[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TM10)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median10[animal][i] = np.append(median10[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian10[animal][i] = np.append(tmedian10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TM20)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 5:
                median20[animal][i]= np.append(median20[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian20[animal][i]= np.append(tmedian20[animal][i], trackTimeToRightBin[animal, session][i])


fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2.5)

        # for k, l in zip(datarevt20[animal][i], datarev20[animal][i]): plt.plot(k-k[0],l, c='darkred',    lw=0.1)
        # for k, l in zip(datarevt10[animal][i], datarev10[animal][i]): plt.plot(k-k[0],l, c='darkorange', lw=0.1)
        # for k, l in zip(datarevt2[animal][i],  datarev2[animal][i]):  plt.plot(k-k[0],l, c='bisque',     lw=0.1)
        # for k, l in zip(datat2[animal][i],     data2[animal][i]):     plt.plot(k-k[0],l, c='greenyellow',lw=0.1)
        # for k, l in zip(datat10[animal][i],    data10[animal][i]):    plt.plot(k-k[0],l, c='lightgreen', lw=0.1)
        # for k, l in zip(datat20[animal][i],    data20[animal][i]):    plt.plot(k-k[0],l, c='darkgreen',  lw=0.1)

        axs = plot_median_trajectory2(medianrev20[animal][i], tmedianrev20[animal][i], animal, 1, 'darkred')    
        axs = plot_median_trajectory2(medianrev10[animal][i], tmedianrev10[animal][i], animal, 1, 'darkorange')
        axs = plot_median_trajectory2(medianrev2[animal][i], tmedianrev2[animal][i], animal, 1, 'bisque')
        axs = plot_median_trajectory2(median2[animal][i], tmedian2[animal][i], animal, 1, 'greenyellow')    
        axs = plot_median_trajectory2(median10[animal][i], tmedian10[animal][i], animal, 1, 'lightgreen')
        axs = plot_median_trajectory2(median20[animal][i], tmedian20[animal][i], animal, 1, 'darkgreen')

        try:    
            time, traj = compute_median_trajectory(sdatarev20[animal][i], sdatarevt20[animal][i])
            ax = plt.plot(time, traj, color= 'r', lw=3, alpha=1) 
            time, traj = compute_median_trajectory(sdatarev10[animal][i], sdatarevt10[animal][i])
            ax = plt.plot(time, traj, color= 'r', lw=1.5, alpha=1) 
            time, traj = compute_median_trajectory(sdatarev2[animal][i], sdatarevt2[animal][i])
            ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
            time, traj = compute_median_trajectory(sdata2[animal][i], sdatat2[animal][i])
            ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
            time, traj = compute_median_trajectory(sdata10[animal][i], sdatat10[animal][i])
            ax = plt.plot(time, traj, color= 'g', lw=1.5, alpha=1) 
            time, traj = compute_median_trajectory(sdata20[animal][i], sdatat20[animal][i])
            ax = plt.plot(time, traj, color= 'g', lw=3, alpha=1) 
        except: TypeError
        # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
        # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:
medianrev20, medianrev10, medianrev2, median2, median10, median20, tmedianrev20, tmedianrev10, tmedianrev2, tmedian2, tmedian10, tmedian20 = ({} for i in range(12))


for animal in animalList:
    medianrev20[animal], medianrev10[animal], medianrev2[animal], median2[animal], median10[animal], median20[animal], tmedianrev20[animal], tmedianrev10[animal], tmedianrev2[animal], tmedian2[animal], tmedian10[animal], tmedian20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(12))
    
    for session in sorted(matchsession(animal, TMrev20)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev20[animal][i] = np.append(medianrev20[animal][i], instantSpeedRightBin[animal, session][i])
                tmedianrev20[animal][i] = np.append(tmedianrev20[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TMrev10)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev10[animal][i] = np.append(medianrev10[animal][i], instantSpeedRightBin[animal, session][i])
                tmedianrev10[animal][i] = np.append(tmedianrev10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TMrev2)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev2[animal][i]= np.append(medianrev2[animal][i], instantSpeedRightBin[animal, session][i])
                tmedianrev2[animal][i]= np.append(tmedianrev2[animal][i], trackTimeToRightBin[animal, session][i])

    for session in sorted(matchsession(animal, TM2)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median2[animal][i] = np.append(median2[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian2[animal][i] = np.append(tmedian2[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TM10)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median10[animal][i] = np.append(median10[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian10[animal][i] = np.append(tmedian10[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, TM20)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 5:
                median20[animal][i]= np.append(median20[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian20[animal][i]= np.append(tmedian20[animal][i], trackTimeToRightBin[animal, session][i])

fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for i in range(0, 12):
        axs= fig.add_subplot(gs00[0, i])
        axs.set_xlim(0, 2)
        axs = plot_median_trajectory2(medianrev20[animal][i], tmedianrev20[animal][i], animal, 1, 'darkred')    
        axs = plot_median_trajectory2(medianrev10[animal][i], tmedianrev10[animal][i], animal, 1, 'darkorange')
        axs = plot_median_trajectory2(medianrev2[animal][i], tmedianrev2[animal][i], animal, 1, 'bisque')
        axs = plot_median_trajectory2(median2[animal][i], tmedian2[animal][i], animal, 1, 'greenyellow')    
        axs = plot_median_trajectory2(median10[animal][i], tmedian10[animal][i], animal, 1, 'lightgreen')
        axs = plot_median_trajectory2(median20[animal][i], tmedian20[animal][i], animal, 1, 'darkgreen')
        axs.set_ylim(0, 150)
plt.show()


In [ ]:

animal = 'RatM02'
i=9
time, traj = compute_median_trajectory(sdatarev20[animal][i], sdatarevt20[animal][i])
ax = plt.plot(time, traj, color= 'r', lw=3, alpha=1) 
time, traj = compute_median_trajectory(sdatarev10[animal][i], sdatarevt10[animal][i])
ax = plt.plot(time, traj, color= 'r', lw=1.5, alpha=1) 
time, traj = compute_median_trajectory(sdatarev2[animal][i], sdatarevt2[animal][i])
ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
time, traj = compute_median_trajectory(sdata2[animal][i], sdatat2[animal][i])
ax = plt.plot(time, traj, color= 'k', lw=1, alpha=1) 
time, traj = compute_median_trajectory(sdata10[animal][i][300:], sdatat10[animal][i][300:])
ax = plt.plot(time, traj, color= 'g', lw=1.5, alpha=1) 
time, traj = compute_median_trajectory(sdata20[animal][i], sdatat20[animal][i])
ax = plt.plot(time, traj, color= 'g', lw=3, alpha=1) 

        # # oldtime, oldtraj, ci = compute_median_trajectoryOLD(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], .30)
        # # plt.plot(oldtime, oldtraj, lw=1.5, c='r')

In [ ]:
sdata10[animal][i][220:]

In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(40, 30))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(19, 37) #row ,col

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
col1 = slice(0, 5)
col2 = slice(6, 11)
col3 = slice(12, 17)

col4 = slice(20, 25)
col5 = slice(26, 31)
col6 = slice(32, 37)


animal = 'RatM00'
sessionID = 0
session = matchsession(animal, dist60)[sessionID]
experiment = 'Distance'
block0=0
blockN=1

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], [0, 0], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","", 14], title=["Pos60", 12], linewidth = [0.5])
ax00 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], animal, 1, 'k')
ax10 = fig.add_subplot(gs[row2, col1])
ax10 = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], [0, 0], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","", 14], title=["Speed60", 12], linewidth = [0.5])
ax10 = plot_median_trajectory2(instantSpeedRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], animal, 1, 'k')
ax20 = fig.add_subplot(gs[row3, col1])
ax20 = plot_trackstest(animal, session,[np.diff(r) for r in instantSpeedRightBin[animal, session][block0]], [t[:-1] for t in trackTimeToRightBin[animal, session][block0]], [0, 0], xylim = [-0.1, 2.5, -15, 15], xyLabels=["Time (s)","", 14], title=["Acc60", 12], linewidth = [0.5])
ax20 = plot_median_trajectory2([np.diff(r) for r in instantSpeedRightBin[animal, session][block0]], [t[:-1] for t in trackTimeToRightBin[animal, session][block0]], animal, 1, 'k')

ax30 = fig.add_subplot(gs[row1, col4])
ax30 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], [0, 0], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","", 14], title=["Pos60", 12], linewidth = [0.5])
ax30 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], animal, 1, 'k')
ax40 = fig.add_subplot(gs[row2, col4])
ax40 = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], [0, 0], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","", 14], title=["Speed60", 12], linewidth = [0.5])
ax40 = plot_median_trajectory2(instantSpeedRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], animal, 1, 'k')
ax50 = fig.add_subplot(gs[row3, col4])
ax50 = plot_trackstest(animal, session,[np.diff(r) for r in instantSpeedRightBin[animal, session][blockN]], [t[:-1] for t in trackTimeToRightBin[animal, session][blockN]], [0, 0], xylim = [-0.1, 2.5, -15, 15], xyLabels=["Time (s)","", 14], title=["Acc60", 12], linewidth = [0.5])
ax50 = plot_median_trajectory2([np.diff(r) for r in instantSpeedRightBin[animal, session][blockN]], [t[:-1] for t in trackTimeToRightBin[animal, session][blockN]], animal, 1, 'k')


session = matchsession(animal, dist90)[sessionID]
experiment = 'Distance'
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], [0, 0], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","", 14], title=["Pos90", 12], linewidth = [0.5])
ax01 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], animal, 1, 'k')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], [0, 0], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","", 14], title=["Speed90" , 12], linewidth = [0.5])
ax11 = plot_median_trajectory2(instantSpeedRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], animal, 1, 'k')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = plot_trackstest(animal, session,[np.diff(r) for r in instantSpeedRightBin[animal, session][block0]], [t[:-1] for t in trackTimeToRightBin[animal, session][block0]], [0, 0], xylim = [-0.1, 2.5, -15, 15], xyLabels=["Time (s)","", 14], title=["Acc90", 12], linewidth = [0.5])
ax22 = plot_median_trajectory2([np.diff(r) for r in instantSpeedRightBin[animal, session][block0]], [t[:-1] for t in trackTimeToRightBin[animal, session][block0]], animal, 1, 'k')


ax31 = fig.add_subplot(gs[row1, col5])
ax31 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], [0, 0], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","", 14], title=["Pos90", 12], linewidth = [0.5])
ax31 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], animal, 1, 'k')
ax41 = fig.add_subplot(gs[row2, col5])
ax41 = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], [0, 0], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","", 14], title=["Speed90" , 12], linewidth = [0.5])
ax41 = plot_median_trajectory2(instantSpeedRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], animal, 1, 'k')
ax51 = fig.add_subplot(gs[row3, col5])
ax51 = plot_trackstest(animal, session,[np.diff(r) for r in instantSpeedRightBin[animal, session][blockN]], [t[:-1] for t in trackTimeToRightBin[animal, session][blockN]], [0, 0], xylim = [-0.1, 2.5, -15, 15], xyLabels=["Time (s)","", 14], title=["Acc90", 12], linewidth = [0.5])
ax52 = plot_median_trajectory2([np.diff(r) for r in instantSpeedRightBin[animal, session][blockN]], [t[:-1] for t in trackTimeToRightBin[animal, session][blockN]], animal, 1, 'k')


session = matchsession(animal, dist120)[sessionID]
experiment = 'Distance'
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], [0, 0], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","", 14], title=["Pos120", 12], linewidth = [0.5])
ax02 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], animal, 1, 'k')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], [0, 0], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","", 14], title=["Speed120", 12], linewidth = [0.5])
ax12 = plot_median_trajectory2(instantSpeedRightBin[animal, session][block0], trackTimeToRightBin[animal, session][block0], animal, 1, 'k')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = plot_trackstest(animal, session,[np.diff(r) for r in instantSpeedRightBin[animal, session][block0]], [t[:-1] for t in trackTimeToRightBin[animal, session][block0]], [0, 0], xylim = [-0.1, 2.5, -15, 15], xyLabels=["Time (s)","", 14], title=["Acc120", 12], linewidth = [0.5])
ax22 = plot_median_trajectory2([np.diff(r) for r in instantSpeedRightBin[animal, session][block0]], [t[:-1] for t in trackTimeToRightBin[animal, session][block0]], animal, 1, 'k')

ax32 = fig.add_subplot(gs[row1, col6])
ax32 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], [0, 0], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","", 14], title=["Pos120", 12], linewidth = [0.5])
ax32 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], animal, 1, 'k')
ax42 = fig.add_subplot(gs[row2, col6])
ax42 = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], [0, 0], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","", 14], title=["Speed120", 12], linewidth = [0.5])
ax42 = plot_median_trajectory2(instantSpeedRightBin[animal, session][blockN], trackTimeToRightBin[animal, session][blockN], animal, 1, 'k')
ax52 = fig.add_subplot(gs[row3, col6])
ax52 = plot_trackstest(animal, session,[np.diff(r) for r in instantSpeedRightBin[animal, session][blockN]], [t[:-1] for t in trackTimeToRightBin[animal, session][blockN]], [0, 0], xylim = [-0.1, 2.5, -15, 15], xyLabels=["Time (s)","", 14], title=["Acc120", 12], linewidth = [0.5])
ax52 = plot_median_trajectory2([np.diff(r) for r in instantSpeedRightBin[animal, session][blockN]], [t[:-1] for t in trackTimeToRightBin[animal, session][blockN]], animal, 1, 'k')


In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    sessionList = sorted(matchsession(animal, dist60+dist90+dist120))
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for sindex, session in enumerate(sessionList):
        for m in range(0, 12):
            axs= fig.add_subplot(gs00[0, m]) 
            axs.set_ylim(0, 150)
            #axs = plot_trackstest(animal, session, instantSpeedRightBin[animal, session][m], trackTimeToRightBin[animal, session][m], [0, 0], xylim = [-0.1, 2.5, 0, 200], xyLabels=["Time (s)","", 14], title=["Speed" + "\n" + " To left and to right", 12], linewidth = [0.5])
            axs = plot_median_trajectory2(instantSpeedRightBin[animal, session][m], trackTimeToRightBin[animal, session][m], animal, 1, 'k')        
plt.show()

In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    sessionList = sorted(matchsession(animal, dist60+dist90+dist120))
    gs00 = gs[index, 0:12].subgridspec(1,12)
    for sindex, session in enumerate(sessionList):
        for m in range(0, 12):
            axs= fig.add_subplot(gs00[0, m]) 
            #axs = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][m], trackTimeToRightBin[animal, session][m], [0, 0], xylim = [-0.1, 2.5, 0, 200], xyLabels=["Time (s)","", 14], title=["Speed" + "\n" + " To left and to right", 12], linewidth = [0.5])
            axs = plot_median_trajectory2(XtrackRunToRightBin[animal, session][m], trackTimeToRightBin[animal, session][m], animal, 1, 'k')        
plt.show()

In [ ]:
# TEST true distance
realdist60 = {}
realdist90 = {}
realdist120= {}

for animal in animalList:
    realdist60[animal], realdist90[animal], realdist120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(3))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            realdist60[animal][i] = np.append(realdist60[animal][i], [abs(r[0] - r[-1]) for r in XtrackRunToRightBin[animal, session][i]])
            realdist60[animal][i] = np.append(realdist60[animal][i], [abs(l[0] - l[-1]) for l in XtrackRunToLeftBin[animal, session][i]])
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            realdist90[animal][i] = np.append(realdist90[animal][i], [abs(r[0] - r[-1]) for r in XtrackRunToRightBin[animal, session][i]])
            realdist90[animal][i] = np.append(realdist90[animal][i], [abs(l[0] - l[-1]) for l in XtrackRunToLeftBin[animal, session][i]])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            realdist120[animal][i] = np.append(realdist120[animal][i], [abs(r[0] - r[-1]) for r in XtrackRunToRightBin[animal, session][i]])
            realdist120[animal][i] = np.append(realdist120[animal][i], [abs(l[0] - l[-1]) for l in XtrackRunToLeftBin[animal, session][i]])

mean60 = []
mean90 = []
mean120= []

for animal in animalList:
    plt.scatter(np.mean([item for sublist in list(realdist60[animal].values()) for item in sublist]), 1, color =rat_markers[animal][0])
    plt.scatter(np.mean([item for sublist in list(realdist90[animal].values()) for item in sublist]), 1, color =rat_markers[animal][0])
    plt.scatter(np.mean([item for sublist in list(realdist120[animal].values()) for item in sublist]), 1, color =rat_markers[animal][0])
    mean60 = np.append(mean60, np.mean([item for sublist in list(realdist60[animal].values()) for item in sublist]))
    mean90 = np.append(mean90, np.mean([item for sublist in list(realdist90[animal].values()) for item in sublist]))
    mean120 = np.append(mean120, np.mean([item for sublist in list(realdist120[animal].values()) for item in sublist]))
plt.scatter(np.mean(mean60), 1.1)
plt.scatter(np.mean(mean90), 1.1)
plt.scatter(np.mean(mean120), 1.1)
print(np.mean(mean60), np.mean(mean90), np.mean(mean120))

#ax.plot((ticks[0][0], ticks[0][1], ticks[0][2]), (np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
               


In [ ]:
#plot explanation

fig = plt.figure(constrained_layout = False, figsize=(30.5, 13)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(29, 71)


row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)


col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)
col4 = slice(25, 30)
col5 = slice(32, 37)
col6 = slice(40, 45)
col7 = slice (50, 55)
col8 = slice (58, 63)
col9 = slice (66, 71)

session_list = ['RatF02_2021_07_22_18_14_16']
experiment = 'Distance'
animalList1 = ['RatF02']

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = across_session_plot("90%", animalList1, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', 'avg. # runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = across_session_plot("10%", animalList1, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = across_session_plot("%",   animalList1, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10 % reward\n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

session_list = dist60+dist90+dist120
experiment = 'Distance'
animalList1 = ['RatF02']

ax00 = fig.add_subplot(gs[row2, col1])
ax00 = across_session_plot("90%", animalList1, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', 'avg. # runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row2, col2])
ax01 = across_session_plot("10%", animalList1, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row2, col3])
ax02 = across_session_plot("%",   animalList1, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10 % reward\n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

session_list = dist60+dist90+dist120
experiment = 'Distance'
animalList1 = ['RatF02']

ax00 = fig.add_subplot(gs[row3, col1])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', 'avg. # runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row3, col2])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row3, col3])
ax02 = across_session_plot("%",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10 % reward\n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')




#save_plot_as_png('dualplot2', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:

aaa= 'RatF02'
sss= 'RatF02_2021_07_22_18_14_16'

fig = plt.figure(constrained_layout = False, figsize=(23, 5))
gs = fig.add_gridspec(5, 23)
ax70 = fig.add_subplot(gs[0:5, 0:5])
ax70 = plot_figBin([nb_runsBin[aaa, sss][i]/5 for i in range(0, 12)], params[aaa, sss]['rewardProbaBlock'], params[aaa, sss]['blocks'], barplotaxes = [0, params[aaa, sss]['sessionDuration']/60, 0, 25], color = ['k'], xyLabels=["Time (min)", "\u0023 runs / min", 14, 12], title=["", 16], linewidth = [1.5]);
ax72 = fig.add_subplot(gs[0:5, 6:11])
ax72 = plot_figBin([np.mean(timeStayInLeftBin[aaa, sss][i]+timeStayInRightBin[aaa, sss][i]) for i in range(0, 12)], params[aaa, sss]['rewardProbaBlock'], params[aaa, sss]['blocks'], barplotaxes = [0, params[aaa, sss]['sessionDuration']/60, 0, 25], color = ['orange'], xyLabels=["Time (min)","Avg. time in sides (s)", 14, 12], title=["", 16], linewidth = [1.5]);
ax74 = fig.add_subplot(gs[0:5, 12:17])
ax74 = plot_figBin([np.mean(speedRunToLeftBin[aaa, sss][i]+speedRunToRightBin[aaa, sss][i]) for i in range(0, 12)], params[aaa, sss]['rewardProbaBlock'], params[aaa, sss]['blocks'], barplotaxes = [0, params[aaa, sss]['sessionDuration']/60, 0, 120], color = ['dodgerblue'], xyLabels=["Time (min)","Avg. run speed (cm/s)", 14, 12], title=["", 16], linewidth = [1.5]);
ax76 = fig.add_subplot(gs[0:5, 18:23])
ax76 = plot_figBin([np.mean(timeRunToLeftBin[aaa, sss][i]+timeRunToRightBin[aaa, sss][i]) for i in range(0, 12)], params[aaa, sss]['rewardProbaBlock'], params[aaa, sss]['blocks'], barplotaxes = [0, params[aaa, sss]['sessionDuration']/60, 0, 10], color = ['red'], xyLabels=["Time (min)", "Avg. run time (s)", 14, 12], title=["", 16], linewidth = [1.5]);

save_plot_as_png('blockplots', dpi = 300, transparent = True, background = 'auto')

In [ ]:
aaa= 'RatF02'
sss= 'RatF02_2021_07_22_18_14_16'


median90 = {}
median120= {}
tmedian90 = {}
tmedian120= {}
animal = 'RatF02'
session = 'RatF02_2021_07_22_18_14_16'
median90[animal], median120[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(4))
for i in range(0, 12):
    if len(XtrackRunToRightBin[animal, session][i]) >= 3:
        median120[animal][i]= np.append(median120[animal][i], XtrackRunToRightBin[animal, session][i])
        tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])
        median90[animal][i]= np.append(median90[animal][i], XtrackRunToLeftBin[animal, session][i])
        tmedian90[animal][i]= np.append(tmedian90[animal][i], trackTimeToLeftBin[animal, session][i])
med90, med120 = {}, {}
tmed90, tmed120 = {}, {}
med90[animal], med120[animal], tmed90[animal], tmed120[animal] = ([]for i in range(4))
for i in range(0, 12):
    med120[animal] = np.append(med120[animal], median120[animal][i])
    tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])
    med90[animal] = np.append(med90[animal], median90[animal][i])
    tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
time90, traj90 = compute_median_trajectory(med90[animal], tmed90[animal])


fig = plt.figure(constrained_layout = False, figsize=(12, 12))
gs = fig.add_gridspec(12, 12)
ax20 = fig.add_subplot(gs[0:5, 0:5])
ax20 = plot_trackstest(aaa, sss, med120[animal], tmed120[animal], params[aaa, sss]["boundaries"], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5])
ax20 = plt.plot(time120, traj120, color='k', lw=2, alpha=1) 

ax21 = fig.add_subplot(gs[0:5, 7:12]) 
ax21 = plot_trackstest(aaa, sss, med90[animal], tmed90[animal], params[aaa, sss]["boundaries"], xylim = [-0.1, 2.5, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Left",  16], linewidth = [1.5])
ax21 = plt.plot(time90, traj90, color= 'k', lw=2, alpha=1)  



In [ ]:
aaa= 'RatF02'
sss= 'RatF02_2021_07_22_18_14_16'


median90 = {}
median120= {}
tmedian90 = {}
tmedian120= {}
animal = 'RatF02'
session = 'RatF02_2021_07_22_18_14_16'
median90[animal], median120[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(4))
for i in range(0, 12):
    if len(instantSpeedRightBin[animal, session][i]) >= 3:
        median120[animal][i]= np.append(median120[animal][i], instantSpeedRightBin[animal, session][i])
        tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])
        median90[animal][i]= np.append(median90[animal][i], instantSpeedLeftBin[animal, session][i])
        tmedian90[animal][i]= np.append(tmedian90[animal][i], trackTimeToLeftBin[animal, session][i])
med90, med120 = {}, {}
tmed90, tmed120 = {}, {}
med90[animal], med120[animal], tmed90[animal], tmed120[animal] = ([]for i in range(4))
for i in range(0, 12):
    med120[animal] = np.append(med120[animal], median120[animal][i])
    tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])
    med90[animal] = np.append(med90[animal], median90[animal][i])
    tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
time90, traj90 = compute_median_trajectory(med90[animal], tmed90[animal])


fig = plt.figure(constrained_layout = False, figsize=(12, 12))
gs = fig.add_gridspec(12, 12)
ax20 = fig.add_subplot(gs[0:5, 0:5])
ax20 = plot_trackstest(aaa, sss, med120[animal], tmed120[animal], params[aaa, sss]["boundaries"], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","Speed (cm/s)", 14], title=["Speed run to Right",  16], linewidth = [1.5])
ax20 = plt.plot(time120, traj120, color='k', lw=2, alpha=1) 

ax21 = fig.add_subplot(gs[0:5, 7:12]) 
ax21 = plot_trackstest(aaa, sss, med90[animal], tmed90[animal], params[aaa, sss]["boundaries"], xylim = [-0.1, 2.5, 0, 150], xyLabels=["Time (s)","Speed (cm/s)", 14], title=["Speed run to Left",  16], linewidth = [1.5])
ax21 = plt.plot(time90, traj90, color= 'k', lw=2, alpha=1)  



In [ ]:

median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], instantSpeedRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])

med60, med90, med120 = {}, {}, {}
tmed60, tmed90, tmed120 = {}, {}, {}
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)


animal = 'RatF02'
med60[animal], med90[animal], med120[animal], tmed60[animal], tmed90[animal], tmed120[animal] = ([]for i in range(6))
gs00 = gs[0, 0:12].subgridspec(1,12)
for i in range(0, 12):
    med60[animal] = np.append(med60[animal], median60[animal][i])
    tmed60[animal] = np.append(tmed60[animal], tmedian60[animal][i])
    med90[animal] = np.append(med90[animal], median90[animal][i])
    tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
    med120[animal] = np.append(med120[animal], median120[animal][i])
    tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])


axs= fig.add_subplot(gs00[0, 1])
axs.set_xlim(0, 2)
axs.set_ylim(0, 130)

  

time60, traj60 = compute_median_trajectory(med60[animal], tmed60[animal])
axs = plt.plot(time60, traj60, color= rat_markers[animal][0], lw=2, alpha=1) 
#print(animal, time60, traj60)

time90, traj90 = compute_median_trajectory(med90[animal], tmed90[animal])
axs = plt.plot(time90, traj90, color= rat_markers[animal][0], lw=2, alpha=1) 
# print(time90, traj90)

time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
axs = plt.plot(time120, traj120, color= rat_markers[animal][0], lw=2, alpha=1) 
# print(time120, traj120)

plt.show()
#print(med60[animal],med90[animal], med120[animal])


In [ ]:

median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])

med60, med90, med120 = {}, {}, {}
tmed60, tmed90, tmed120 = {}, {}, {}
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)


animal = 'RatF02'
med60[animal], med90[animal], med120[animal], tmed60[animal], tmed90[animal], tmed120[animal] = ([]for i in range(6))
gs00 = gs[0, 0:12].subgridspec(1,12)
for i in range(0, 12):
    med60[animal] = np.append(med60[animal], median60[animal][i])
    tmed60[animal] = np.append(tmed60[animal], tmedian60[animal][i])
    med90[animal] = np.append(med90[animal], median90[animal][i])
    tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
    med120[animal] = np.append(med120[animal], median120[animal][i])
    tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])


axs= fig.add_subplot(gs00[0, 1])
axs.set_xlim(0, 2)
axs.set_ylim(0, 130)

  

time60, traj60 = compute_median_trajectory(med60[animal], tmed60[animal])
axs = plt.plot(time60, traj60, color= rat_markers[animal][0], lw=2, alpha=1) 
#print(animal, time60, traj60)

time90, traj90 = compute_median_trajectory(med90[animal], tmed90[animal])
axs = plt.plot(time90, traj90, color= rat_markers[animal][0], lw=2, alpha=1) 
# print(time90, traj90)

time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
axs = plt.plot(time120, traj120, color= rat_markers[animal][0], lw=2, alpha=1) 
# print(time120, traj120)

plt.show()
#print(med60[animal],med90[animal], med120[animal])


In [ ]:

median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}
animal = 'RatF02'

median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
session = 'RatF02_2021_07_22_18_14_16'
for i in range(0, 12):
    if len(XtrackRunToRightBin[animal, session][i]) >= 3:
        median120[animal][i]= np.append(median120[animal][i], XtrackRunToRightBin[animal, session][i])
        tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])

med60, med90, med120 = {}, {}, {}
tmed60, tmed90, tmed120 = {}, {}, {}
fig = plt.figure(constrained_layout = False, figsize=(30, 20))
gs = fig.add_gridspec(len(animalList), 12)


animal = 'RatF02'
med60[animal], med90[animal], med120[animal], tmed60[animal], tmed90[animal], tmed120[animal] = ([]for i in range(6))
gs00 = gs[0, 0:12].subgridspec(1,12)
for i in range(0, 12):
    med120[animal] = np.append(med120[animal], median120[animal][i])
    tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])


axs= fig.add_subplot(gs00[0, 1])
axs.set_xlim(0, 2)
axs.set_ylim(0, 130)

time120, traj120 = compute_median_trajectory(med120[animal], tmed120[animal])
axs = plt.plot(time120, traj120, color= rat_markers[animal][0], lw=2, alpha=1) 
plt.show()

In [ ]:
#TAC plots
fig = plt.figure(constrained_layout = False, figsize=(30.5, 13)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(29, 71)

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)
col4 = slice(25, 30)
col5 = slice(32, 37)
col6 = slice(40, 45)
col7 = slice (50, 55)
col8 = slice (58, 63)
col9 = slice (66, 71)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = across_session_plot("%",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 2], ticks = [[60, 90, 120],[0, 1, 2]], titles_plot_xaxis_yaxis = ['ratio 10 % reward\n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = across_session_plot("90%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = across_session_plot("10%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = across_session_plot("%",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = across_session_plot("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = across_session_plot("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = across_session_plot("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0, 2], ticks = [[60, 90, 120],[0, 1, 2]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = across_session_plot("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 3], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = across_session_plot("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 3], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', ''], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = across_session_plot("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 2], ticks = [[60, 90, 120],[0, 1, 2]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', 'ratio'], datatype = 'runningtime')



session_list = TM20+TM10+TM2+TMrev2+TMrev10+TMrev20
experiment = 'TM_ON'

ax00 = fig.add_subplot(gs[row1, col4])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [25, -25, 0, 25], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col5])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [25, -25, 0, 25], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col6])
ax02 = across_session_plot("%",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [25, -25, 0, 2], ticks = [[-20, -10, -2, 2, 10, 20],[0, 1, 2]], titles_plot_xaxis_yaxis = ['ratio 10 % reward \n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col4])
ax10 = across_session_plot("90%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [25, -25, 0, 15], ticks = [[-20, -10, -2, 2, 10, 20],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col5])
ax11 = across_session_plot("10%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [25, -25, 0, 15], ticks = [[-20, -10, -2, 2, 10, 20],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col6])
ax12 = across_session_plot("%",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [25, -25, 0, 5], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col4])
ax20 = across_session_plot("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [25, -25, 20, 100], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col5])
ax21 = across_session_plot("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [25, -25, 20, 100], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col6])
ax22 = across_session_plot("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [25, -25, 0, 2], ticks = [[-20, -10, -2, 2, 10, 20],[0, 1, 2]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col4])
ax30 = across_session_plot("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [25, -25, 0, 3], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', 'Treadmill speed\n(cm/s)', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col5])
ax31 = across_session_plot("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [25, -25, 0, 3], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', 'Treadmill speed\n(cm/s)', ''], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col6])
ax32 = across_session_plot("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [25, -25, 0, 2], ticks = [[-20, -10, -2, 2, 10, 20],[0, 1, 2]], titles_plot_xaxis_yaxis = ['', 'Treadmill speed\n(cm/s)', 'ratio'], datatype = 'runningtime')


#save_plot_as_png('dualplot2', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Poster plots
fig = plt.figure(constrained_layout = False, figsize=(30.5, 14.5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(29, 71)

row1 = slice(0, 5)
row2 = slice(8, 13)
row3 = slice(16, 21)
row4 = slice(24, 29)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)
col4 = slice(25, 30)
col5 = slice(32, 37)
col6 = slice(40, 45)
col7 = slice (50, 55)
col8 = slice (58, 63)
col9 = slice (66, 71)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = across_session_plot("%",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10 % reward\n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = across_session_plot("90%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = across_session_plot("10%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = across_session_plot("%",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = across_session_plot("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = across_session_plot("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = across_session_plot("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = across_session_plot("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 3], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = across_session_plot("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 3], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', ''], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = across_session_plot("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 2], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', 'ratio'], datatype = 'runningtime')



session_list = TM20+TM10+TM2+TMrev2+TMrev10+TMrev20
experiment = 'TM_ON'

ax00 = fig.add_subplot(gs[row1, col4])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [25, -25, 0, 25], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col5])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [25, -25, 0, 25], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col6])
ax02 = across_session_plot("%",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [25, -25, 0, 1.50], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['ratio 10 % reward \n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col4])
ax10 = across_session_plot("90%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [25, -25, 0, 15], ticks = [[-20, -10, -2, 2, 10, 20],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col5])
ax11 = across_session_plot("10%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [25, -25, 0, 15], ticks = [[-20, -10, -2, 2, 10, 20],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col6])
ax12 = across_session_plot("%",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [25, -25, 0, 5], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col4])
ax20 = across_session_plot("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [25, -25, 20, 100], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col5])
ax21 = across_session_plot("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [25, -25, 20, 100], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col6])
ax22 = across_session_plot("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [25, -25, 0, 1.50], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col4])
ax30 = across_session_plot("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [25, -25, 0, 3], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', 'Treadmill speed\n(cm/s)', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col5])
ax31 = across_session_plot("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [25, -25, 0, 3], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', 'Treadmill speed\n(cm/s)', ''], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col6])
ax32 = across_session_plot("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [25, -25, 0, 2], ticks = [[-20, -10, -2, 2, 10, 20],[]], titles_plot_xaxis_yaxis = ['', 'Treadmill speed\n(cm/s)', 'ratio'], datatype = 'runningtime')



session_list1 = dist60+dist90+dist120
experiment1 = 'Distance'
session_list2 = TM20+TM10+TM2+TMrev2+TMrev10+TMrev20
experiment2 = 'TM_ON'

ax00 = fig.add_subplot(gs[row1, col7])
ax00 = corr_twoExps("90%", animalList, nb_runsBin, [], session_list1, session_list2, experiment1, experiment2, plot_axes = [10, 20, 10, 20], ticks = [[], []], titles_plot_xaxis_yaxis = ['# runs', 'nb_runs\n90% Distance experiment', 'nb_runs\n90% TM_ON experiment'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col8])
ax01 = corr_twoExps("10%", animalList, nb_runsBin, [], session_list1, session_list2, experiment1, experiment2, plot_axes = [0, 10, 0, 10], ticks = [[], []], titles_plot_xaxis_yaxis = ['# runs', 'nb_runs\n10% Distance experiment', 'nb_runs\n10% TM_ON experiment'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col9])
ax02 = corr_twoExps("%",   animalList, nb_runsBin, [], session_list1, session_list2, experiment1, experiment2, plot_axes = [0.2, 0.6, 0.2, 0.6], ticks = [[], []], titles_plot_xaxis_yaxis = ['# runs', 'ratio nb_runs\nDistance experiment', 'ratio nb_runs\nTM_ON experiment'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col7])
ax10 = corr_twoExps("90%", animalList, timeStayInLeftBin, timeStayInRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 4, 1, 4], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. idle time (s)', 'idletime\n90% Distance experiment', 'idletime\n90% TM_ON experiment'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col8])
ax11 = corr_twoExps("10%", animalList, timeStayInLeftBin, timeStayInRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [5, 15, 5, 15], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. idle time (s)', 'idletime\n10% Distance experiment', 'idletime\n10% TM_ON experiment'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col9])
ax12 = corr_twoExps("%",   animalList, timeStayInLeftBin, timeStayInRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 5, 1, 5], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. idle time (s)', 'ratio idletime\nDistance experiment', 'ratio idletime\nTM_ON experiment'], datatype = 'idletime')

ax60 = fig.add_subplot(gs[row3, col7])
ax60 = corr_twoExps("90%", animalList, speedRunToLeftBin, speedRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [40, 70, 40, 70], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n90% Distance experiment', 'avgrunspeed\n90% TM_ON experiment'], datatype = 'avgrunspeed')
ax70 = fig.add_subplot(gs[row3, col8])
ax70 = corr_twoExps("10%", animalList, speedRunToLeftBin, speedRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [40, 70, 40, 70], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n10% Distance experiment', 'avgrunspeed\n10% TM_ON experiment'], datatype = 'avgrunspeed')
ax80 = fig.add_subplot(gs[row3, col9])
ax80 = corr_twoExps("%",   animalList, speedRunToLeftBin, speedRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [0.8, 1.2, 0.8, 1.2], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'ratio avgrunspeed\nDistance experiment', 'ratio avgrunspeed\nTM_ON experiment'], datatype = 'avgrunspeed')

ax60 = fig.add_subplot(gs[row4, col7])
ax60 = corr_twoExps("90%", animalList, timeRunToLeftBin, timeRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 2, 1, 2], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n90% Distance experiment', 'avgrunspeed\n90% TM_ON experiment'], datatype = 'avgrunspeed')
ax70 = fig.add_subplot(gs[row4, col8])
ax70 = corr_twoExps("10%", animalList, timeRunToLeftBin, timeRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 3, 1, 3], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n10% Distance experiment', 'avgrunspeed\n10% TM_ON experiment'], datatype = 'avgrunspeed')
ax80 = fig.add_subplot(gs[row4, col9])
ax80 = corr_twoExps("%",   animalList, timeRunToLeftBin, timeRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [0, 2, 0, 2], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'ratio avgrunspeed\nDistance experiment', 'ratio avgrunspeed\nTM_ON experiment'], datatype = 'avgrunspeed')



#save_plot_as_png('dualplot2', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(10, 14))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(26, 19)

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(14, 19)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = across_Binsession_plot("60",  animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['60',  '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = across_Binsession_plot("90",  animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['90',  '', '# runs'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = across_Binsession_plot("120", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['120', '', '# runs'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = across_Binsession_plot("60",  animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['60',  '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = across_Binsession_plot("90",  animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['90',  '', 'avg. idle time (s)'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = across_Binsession_plot("120", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['120', '', 'avg. idle time (s)'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = across_Binsession_plot("60",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['60',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = across_Binsession_plot("90",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['90',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = across_Binsession_plot("120", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['120', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = across_Binsession_plot("60",  animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['60',  'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = across_Binsession_plot("90",  animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['90',  'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = across_Binsession_plot("120", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['120', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
#save_plot_as_png('test8', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(20, 14))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(26, 40)

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(14, 19)
col4 = slice(21, 26)
col5 = slice(28, 33)
col6 = slice(35, 40)

session_list = TM20+TM10+TM2+TMrev2+TMrev10+TMrev20
experiment = 'TM_ON'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = across_Binsession_plot("-20", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-20', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = across_Binsession_plot("-10", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-10', '', '# runs'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = across_Binsession_plot("-2",  animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-2', '', '# runs'], datatype = 'nb_runs')
ax03 = fig.add_subplot(gs[row1, col4])
ax03 = across_Binsession_plot("2",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['2', '', '# runs'], datatype = 'nb_runs')
ax04 = fig.add_subplot(gs[row1, col5])
ax04 = across_Binsession_plot("10",  animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['10', '', '# runs'], datatype = 'nb_runs')
ax05 = fig.add_subplot(gs[row1, col6])
ax05 = across_Binsession_plot("20",  animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['20', '', '# runs'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = across_Binsession_plot("-20", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['-20', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = across_Binsession_plot("-10", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['-10', '', 'avg. idle time (s)'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = across_Binsession_plot("-2",  animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['-2',  '', 'avg. idle time (s)'], datatype = 'idletime')
ax13 = fig.add_subplot(gs[row2, col4])
ax13 = across_Binsession_plot("2",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['2',   '', 'avg. idle time (s)'], datatype = 'idletime')
ax14 = fig.add_subplot(gs[row2, col5])
ax14 = across_Binsession_plot("10",  animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['10',  '', 'avg. idle time (s)'], datatype = 'idletime')
ax15 = fig.add_subplot(gs[row2, col6])
ax15 = across_Binsession_plot("20",  animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['20',  '', 'avg. idle time (s)'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = across_Binsession_plot("-20", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-20', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = across_Binsession_plot("-10", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-10', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = across_Binsession_plot("-2",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-2',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax23 = fig.add_subplot(gs[row3, col4])
ax23 = across_Binsession_plot("2",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['2',   '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax24 = fig.add_subplot(gs[row3, col5])
ax24 = across_Binsession_plot("10",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['10',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax25 = fig.add_subplot(gs[row3, col6])
ax25 = across_Binsession_plot("20",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['20',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = across_Binsession_plot("-20", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-20', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = across_Binsession_plot("-10", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-10', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = across_Binsession_plot("-2",  animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['-2',  'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax33 = fig.add_subplot(gs[row4, col4])
ax33 = across_Binsession_plot("2",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['2',   'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax34 = fig.add_subplot(gs[row4, col5])
ax34 = across_Binsession_plot("10",  animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['10',  'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax35 = fig.add_subplot(gs[row4, col6])
ax35 = across_Binsession_plot("20",  animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['20',  'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
#save_plot_as_png('test8', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(20, 14))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(26, 40)

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(14, 19)
col4 = slice(21, 26)
col5 = slice(28, 33)
col6 = slice(35, 40)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = rats_Binsession_plot("RatF00", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF00', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = rats_Binsession_plot("RatF01", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF01', '', '# runs'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = rats_Binsession_plot("RatF02", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF02', '', '# runs'], datatype = 'nb_runs')
ax03 = fig.add_subplot(gs[row1, col4])
ax03 = rats_Binsession_plot("RatM00", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM00', '', '# runs'], datatype = 'nb_runs')
ax04 = fig.add_subplot(gs[row1, col5])
ax04 = rats_Binsession_plot("RatM01", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM01', '', '# runs'], datatype = 'nb_runs')
ax05 = fig.add_subplot(gs[row1, col6])
ax05 = rats_Binsession_plot("RatM02", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM02', '', '# runs'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = rats_Binsession_plot("RatF00", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatF00', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = rats_Binsession_plot("RatF01", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatF01', '', 'avg. idle time (s)'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = rats_Binsession_plot("RatF02", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatF02', '', 'avg. idle time (s)'], datatype = 'idletime')
ax13 = fig.add_subplot(gs[row2, col4])
ax13 = rats_Binsession_plot("RatM00", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatM00', '', 'avg. idle time (s)'], datatype = 'idletime')
ax14 = fig.add_subplot(gs[row2, col5])
ax14 = rats_Binsession_plot("RatM01", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatM01', '', 'avg. idle time (s)'], datatype = 'idletime')
ax15 = fig.add_subplot(gs[row2, col6])
ax15 = rats_Binsession_plot("RatM02", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 50], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatM02', '', 'avg. idle time (s)'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = rats_Binsession_plot("RatF00", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF00', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = rats_Binsession_plot("RatF01", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF01', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = rats_Binsession_plot("RatF02", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF02', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax23 = fig.add_subplot(gs[row3, col4])
ax23 = rats_Binsession_plot("RatM00", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM00', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax24 = fig.add_subplot(gs[row3, col5])
ax24 = rats_Binsession_plot("RatM01", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM01', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax25 = fig.add_subplot(gs[row3, col6])
ax25 = rats_Binsession_plot("RatM02", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM02', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = rats_Binsession_plot("RatF00", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF00', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = rats_Binsession_plot("RatF01", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF01', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = rats_Binsession_plot("RatF02", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF02', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax33 = fig.add_subplot(gs[row4, col4])
ax33 = rats_Binsession_plot("RatM00", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM00', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax34 = fig.add_subplot(gs[row4, col5])
ax34 = rats_Binsession_plot("RatM01", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM01', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax35 = fig.add_subplot(gs[row4, col6])
ax35 = rats_Binsession_plot("RatM02", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM02', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
#save_plot_as_png('test8', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(20, 14))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(26, 40)

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(14, 19)
col4 = slice(21, 26)
col5 = slice(28, 33)
col6 = slice(35, 40)

session_list = TM20+TM10+TM2+TMrev2+TMrev10+TMrev20
experiment='TM_ON'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = rats_Binsession_plot("RatF00", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF00', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = rats_Binsession_plot("RatF01", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF01', '', '# runs'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = rats_Binsession_plot("RatF02", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF02', '', '# runs'], datatype = 'nb_runs')
ax03 = fig.add_subplot(gs[row1, col4])
ax03 = rats_Binsession_plot("RatM00", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM00', '', '# runs'], datatype = 'nb_runs')
ax04 = fig.add_subplot(gs[row1, col5])
ax04 = rats_Binsession_plot("RatM01", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM01', '', '# runs'], datatype = 'nb_runs')
ax05 = fig.add_subplot(gs[row1, col6])
ax05 = rats_Binsession_plot("RatM02", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [0, 13, 0, 30], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM02', '', '# runs'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = rats_Binsession_plot("RatF00", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 15], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatF00', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = rats_Binsession_plot("RatF01", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 15], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatF01', '', 'avg. idle time (s)'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = rats_Binsession_plot("RatF02", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 15], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatF02', '', 'avg. idle time (s)'], datatype = 'idletime')
ax13 = fig.add_subplot(gs[row2, col4])
ax13 = rats_Binsession_plot("RatM00", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 15], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatM00', '', 'avg. idle time (s)'], datatype = 'idletime')
ax14 = fig.add_subplot(gs[row2, col5])
ax14 = rats_Binsession_plot("RatM01", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 15], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatM01', '', 'avg. idle time (s)'], datatype = 'idletime')
ax15 = fig.add_subplot(gs[row2, col6])
ax15 = rats_Binsession_plot("RatM02", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [0, 13, 0, 15], ticks = [[*range(1, 13, 1)],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['RatM02', '', 'avg. idle time (s)'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = rats_Binsession_plot("RatF00", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF00', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = rats_Binsession_plot("RatF01", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF01', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = rats_Binsession_plot("RatF02", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF02', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax23 = fig.add_subplot(gs[row3, col4])
ax23 = rats_Binsession_plot("RatM00", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM00', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax24 = fig.add_subplot(gs[row3, col5])
ax24 = rats_Binsession_plot("RatM01", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM01', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax25 = fig.add_subplot(gs[row3, col6])
ax25 = rats_Binsession_plot("RatM02", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM02', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = rats_Binsession_plot("RatF00", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF00', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = rats_Binsession_plot("RatF01", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF01', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = rats_Binsession_plot("RatF02", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatF02', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax33 = fig.add_subplot(gs[row4, col4])
ax33 = rats_Binsession_plot("RatM00", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM00', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax34 = fig.add_subplot(gs[row4, col5])
ax34 = rats_Binsession_plot("RatM01", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM01', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
ax35 = fig.add_subplot(gs[row4, col6])
ax35 = rats_Binsession_plot("RatM02", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [0, 13, 0, 5], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['RatM02', 'Blocks', 'avg. run time (s)'], datatype = 'runningtime')
#save_plot_as_png('test8', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Rwd% plots
fig = plt.figure(constrained_layout = False, figsize=(10, 14))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(26, 20)

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist90
experiment ='Distance'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = rwd_plot("%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [-30, 130, 0, 25], ticks = [[10, 90],[]], titles_plot_xaxis_yaxis = ['', '', '# runs'], datatype = 'nb_runs')
ax10 = fig.add_subplot(gs[row2, col1])
ax10 = rwd_plot("%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [-30, 130, 0, 15], ticks = [[10, 90],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', 'avg. idle time (s)'], datatype = 'idletime')
ax20 = fig.add_subplot(gs[row3, col1])
ax20 = rwd_plot("%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [-30, 130, 20, 100], ticks = [[10, 90],[]], titles_plot_xaxis_yaxis = ['', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax30 = fig.add_subplot(gs[row4, col1])
ax30 = rwd_plot("%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [-30, 130, 0, 2.5], ticks = [[10, 90],[]], titles_plot_xaxis_yaxis = ['', 'rwd%', 'avg. run time (s)'], datatype = 'runningtime')
#save_plot_as_png('rwd%', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
#Poster plots
fig = plt.figure(constrained_layout = False, figsize=(30.5, 14.5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(29, 71)

row1 = slice(0, 5)
row2 = slice(8, 13)
row3 = slice(16, 21)
row4 = slice(24, 29)

col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)
col4 = slice(25, 30)
col5 = slice(32, 37)
col6 = slice(40, 45)
col7 = slice (50, 55)
col8 = slice (58, 63)
col9 = slice (66, 71)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90 % reward', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col2])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10 % reward', '', ''], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col3])
ax02 = across_session_plot("%",   animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10 % reward\n/ 90 % reward', '', 'ratio'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col1])
ax10 = across_session_plot("90%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = across_session_plot("10%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = across_session_plot("%",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col1])
ax20 = across_session_plot("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col2])
ax21 = across_session_plot("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', ''], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col3])
ax22 = across_session_plot("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', '', 'ratio'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col1])
ax30 = across_session_plot("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col2])
ax31 = across_session_plot("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', ''], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col3])
ax32 = across_session_plot("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 2], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['', 'Distance (cm)', 'ratio'], datatype = 'runningtime')


session_list = postVacs60BIS+postVacs90_2+postVacs120
experiment = 'Distance'

ax00 = fig.add_subplot(gs[row1, col4])
ax00 = across_session_plot("90%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', '', '# runs'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col5])
ax01 = across_session_plot("10%", animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 25], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', '', '# runs'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col6])
ax02 = across_session_plot("%"  , animalList, session_list, nb_runsBin, [], experiment, plot_axes = [55, 125, 0, 1.5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', '', 'ratio'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col4])
ax10 = across_session_plot("90%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['90%', '', 'avg. idle time (s)'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col5])
ax11 = across_session_plot("10%", animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 15], ticks = [[60, 90, 120],[0, 5, 10, 15]], titles_plot_xaxis_yaxis = ['10%', '', 'avg. idle time (s)'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col6])
ax12 = across_session_plot("%",   animalList, session_list, timeStayInLeftBin, timeStayInRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', '', 'ratio'], datatype = 'idletime')

ax20 = fig.add_subplot(gs[row3, col4])
ax20 = across_session_plot("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax21 = fig.add_subplot(gs[row3, col5])
ax21 = across_session_plot("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 100], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
ax22 = fig.add_subplot(gs[row3, col6])
ax22 = across_session_plot("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0, 1.50], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', '', 'ratio'], datatype = 'avgrunspeed')

ax30 = fig.add_subplot(gs[row4, col4])
ax30 = across_session_plot("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row4, col5])
ax31 = across_session_plot("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 5], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row4, col6])
ax32 = across_session_plot("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 2], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', 'Distance', 'ratio'], datatype = 'runningtime')


session_list1 = dist60+dist90+dist120
session_list2 = postVacs60BIS + postVacs90_2 + postVacs120
experiment1 ='Distance'
experiment2 ='Distance'

ax00 = fig.add_subplot(gs[row1, col7])
ax00 = corr_twoExps("90%", animalList, nb_runsBin, [], session_list1, session_list2, experiment1, experiment2, plot_axes = [10, 20, 10, 20], ticks = [[], []], titles_plot_xaxis_yaxis = ['# runs', 'nb_runs\n90% Distance experiment', 'nb_runs\n90% PostVacs experiment'], datatype = 'nb_runs')
ax01 = fig.add_subplot(gs[row1, col8])
ax01 = corr_twoExps("10%", animalList, nb_runsBin, [], session_list1, session_list2, experiment1, experiment2, plot_axes = [0, 10, 0, 10], ticks = [[], []], titles_plot_xaxis_yaxis = ['# runs', 'nb_runs\n10% Distance experiment', 'nb_runs\n10% PostVacs experiment'], datatype = 'nb_runs')
ax02 = fig.add_subplot(gs[row1, col9])
ax02 = corr_twoExps("%",   animalList, nb_runsBin, [], session_list1, session_list2, experiment1, experiment2, plot_axes = [0.2, 0.6, 0.2, 0.6], ticks = [[], []], titles_plot_xaxis_yaxis = ['# runs', 'ratio nb_runs\nDistance experiment', 'ratio nb_runs\nPostVacs experiment'], datatype = 'nb_runs')

ax10 = fig.add_subplot(gs[row2, col7])
ax10 = corr_twoExps("90%", animalList, timeStayInLeftBin, timeStayInRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 5, 1, 5], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. idle time (s)', 'idletime\n90% Distance experiment', 'idletime\n90% PostVacs experiment'], datatype = 'idletime')
ax11 = fig.add_subplot(gs[row2, col8])
ax11 = corr_twoExps("10%", animalList, timeStayInLeftBin, timeStayInRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [5, 15, 5, 15], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. idle time (s)', 'idletime\n10% Distance experiment', 'idletime\n10% PostVacs experiment'], datatype = 'idletime')
ax12 = fig.add_subplot(gs[row2, col9])
ax12 = corr_twoExps("%",   animalList, timeStayInLeftBin, timeStayInRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [0, 5, 0, 5], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. idle time (s)', 'ratio idletime\nDistance experiment', 'ratio idletime\PostVacs experiment'], datatype = 'idletime')

ax60 = fig.add_subplot(gs[row3, col7])
ax60 = corr_twoExps("90%", animalList, speedRunToLeftBin, speedRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [30, 60, 30, 60], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n90% Distance experiment', 'avgrunspeed\n90% PostVacs experiment'], datatype = 'avgrunspeed')
ax70 = fig.add_subplot(gs[row3, col8])
ax70 = corr_twoExps("10%", animalList, speedRunToLeftBin, speedRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [30, 60, 30, 60], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n10% Distance experiment', 'avgrunspeed\n10% PostVacs experiment'], datatype = 'avgrunspeed')
ax80 = fig.add_subplot(gs[row3, col9])
ax80 = corr_twoExps("%",   animalList, speedRunToLeftBin, speedRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [0.8, 1.2, 0.8, 1.2], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'ratio avgrunspeed\nDistance experiment', 'ratio avgrunspeed\n PostVacs experiment'], datatype = 'avgrunspeed')

ax60 = fig.add_subplot(gs[row4, col7])
ax60 = corr_twoExps("90%", animalList, timeRunToLeftBin, timeRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 2, 1, 2], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n90% Distance experiment', 'avgrunspeed\n90% TM_ON experiment'], datatype = 'avgrunspeed')
ax70 = fig.add_subplot(gs[row4, col8])
ax70 = corr_twoExps("10%", animalList, timeRunToLeftBin, timeRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [1, 3, 1, 3], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'avgrunspeed\n10% Distance experiment', 'avgrunspeed\n10% TM_ON experiment'], datatype = 'avgrunspeed')
ax80 = fig.add_subplot(gs[row4, col9])
ax80 = corr_twoExps("%",   animalList, timeRunToLeftBin, timeRunToRightBin, session_list1, session_list2, experiment1, experiment2, plot_axes = [0, 2, 0, 2], ticks = [[], []], titles_plot_xaxis_yaxis = ['avg. run speed (cm/s)', 'ratio avgrunspeed\nDistance experiment', 'ratio avgrunspeed\nTM_ON experiment'], datatype = 'avgrunspeed')



In [ ]:
###TODO###
#explain how data is processed
# Look at all the stuff in the bottom, possible cool stuff (explore weight effect and evolution /speed /peakspeed /number of runs, etc.)
# look at speed by computing it on X and Y axis, not only X
#compare between days
#add stats on figs
#average track per animal/group
#analyse training data
#stats 
#spread in scatter plot proportional to density*

In [ ]:
for animal in animalList:
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        stringmatch    = re.search(r'\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}', session)
        experimentDay  = str(datetime.datetime.strptime(stringmatch.group(), '%Y_%m_%d_%H_%M_%S'))
        daysSinceadlib = (int(experimentDay[0:4]), int(experimentDay[5:7]), int(experimentDay[8:10]), int(experimentDay[11:13]), int(experimentDay[14:16]), int(experimentDay[17:19]))

        plt.scatter(params[animal, session]['daysSinceadLib'], daysSinceadlib[3]+(daysSinceadlib[4]/60), color = rat_markers[animal][0])

In [ ]:
for animal in animalList:
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        plt.scatter(params[animal, session]['daysSinceadLib'], total_trials[animal, session], color = rat_markers[animal][0])

In [ ]:
for animal in animalList:
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        plt.scatter(params[animal, session]['daysSinceadLib'], totalDistance[animal, session], color = rat_markers[animal][0])


In [ ]:
bar = {}
baar = {}
baz = {}
fig = plt.figure(constrained_layout = False, figsize=(10, 5))
fig.suptitle("cumulplottotaldist", y=0.95, fontsize = 24)
for animal in animalList:
    baz[animal] = []
    bar[animal] = [0]
    baar[animal] = []
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        baz[animal] = np.append(baz[animal], params[animal, session]['daysSinceadLib'])
        bar[animal] = bar[animal] + totalDistance[animal, session]
        baar[animal] = np.append(baar[animal], bar[animal] + totalDistance[animal, session])
        plt.scatter(params[animal, session]['daysSinceadLib'], bar[animal], color = rat_markers[animal][0])
        plt.plot((0, 40), (42195, 42195))
    print(animal, baar[animal][-1])

In [ ]:
for animal in animalList:
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        plt.scatter(params[animal, session]['daysSinceadLib'], totalWater[animal, session][0], color = rat_markers[animal][0])

In [ ]:
for animal in animalList:
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        plt.scatter(params[animal, session]['weight'], totalWater[animal, session][0], color = rat_markers[animal][0])

In [ ]:
avgweight = {}
for animal in animalList:
    avgweight[animal] = []
    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]):
        avgweight[animal].extend([params[animal, session]['weight']])
        plt.scatter(params[animal, session]['daysSinceadLib'], params[animal, session]['weight'], color = rat_markers[animal][0])
        #plt.plot((0, 7, 14, 21, 28), (250, 275, 300, 325, 350), color='g', lw=2, marker="$\u2642$")
        #plt.plot((0, 7, 14, 21, 28), (200, 215, 225, 240, 250), color='r', lw=2, marker="$\u2640$")
    avgweight[animal] = sum(avgweight[animal]) / len(avgweight[animal])
    plt.scatter(40, avgweight[animal], color = rat_markers[animal][0], marker ='x')
    plt.annotate("  AVG\nweight", (38, 300))
    print(animal, avgweight[animal])

# WIP check session health
# same code but lighter to check if everything is ok (tracking, lick data, boundaries)

In [ ]:
def checkHealth(arr, ID, sessionIN, index, redoFig = False, printFigs = False, redoMask=False):
    index = index
    animal = ID 

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### INIT
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # initialise all Var dicts
    params, rat_markers, water = {}, {}, {}
    extractTime, extractPositionX, extractPositionY, extractLickLeft, extractLickRight, framebuffer, solenoid_ON_Left, solenoid_ON_Right, cameraEdit = ({} for i in range(9)) 
    rawTime, rawPositionX, rawPositionY, rawLickLeftX, rawLickRightX, rawLickLeftY, rawLickRightY, smoothMask, rawSpeed = ({} for i in range(9)) 
    binPositionX, binPositionY, binTime, binLickLeftX, binLickRightX, binSolenoid_ON_Left, binSolenoid_ON_Right = ({} for i in range(7))
    leftBoundaryPeak, rightBoundaryPeak, kde = {}, {}, {}
    smoothMask, rawMask, binSpeed, binMask = {}, {}, {}, {}
    goodPos, badPos, goodSpeed, badSpeed = {}, {}, {}, {}
    speedToRightCharacteristics, speedToLeftCharacteristics, speedToRightCharacteristicsBin, speedToLeftCharacteristicsBin = {}, {}, {}, {}
    limspeedRunToRight, limspeedRunToLeft, limstayRight, limstayLeft, all_speedRunToRight, all_speedRunToLeft, all_timeRunToRight, all_timeRunToLeft, all_timeStayInRight, all_timeStayInLeft, all_TtrackStayInRight, all_TtrackStayInLeft, all_instantSpeedRight, all_instantSpeedLeft, all_maxSpeedRight, all_maxSpeedLeft, good_speedRunToRight, good_speedRunToLeft, good_XtrackRunToRight, good_XtrackRunToLeft, good_timeRunToRight, good_timeRunToLeft, bad_speedRunToRight, bad_speedRunToLeft, bad_XtrackRunToRight,bad_XtrackRunToLeft, bad_timeRunToRight, bad_timeRunToLeft, good_instantSpeedRight, good_instantSpeedLeft, good_maxSpeedRight, good_maxSpeedLeft, bad_instantSpeedRight, bad_instantSpeedLeft, bad_maxSpeedRight, bad_maxSpeedLeft, good_timeStayInRight, good_timeStayInLeft, good_XtrackStayInRight, good_XtrackStayInLeft, good_TtrackStayInRight, good_TtrackStayInLeft, bad_timeStayInRight, bad_timeStayInLeft, bad_XtrackStayInRight, bad_XtrackStayInLeft, bad_TtrackStayInRight, bad_TtrackStayInLeft, lick_arrivalRight, lick_drinkingRight, lick_waitRight, lick_arrivalLeft, lick_drinkingLeft, lick_waitLeft = ({} for i in range(54))
    limspeedRunToRightBin, limspeedRunToLeftBin, limstayRightBin, limstayLeftBin, all_speedRunToRightBin, all_speedRunToLeftBin, all_timeRunToRightBin, all_timeRunToLeftBin, all_timeStayInRightBin, all_timeStayInLeftBin, all_TtrackStayInRightBin, all_TtrackStayInLeftBin, all_instantSpeedRightBin, all_instantSpeedLeftBin, all_maxSpeedRightBin, all_maxSpeedLeftBin, good_speedRunToRightBin, good_speedRunToLeftBin, good_XtrackRunToRightBin, good_XtrackRunToLeftBin, good_timeRunToRightBin, good_timeRunToLeftBin, bad_speedRunToRightBin, bad_speedRunToLeftBin, bad_XtrackRunToRightBin, bad_XtrackRunToLeftBin, bad_timeRunToRightBin, bad_timeRunToLeftBin, good_instantSpeedRightBin, good_instantSpeedLeftBin, good_maxSpeedRightBin, good_maxSpeedLeftBin, bad_instantSpeedRightBin, bad_instantSpeedLeftBin, bad_maxSpeedRightBin, bad_maxSpeedLeftBin, good_timeStayInRightBin, good_timeStayInLeftBin, good_XtrackStayInRightBin, good_XtrackStayInLeftBin, good_TtrackStayInRightBin, good_TtrackStayInLeftBin, bad_timeStayInRightBin, bad_timeStayInLeftBin, bad_XtrackStayInRightBin, bad_XtrackStayInLeftBin, bad_TtrackStayInRightBin, bad_TtrackStayInLeftBin, lick_arrivalRightBin, lick_drinkingRightBin, lick_waitRightBin, lick_arrivalLeftBin, lick_drinkingLeftBin, lick_waitLeftBin = ({} for i in range(54))
    nb_runs_to_rightBin, nb_runs_to_leftBin, nb_runsBin, total_trials = {}, {}, {}, {}
    nb_rewardBlockLeft, nb_rewardBlockRight,nbWaterLeft, nbWaterRight, totalWater, totalDistance =({} for i in range(6))
    rawLickLeftplot, rawLickRightplot = {}, {}

    palette = [(0.55, 0.0, 0.0),  (0.8, 0.36, 0.36),   (1.0, 0.27, 0.0),   (0.0, 0.39, 0.0),    (0.13, 0.55, 0.13),   (0.2, 0.8, 0.2)]### we use RGB [0-1] not [0-255]. See www.colorhexa.com for conversion #old#palette = ['darkred', 'indianred', 'orangered', 'darkgreen', 'forestgreen', 'limegreen']
    if fnmatch.fnmatch(animal, 'RatF*'): rat_markers[animal]=[palette[index], "$\u2640$"]
    elif fnmatch.fnmatch(animal, 'RatM*'): rat_markers[animal]=[palette[index], "$\u2642$"]
    elif fnmatch.fnmatch(animal, 'Rat00*'): rat_markers[animal]=[palette[index], "$\u2426$"]
    else: print("error, this is not a rat you got here")

    if sessionIN != []: sessionList = sessionIN
    else: sessionList = sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")])
    arr[index] = 0
    for sessionindex, session in enumerate(sessionList):
        #time.sleep(0.1*(index+1))
        #clear_output(wait=True)
        #update_progress(arr[:])
        figPath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session+os.sep+"Figures" +os.sep+"healthFIG%s.png"%session
        if os.path.exists(figPath) and (not redoFig):
            if printFigs == True: display(Image(filename=figPath))
        else:

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### LOAD AND PREPROCESS DATA
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

            #extract/compute parameters from behav_params and create a parameter dictionnary for each rat and each session
            #change of behav_param format 07/2020 -> labview ok 27/07/2020 before nOk #format behavparam ? #catchup manual up to 27/07
            params[animal, session] = { "sessionDuration" : read_params(animal, session, "sessionDuration"),
                                        "acqPer" : read_params(animal, session, "acqPer"),
                                        "waterLeft" : round((read_params(animal, session, "waterLeft", valueType = float) - read_params(animal, session, "cupWeight", valueType = float))/10*1000, 2),
                                        "waterRight" : round((read_params(animal, session, "waterRight", valueType = float) - read_params(animal, session, "cupWeight", valueType = float))/10*1000, 2), 
                                        "treadmillDist" : read_params(animal, session, "treadmillSize"),      
                                        "weight" : read_params(animal, session, "ratWeight"), 
                                        "lastWeightadlib" : read_params(animal, session, "ratWeightadlib"),
                                        "lastDayadlib" : read_params(animal, session, "lastDayadlib"),
                                        "lickthresholdLeft" : read_params(animal, session, "lickthresholdLeft"), #added in Labview 2021/07/06. Now uses the custom lickthreshold for each side. Useful when lickdata baseline drifts and value is directly changed in LV. Only one session might be bugged, so this parameter is session specific. Before, the default value (300) was used and modified manually during the analysis. 
                                        "lickthresholdRight" : read_params(animal, session, "lickthresholdRight"),
                                        "realEnd" : str(read_params(animal, session, "ClockStop"))} 

            #initialize boundaries to be computed later using the KDE function
            params[animal, session]["boundaries"] = []

            #compute number of days elapsed between experiment day and removal of the water bottle
            lastDayadlib   = str(datetime.datetime.strptime(str(read_params(animal, session, "lastDayadlib")), "%Y%m%d").date())
            stringmatch    = re.search(r'\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}', session)
            experimentDay  = str(datetime.datetime.strptime(stringmatch.group(), '%Y_%m_%d_%H_%M_%S'))
            daysSinceadlib = datetime.date(int(experimentDay[0:4]), int(experimentDay[5:7]), int(experimentDay[8:10])) - datetime.date(int(lastDayadlib[0:4]), int(lastDayadlib[5:7]), int(lastDayadlib[8:10]))
            params[animal, session]["daysSinceadLib"] = daysSinceadlib.days

            #compute IRL elapsed session time
            if params[animal, session]['realEnd'] != 'None':
                startExpe = datetime.time(int(experimentDay[11:13]), int(experimentDay[14:16]), int(experimentDay[17:19]))
                endExpe   = datetime.time(hour = int(params[animal, session]['realEnd'][0:2]), minute = int(params[animal, session]['realEnd'][2:4]), second = int(params[animal, session]['realEnd'][4:6]))
                params[animal, session]["realSessionDuration"] = datetime.datetime.combine(datetime.date(1, 1, 1), endExpe) - datetime.datetime.combine(datetime.date(1, 1, 1), startExpe)
            else: params[animal, session]["realSessionDuration"] = None

            #determine block duration set based on the block timing defined in labview. 1 block in labview is comprised of a ON period and a OFF period. Max 12 blocks in LabView (12 On + 12 Off)*repeat.
            blocklist = []# raw blocks from LabView -> 1 block (ON+OFF) + etc
            for blockN in range(1,13): #13? or more ? Max 12 blocks, coded in LabView...
                #add block if  block >0 seconds then get data from file. 
                #Data from behav_params as follows: Block N°: // ON block Duration // OFF block duration // Repeat block // % reward ON // % reward OFF // Treadmill speed.
                if read_params(animal, session, "Block "+ str(blockN), dataindex =  -6, valueType = str) != 0:
                    blocklist.append([read_params(animal, session, "Block "+ str(blockN), dataindex =  -6, valueType = str), read_params(animal, session, "Block "+ str(blockN), dataindex =  -5, valueType = str), 
                                    read_params(animal, session, "Block "+ str(blockN), dataindex =  -4, valueType = str), read_params(animal, session, "Block "+ str(blockN), dataindex =  -3, valueType = str), 
                                    read_params(animal, session, "Block "+ str(blockN), dataindex =  -2, valueType = str), read_params(animal, session, "Block "+ str(blockN), dataindex =  -1, valueType = str), blockN])
            
            #create an array [start_block, end_block] for each block using the values we have just read -> 1 block ON + 1 bloc OFF + etc.
            timecount, blockON_start, blockON_end, blockOFF_start, blockOFF_end = 0, 0, 0, 0, 0
            blocks = [] #blocks that we are going to use in the data processing. 1 block ON + 1 bloc OFF + etc.
            rewardP_ON = [] #probability of getting the reward in each ON phase 
            rewardP_OFF = [] #same for OFF
            treadmillSpeed = [] #treadmill speed for each block (ON + OFF blocks not differenciated for now)
            rewardProbaBlock = []
            for block in blocklist:
                for repeat in range(0, block[2]): #in essence blocks = [a, b], [b, c], [c, d], ...
                    blockON_start = timecount
                    timecount += block[0]
                    blockON_end = timecount
                    blockOFF_start = timecount
                    timecount += block[1]
                    blockOFF_end = timecount
                    blocks.append([blockON_start, blockON_end])
                    if blockOFF_start - blockOFF_end != 0:
                        blocks.append([blockOFF_start, blockOFF_end])
                    rewardP_ON.append(block[3])
                    rewardP_OFF.append(block[4])
                    rewardProbaBlock.extend(block[3:5])
                    treadmillSpeed.append(block[5])
                    treadmillSpeed.append(block[5])
            params[animal, session]["blocks"], params[animal, session]["rewardP_ON"], params[animal, session]["rewardP_OFF"], params[animal, session]["treadmillSpeed"], params[animal, session]['rewardProbaBlock'] = blocks, rewardP_ON, rewardP_OFF, treadmillSpeed, rewardProbaBlock
            #print(blocks)
            #Extract data for each .position file generated from LabView
            #Data loaded : time array, position of the animal X and Y axis, Licks to the left and to the right, and frame number
            extractTime[animal, session]      = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[3])#old format = 5
            extractPositionX[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[4])#old format = 6
            extractPositionY[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[5])
            extractLickLeft[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[6])
            extractLickRight[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[7])
            solenoid_ON_Left[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[8])
            solenoid_ON_Right[animal, session]= read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[9])
            framebuffer[animal, session]      = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[10])
            cameraEdit[animal, session]       = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[11])

            #Cut leftover data at the end of the session (e.g. session is 1800s long, data goes up to 1820s because session has not been stopped properly/stopped manually, so we remove the extra 20s)
            rawTime[animal, session]          = extractTime[animal, session][extractTime[animal, session]      <= params[animal, session]["sessionDuration"]]
            rawPositionX[animal, session]     = extractPositionX[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            rawPositionY[animal, session]     = extractPositionY[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            rawLickLeftX[animal, session]     = extractLickLeft[animal, session][extractTime[animal, session]  <= params[animal, session]["sessionDuration"]]
            rawLickLeftY[animal, session]     = extractLickLeft[animal, session][extractTime[animal, session]  <= params[animal, session]["sessionDuration"]]# not needed, check
            rawLickLeftplot[animal, session]     = extractLickLeft[animal, session][extractTime[animal, session]  <= params[animal, session]["sessionDuration"]]
            rawLickRightplot[animal, session]     = extractLickRight[animal, session][extractTime[animal, session]  <= params[animal, session]["sessionDuration"]]# not needed, check
            rawLickRightX[animal, session]    = extractLickRight[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            rawLickRightY[animal, session]    = extractLickRight[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]# not needed, check
            solenoid_ON_Left[animal, session] = solenoid_ON_Left[animal, session][extractTime[animal, session] <= params[animal, session]["sessionDuration"]]
            solenoid_ON_Right[animal, session]= solenoid_ON_Right[animal, session][extractTime[animal, session]<= params[animal, session]["sessionDuration"]]# not needed, check
            
            #convert data from px to cm
            rawPositionX[animal, session], rawPositionY[animal, session] = datapx2cm(rawPositionX[animal, session]), datapx2cm(rawPositionY[animal, session])
            rawSpeed[animal, session]  = compute_speed(rawPositionX[animal, session], rawTime[animal, session])
            smoothMask[animal,session] = np.array([True])

            #smoothing
            smoothPos, smoothSpeed = True, True
            sigmaPos, sigmaSpeed = 2, 2 #seems to work, less: not smoothed enough, more: too smoothed, not sure how to objectively compute an optimal value.
            if smoothPos == True:
                if smoothSpeed == True:
                    rawPositionX[animal, session]  = smooth(rawPositionX[animal, session], sigmaPos)
                    rawSpeed[animal, session]      = smooth(compute_speed(rawPositionX[animal, session], rawTime[animal, session]), sigmaSpeed)
                else: rawPositionX[animal, session]= smooth(rawPositionX[animal, session], sigmaPos)

            # Load lick data -- Licks == measure of conductance at the reward port. Conductance is ____ and when lick, increase of conductance so ___|_|___, we define it as a lick if it is above a threshold. But baseline value can randomly increase like this ___----, so baseline can be above threshold, so false detections. -> compute moving median to get the moving baseline (median, this way we eliminate the peaks in the calculation of the baseline) and then compare with threshold. __|_|__---|---|----
            window = 200
            if params[animal, session]["lickthresholdLeft"] == None: params[animal, session]["lickthresholdLeft"] = 300
            if params[animal, session]["lickthresholdRight"]== None: params[animal, session]["lickthresholdRight"] = 300
            rawLickLeftX[animal, session]  = [k if i-j >= params[animal, session]["lickthresholdLeft"]  else 0 for i, j, k in zip(rawLickLeftX[animal, session],  reversemovinmedian(rawLickLeftX[animal, session],  window), rawPositionX[animal, session])]
            rawLickRightX[animal, session] = [k if i-j >= params[animal, session]["lickthresholdRight"] else 0 for i, j, k in zip(rawLickRightX[animal, session], reversemovinmedian(rawLickRightX[animal, session], window), rawPositionX[animal, session])]
            
            # Specify if a session has lick data problems, so we don't discard the whole session (keep the run behavior, remove lick data)     
            if   all(v == 0 for v in rawLickLeftX[animal, session]):  params[animal, session]["hasLick"] = False
            elif all(v == 0 for v in rawLickRightX[animal, session]): params[animal, session]["hasLick"] = False       
            elif animal + " " + session in lickBug: params[animal, session]["hasLick"] = False
            else: params[animal, session]["hasLick"] = True

            # Water data. Drop size and volume rewarded. Compute drop size for each reward port. Determine if drops are equal, or which one is bigger. Assign properties (e.g. line width for plots) accordingly.
            limitWater_diff = 5
            watL = round(params[animal, session]["waterLeft"], 1) #print(round(params[animal, session]["waterLeft"], 1), "µL/drop")
            watR = round(params[animal, session]["waterRight"], 1) #print(round(params[animal, session]["waterRight"], 1), "µL/drop")
            if watL-(watL*limitWater_diff/100) <= watR <= watL+(watL*limitWater_diff/100): water[animal, session] = ["Same Reward Size", "Same Reward Size", 2, 2] #print(session, "::", watL, watR, "     same L-R") #print(watL-(watL*limitWater_diff/100)) #print(watL+(watL*limitWater_diff/100))
            elif watL < watR: water[animal, session] = ["Small Reward", "Big Reward", 1, 5]#print(session, "::", watL, watR, "     bigR")
            elif watL > watR: water[animal, session] = ["Big Reward", "Small Reward", 5, 1] #print(session, "::", watL, watR, "     bigL")
            else: print(session, "error, bypass? Y/N")

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### DATA PROCESSING
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------      
    
            # Compute boundaries
            border = 5 #define arbitrary border
            leftBoundaryPeak[animal, session], rightBoundaryPeak[animal, session], kde[animal, session] = extract_boundaries(rawPositionX[animal, session], animal, session, params[animal, session]['treadmillDist'], height = 0.001)
            for s in boundariesBug: 
                if animal + " " + session == s[0]: 
                    params[animal, session]["boundaries"] = s[1]
                    break
                else: 
                    params[animal, session]["boundaries"] = [rightBoundaryPeak[animal, session] - border, leftBoundaryPeak[animal, session] + border]
            #print(params[animal, session]["boundaries"])

            # Compute or pickle run/stay mask
            maskpicklePath = root+os.sep+animal+os.sep+"Experiments"+os.sep+session+os.sep+"Analysis"+os.sep+"mask.p"
            if os.path.exists(maskpicklePath) and (not redoMask): 
                binMask[animal, session] = get_from_pickle(animal, session, name="mask.p")
            else: 
                if animal + " " + session in runstaysepbug: 
                    septhreshold = 0.0004
                else: 
                    septhreshold = 0.0002
                print(septhreshold)
                rawMask[animal,session]     = filterspeed(animal, session, rawPositionX[animal, session], rawSpeed[animal, session], rawTime[animal, session], septhreshold, params[animal, session]["treadmillDist"])#threshold 0.0004 seems to work ok for all TM distances. lower the thresh the bigger the wait blob zone taken, which caused problems in 60cm configuration.
                smoothMask[animal, session] = removeSplits_Mask(rawMask, rawPositionX, animal, session, params[animal, session]["treadmillDist"])
                binMask[animal,session]     = fixSplittedRunsMask(animal, session, bin_session(animal, session, smoothMask, rawTime, blocks), blocks)
            smoothMask[animal, session] = stitch([binMask[animal, session]])[0]
            goodPos[animal,session]     = [val[0] if val[1] == True  else None for val in [[i, j] for i, j in zip(rawPositionX[animal,session], smoothMask[animal,session])]]
            badPos[animal,session]      = [val[0] if val[1] == False else None for val in [[i, j] for i, j in zip(rawPositionX[animal,session], smoothMask[animal,session])]]
            goodSpeed[animal,session]   = [val[0] if val[1] == True  else None for val in [[i, j] for i, j in zip(rawSpeed[animal,session], smoothMask[animal,session])]]
            badSpeed[animal,session]    = [val[0] if val[1] == False else None for val in [[i, j] for i, j in zip(rawSpeed[animal,session], smoothMask[animal,session])]]

            total_trials[animal, session] = len(split_a_list_at_zeros(stitch([binMask[animal, session]])[0])) #approximated, because we don't separate and filter runs in there, so nb of runs is deduced from the mask
            totalDistance[animal, session] = sum(abs(np.diff(rawPositionX[animal, session])))/100
            binSolenoid_ON_Left[animal, session]  = reCutBins(solenoid_ON_Left[animal, session], binMask[animal, session])
            binSolenoid_ON_Right[animal, session] = reCutBins(solenoid_ON_Right[animal, session], binMask[animal, session]) 
            
            nb_rewardBlockLeft[animal, session], nb_rewardBlockRight[animal, session],nbWaterLeft[animal, session], nbWaterRight[animal, session] = {}, {}, 0, 0
            for i in range(0, len(params[animal, session]['blocks'])):
                nb_rewardBlockLeft[animal, session][i] = sum([1 if t[0] <= params[animal, session]['rewardProbaBlock'][i] else 0 for t in split_a_list_at_zeros(binSolenoid_ON_Left[animal, session][i])]) #split a list because in data file we have %open written along valve opening time duration (same value multiple time), so we only take the first one, verify >threshold, ...
                nb_rewardBlockRight[animal, session][i] = sum([1 if t[0] <= params[animal, session]['rewardProbaBlock'][i] else 0 for t in split_a_list_at_zeros(binSolenoid_ON_Right[animal, session][i])]) #print(i+1, nb_rewardBlockLeft[animal, session][i], nb_rewardBlockRight[animal, session][i])
            nbWaterLeft[animal, session] = sum(nb_rewardBlockLeft[animal, session].values())
            nbWaterRight[animal, session] = sum(nb_rewardBlockRight[animal, session].values())
            totalWater[animal, session] = round((nbWaterLeft[animal, session] * params[animal, session]["waterLeft"] + nbWaterRight[animal, session] * params[animal, session]["waterRight"])/1000, 2), 'mL'             #totalWater[animal, session] = nbWaterLeft[animal, session] * params[animal, session]["waterLeft"], "+", nbWaterRight[animal, session] * params[animal, session]["waterRight"]

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### FIGURES
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
    
            # Plot figure
            fig = plt.figure(constrained_layout = False, figsize=(20, 16))
            fig.suptitle(session, y=0.9, fontsize = 24)
            gs = fig.add_gridspec(45, 75)
            ax00 = fig.add_subplot(gs[0:7, 0:4])
            ax00 = plot_peak(rawPositionX[animal, session], animal, session, leftBoundaryPeak[animal, session], rightBoundaryPeak[animal, session], kde[animal, session],[0.05, 0, 0],[0,120,0],  marker = [""], xyLabels=["Position (cm)", "%"])
            ax01 = fig.add_subplot(gs[0:7, 5:75])
            ax01 = plot_BASEtrajectoryV2(animal, session, rawTime[animal, session], goodPos[animal, session], badPos[animal, session], rawLickLeftX[animal, session], rawLickRightX[animal, session], params[animal, session]['rewardProbaBlock'], params[animal, session]['blocks'], barplotaxes = [0, params[animal, session]['sessionDuration'], 50, 90, 0, 22, 10],  xyLabels=["Time (min)","", "Position (cm)", "", "", "", 14, 12], title=[session, "", "", "", 16], linewidth = [1.5])
            plt.plot([0, params[animal, session]['sessionDuration']], [params[animal, session]["boundaries"][0], params[animal, session]["boundaries"][0]], ":", color='k', alpha = 0.5)
            plt.plot([0, params[animal, session]['sessionDuration']], [params[animal, session]["boundaries"][1], params[animal, session]["boundaries"][1]], ":", color='k', alpha = 0.5)

            gs00 = gs[8:13, 0:75].subgridspec(2,75)
            ax11 = fig.add_subplot(gs00[0, 5:75])
            ax12 = fig.add_subplot(gs00[1, 0:75])
            ax11.plot(rawTime[animal,session], goodSpeed[animal,session], color = 'dodgerblue')
            ax11.plot(rawTime[animal,session], badSpeed[animal, session], color = 'orange')
            ax11.set_xlabel('time (s)')
            ax11.set_ylabel('speed (cm/s)')
            ax11.set_xlim(0, 3600)
            ax11.set_ylim(-200,200)
            ax11.spines['top'].set_color("none")
            ax11.spines['right'].set_color("none")
            ax11.spines['left'].set_color("none")
            ax11.spines['bottom'].set_color("none")
            ax12.scatter(rawPositionX[animal,session], goodSpeed[animal,session], color = 'dodgerblue', s=0.5)
            ax12.scatter(rawPositionX[animal,session], badSpeed[animal,session], color = 'orange', s=0.5)
            ax12.set_xlabel('position (cm)')
            ax12.set_ylabel('speed (cm/s)')
            ax12.set_xlim(0,130)
            ax12.set_ylim(-150,150)
            ax12.spines['top'].set_color("none")
            ax12.spines['right'].set_color("none")
            ax12.spines['left'].set_color("none")
            ax12.spines['bottom'].set_color("none")
            yline = [0, 120]
            xline = [0,0]
            ax12.plot(yline, xline, ":", color='k')
            
            gs23 = gs[15:20, 0:75].subgridspec(5,5)
            ax231 = fig.add_subplot(gs23[0:5, 0:2])
            if len(framebuffer[animal, session]) != 0:
                ax231.set_title("NbBug/TotFrames: %s/%s = %.2f" %(sum(np.diff(framebuffer[animal, session])-1), len(framebuffer[animal, session]), sum(np.diff(framebuffer[animal, session])-1)/len(framebuffer[animal, session])), fontsize = 16)
            ax231.scatter(list(range(1, len(framebuffer[animal, session]))), [ x-1 for x in np.diff(framebuffer[animal, session])], s=5)
            ax231.set_xlabel("frame index")
            ax231.set_ylabel("dFrame -1 (0 is ok)")
            ax232 = fig.add_subplot(gs23[0:5, 3:5])
            ax232.set_title(params[animal, session]["realSessionDuration"], fontsize = 16)
            ax232.plot(np.diff(rawTime[animal, session]), label = "data")
            ax232.plot(movinavg(np.diff(rawTime[animal, session]), 100), label="moving average")
            ax232.set_xlim(0, len(np.diff(rawTime[animal, session])))
            ax232.set_ylim(0, 0.1)
            ax232.set_xlabel("frame index")
            ax232.set_ylabel("time per frame (s)")

            aaa = range(0, len(rawLickLeftplot[animal, session]))
            ax30 = fig.add_subplot(gs[22:27, 0:75])
            ax30.plot(rawLickLeftplot[animal, session], lw=0.5, c='k')
            ax30.plot(movinmedian(rawLickLeftplot[animal, session], window)+params[animal, session]["lickthresholdLeft"], lw=0.5, c='r')
            ax30.plot([i + params[animal, session]["lickthresholdLeft"] for i in reversemovinmedian(rawLickLeftplot[animal, session], window)], lw=1, c='g')
            ax30.scatter(aaa, [i if i-j >= params[animal, session]["lickthresholdLeft"]  else None for i, j in zip(rawLickLeftplot[animal, session],  reversemovinmedian(rawLickLeftplot[animal, session],  window))], s =10)
            ax30.set_xlabel("time")
            ax30.set_ylabel("conductance")
            ax30.set_title("LEFTLICKS, Lthrsh{0}".format(params[animal, session]["lickthresholdLeft"]), fontsize = 16)
            ax30.set_xlim(0, params[animal, session]['sessionDuration']*25)
            ax30.set_ylim(0, 2000)

            ax31 = fig.add_subplot(gs[28:33, 0:75])
            ax31.plot(rawLickRightplot[animal, session], lw=0.5, c='k')
            ax31.plot(movinmedian(rawLickRightplot[animal, session], window)+params[animal, session]["lickthresholdRight"], lw=0.5, c='r')
            ax31.plot([i + params[animal, session]["lickthresholdRight"] for i in reversemovinmedian(rawLickRightplot[animal, session], window)], lw=1, c='g')
            ax31.scatter(aaa, [i if i-j >= params[animal, session]["lickthresholdRight"] else None for i, j in zip(rawLickRightplot[animal, session], reversemovinmedian(rawLickRightplot[animal, session], window))], s = 10)
            ax31.set_xlabel("time")
            ax31.set_ylabel("conductance")
            ax31.set_title("RIGHTLICKS, Rthrsh{0}".format(params[animal, session]["lickthresholdRight"]), fontsize = 16)
            ax31.set_xlim(0, params[animal, session]['sessionDuration']*25)
            ax31.set_ylim(0, 2000)

            ax32 = fig.add_subplot(gs[34:39, 0:75])
            ax32 = plot_BASEtrajectory(rawTime[animal, session],rawPositionX[animal, session], rawLickLeftX[animal, session], rawLickRightX[animal, session], [0, params[animal, session]['sessionDuration'], 1],[0,120,1],  color = ["b", "c"], marker = ["", "o", 1], linewidth = [0.5], xyLabels=["Position (px)", "Time(s)"])


            %config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
            ax80 = fig.add_subplot(gs[43:45, 0:60])
            ax80.spines['top'].set_color("none")
            ax80.spines['right'].set_color("none")
            ax80.spines['left'].set_color("none")
            ax80.spines['bottom'].set_color("none")
            ax80.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) 
            ax80.tick_params(axis='y', which='both', left=False, right=False, labelleft=False) 
            text = ("sessionDuration: {0} | acqPer: {1} | waterLeft: {2} | waterRight: {3} | treadmillDist: {4} | weight: {5} | lastWeightadlib: {6} | lastDayadlib: {7} | lickthresholdLeft: {8} | lickthresholdRight: {9} | realEnd: {10} | boundaries: {11} | daysSinceadLib: {12} \n realSessionDuration: {13} | blocks: {14} | \n rewardP_ON: {15} | rewardP_OFF: {16} | treadmillSpeed: {17} | rewardProbaBlock: {18} | hasLick: {19}").format(params[animal,session]['sessionDuration'], params[animal,session]['acqPer'], params[animal,session]['waterLeft'], params[animal,session]['waterRight'], params[animal,session]['treadmillDist'], params[animal,session]['weight'], params[animal,session]['lastWeightadlib'], params[animal,session]['lastDayadlib'], params[animal,session]['lickthresholdLeft'], params[animal,session]['lickthresholdRight'], params[animal,session]['realEnd'], params[animal,session]['boundaries'], params[animal,session]['daysSinceadLib'],params[animal,session]['realSessionDuration'], params[animal,session]['blocks'], params[animal,session]['rewardP_ON'], params[animal,session]['rewardP_OFF'], params[animal,session]['treadmillSpeed'], params[animal,session]['rewardProbaBlock'], params[animal,session]['hasLick'])
            ax80 = plt.text(0,0, str(text), wrap = True)

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### SAVE + PICKLE
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

            save_sessionplot_as_png(animal, session, 'healthFIG%s.png'%session, dpi='figure', transparent = False, background = 'w')
            save_as_pickle([totalDistance[animal, session], totalWater[animal, session], total_trials[animal, session]], animal, session, "misc.p")
            save_as_pickle(params[animal, session],     animal, session, "params.p")
            save_as_pickle(binMask[animal, session], animal, session, "mask.p")
            if printFigs == False:
                plt.close('all')

### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### FLUSH
### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
            # Delete all data for this session
            params, rat_markers, water = {}, {}, {}
            extractTime, extractPositionX, extractPositionY, extractLickLeft, extractLickRight, framebuffer, solenoid_ON_Left, solenoid_ON_Right, cameraEdit = ({} for i in range(9)) 
            rawTime, rawPositionX, rawPositionY, rawLickLeftX, rawLickRightX, rawLickLeftY, rawLickRightY, smoothMask, rawSpeed = ({} for i in range(9)) 
            binPositionX, binPositionY, binTime, binLickLeftX, binLickRightX, binSolenoid_ON_Left, binSolenoid_ON_Right = ({} for i in range(7))
            leftBoundaryPeak, rightBoundaryPeak, kde = {}, {}, {}
            smoothMask, rawMask, binSpeed, binMask = {}, {}, {}, {}
            goodPos, badPos, goodSpeed, badSpeed = {}, {}, {}, {}
            speedToRightCharacteristics, speedToLeftCharacteristics, speedToRightCharacteristicsBin, speedToLeftCharacteristicsBin = {}, {}, {}, {}
            limspeedRunToRight, limspeedRunToLeft, limstayRight, limstayLeft, all_speedRunToRight, all_speedRunToLeft, all_timeRunToRight, all_timeRunToLeft, all_timeStayInRight, all_timeStayInLeft, all_TtrackStayInRight, all_TtrackStayInLeft, all_instantSpeedRight, all_instantSpeedLeft, all_maxSpeedRight, all_maxSpeedLeft, good_speedRunToRight, good_speedRunToLeft, good_XtrackRunToRight, good_XtrackRunToLeft, good_timeRunToRight, good_timeRunToLeft, bad_speedRunToRight, bad_speedRunToLeft, bad_XtrackRunToRight,bad_XtrackRunToLeft, bad_timeRunToRight, bad_timeRunToLeft, good_instantSpeedRight, good_instantSpeedLeft, good_maxSpeedRight, good_maxSpeedLeft, bad_instantSpeedRight, bad_instantSpeedLeft, bad_maxSpeedRight, bad_maxSpeedLeft, good_timeStayInRight, good_timeStayInLeft, good_XtrackStayInRight, good_XtrackStayInLeft, good_TtrackStayInRight, good_TtrackStayInLeft, bad_timeStayInRight, bad_timeStayInLeft, bad_XtrackStayInRight, bad_XtrackStayInLeft, bad_TtrackStayInRight, bad_TtrackStayInLeft, lick_arrivalRight, lick_drinkingRight, lick_waitRight, lick_arrivalLeft, lick_drinkingLeft, lick_waitLeft = ({} for i in range(54))
            limspeedRunToRightBin, limspeedRunToLeftBin, limstayRightBin, limstayLeftBin, all_speedRunToRightBin, all_speedRunToLeftBin, all_timeRunToRightBin, all_timeRunToLeftBin, all_timeStayInRightBin, all_timeStayInLeftBin, all_TtrackStayInRightBin, all_TtrackStayInLeftBin, all_instantSpeedRightBin, all_instantSpeedLeftBin, all_maxSpeedRightBin, all_maxSpeedLeftBin, good_speedRunToRightBin, good_speedRunToLeftBin, good_XtrackRunToRightBin, good_XtrackRunToLeftBin, good_timeRunToRightBin, good_timeRunToLeftBin, bad_speedRunToRightBin, bad_speedRunToLeftBin, bad_XtrackRunToRightBin, bad_XtrackRunToLeftBin, bad_timeRunToRightBin, bad_timeRunToLeftBin, good_instantSpeedRightBin, good_instantSpeedLeftBin, good_maxSpeedRightBin, good_maxSpeedLeftBin, bad_instantSpeedRightBin, bad_instantSpeedLeftBin, bad_maxSpeedRightBin, bad_maxSpeedLeftBin, good_timeStayInRightBin, good_timeStayInLeftBin, good_XtrackStayInRightBin, good_XtrackStayInLeftBin, good_TtrackStayInRightBin, good_TtrackStayInLeftBin, bad_timeStayInRightBin, bad_timeStayInLeftBin, bad_XtrackStayInRightBin, bad_XtrackStayInLeftBin, bad_TtrackStayInRightBin, bad_TtrackStayInLeftBin, lick_arrivalRightBin, lick_drinkingRightBin, lick_waitRightBin, lick_arrivalLeftBin, lick_drinkingLeftBin, lick_waitLeftBin = ({} for i in range(54))
            nb_runs_to_rightBin, nb_runs_to_leftBin, nb_runsBin, total_trials = {}, {}, {}, {}
            rawLickLeftplot, rawLickRightplot = {}, {}

        #arr[index] += (1/len(sessionList))
        #clear_output(wait=True)
        #update_progress(arr[:])


In [ ]:
lickBug = []#'RatF01 RatF01_2021_07_21_16_21_18'
notfixed = ['RatM02 RatM02_2021_07_18_17_13_11', 'RatF00 RatF00_2021_08_19_13_14_34', 'RatF00 RatF00_2021_08_19_15_40_01', 'RatF01 RatF01_2021_07_21_16_21_18', 'RatF00 RatF00_2021_07_21_10_43_27', 'RatF00 RatF00_2021_07_21_15_19_33']
F00lostTRACKlick = ['RatF00 RatF00_2021_08_20_10_49_54', 'RatF00 RatF00_2021_08_20_15_40_51']
buggedRatSessions = ['RatF00 RatF00_2021_07_19_13_12_15', 'RatF00_2021_07_19_10_32_00']
boundariesBug = [["RatF00 RatF00_2021_07_19_15_25_33", [40, 20]], ["RatF01 RatF01_2021_07_19_11_09_43", [40,20]], ['RatM02 RatM02_2021_08_19_12_34_15', [35, 17]], ['RatM02 RatM02_2021_08_19_12_34_15', [35, 17]], ['RatM02 RatM02_2021_08_19_12_34_15', [35, 17]], ['RatF00 RatF00_2021_08_18_11_11_19', [75, 22]], ['RatF02 RatF02_2021_08_18_18_08_44', [75, 20]], ['RatF02 RatF02_2021_08_19_17_42_46', [40, 18]], ['RatF02 RatF02_2021_07_18_17_11_30', [75, 23]], ['RatF02 RatF02_2021_07_21_12_46_49', [43, 21]], ['RatF02 RatF02_2021_07_23_17_08_21', [75, 25]], ['RatF02 RatF02_2021_07_26_12_27_04', [40, 22]], ['RatF00 RatF00_2021_07_20_10_09_39', [100, 20]], ['RatF00 RatF00_2021_07_21_10_43_27', [40, 20]], ['RatF00 RatF00_2021_07_21_15_19_33', [40, 20]], ['RatF00 RatF00_2021_07_23_10_38_43', [75, 25]], ['RatF00 RatF00_2021_07_23_15_04_12', [75, 25]], ['RatF00 RatF00_2021_07_24_10_49_55', [100, 25]], ['RatF00 RatF00_2021_07_25_10_54_14', [75, 25]], ['RatF00 RatF00_2021_07_25_15_38_20', [75, 25]], ['RatF00 RatF00_2021_07_26_15_17_06', [45, 22]], ]
runstaysepbug = ['RatF00 RatF00_2021_07_18_10_36_33', 'RatM01 RatM01_2021_07_18_11_40_37', 'RatM01 RatM01_2021_07_18_16_12_12', 'RatM01 RatM01_2021_07_19_16_28_01', 'RatM00 RatM00_2021_08_19_15_40_49', 'RatF01 RatF01_2021_08_19_11_09_37', 'RatM02 RatM02_2021_07_21_12_48_07', 'RatM02 RatM02_2021_07_21_17_25_04', 'RatM02 RatM02_2021_07_25_17_37_53', 'RatM02 RatM02_2021_07_26_12_28_45', 'RatM02 RatM02_2021_07_26_17_21_19', 'RatM01 RatM01_2021_07_21_11_46_29', 'RatM01 RatM01_2021_07_26_11_26_38', 'RatM01 RatM01_2021_07_26_16_19_55', 'RatM00 RatM00_2021_07_21_10_45_30', 'RatM00 RatM00_2021_07_26_10_25_22', 'RatM00 RatM00_2021_07_26_15_18_23', 'RatF02 RatF02_2021_07_20_17_12_04', 'RatF00 RatF00_2021_07_20_10_09_39', 'RatF00 RatF00_2021_07_21_10_43_27', ] 
sessionstofix = ['', '']

#animal = 'RatM02'
#for animal in animalList:
#    for session in sorted([os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]): print(animal, session)#print all so don't have to type
#for session in sessionstofix: print(session)
                # animal, sessions,index, redoCompute = False, printFigs = False, redoMask=False):
index = index
checkHealth([0, 0, 0, 0, 0, 0], 'RatF00', matchsession('RatF00', ""), index, False, False,     False,) #check on one session False,    True,     False, postVacsTrain + postVacs60 + postVacs60BIS + postVacs90_1 + postVacs90_2 + postVacs120

In [ ]:
#check health on all sessions
p = {}
animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
progressHealth = Array('d', [0, 0, 0, 0, 0, 0])
for index, animal in enumerate(animalList):                       # function,      animal, sessions,                                                                                     index, redoFig, printFigs, redoMask
    p[animal] = multiprocessing.Process(target = checkHealth, args=(progressHealth, animal, matchsession(animal, postVacsTrain + postVacs60 + postVacs60BIS +  postVacs90_1 + postVacs90_2  + postVacs120), index,   False,   False,   False,  ))
for index, animal in enumerate(animalList):
    p[animal].start()
for index, animal in enumerate(animalList):
    p[animal].join()
print("end")

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

In [ ]:
endTimeNotebook = datetime.datetime.now()
timeExecutionNotebook = endTimeNotebook-startTimeNotebook
print(timeExecutionNotebook)

In [ ]:
def across_session_plot2(plot, animal_list, session_list, dataLeft, dataRight, experiment, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype): 
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16)
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12) 
    #if experiment == 'TM_ON': ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.yaxis.label.set_color('dodgerblue' if datatype == 'avgrunspeed' else 
                             'dodgerblue' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')
    xdata, ydata = {}, {}
    if experiment == 'Distance':
        data90_60, data90_90, data90_120, data10_60, data10_90, data10_120 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, False)
        for animal in animal_list:
            xdata[animal] = (60, 90, 120)
            if datatype == 'nb_runs': 
                if plot == "90%": 
                    ax.scatter((60, 90, 120), (np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])), marker = 'o', s=3, color = rat_markers[animal][0])
                    ydata[animal] = (np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal]))
                    #ax.errorbar((60, 90, 120), (np.mean(data90_60[animal]), np.mean(data90_90[animal]), np.mean(data90_120[animal])), yerr = (stats.sem(data90_60[animal]), stats.sem(data90_90[animal]), stats.sem(data90_120[animal])), color = rat_markers[animal][0])
                if plot == "10%": 
                    ax.scatter((60, 90, 120), (np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal])), marker = 'o', s=3, color = rat_markers[animal][0])
                    ydata[animal] = (np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal]))
                    #ax.errorbar((60, 90, 120), (np.mean(data10_60[animal]), np.mean(data10_90[animal]), np.mean(data10_120[animal])), yerr = (stats.sem(data10_60[animal]), stats.sem(data10_90[animal]), stats.sem(data10_120[animal])), color = rat_markers[animal][0])
                if plot == "%"  : 
                    ax.scatter((60, 90, 120), ((np.mean(data10_60[animal])  / np.mean(data90_60[animal])), (np.mean(data10_90[animal])  / np.mean(data90_90[animal])), (np.mean(data10_120[animal]) / np.mean(data90_120[animal]))), marker = 'o', s=3, color = rat_markers[animal][0])
                    ydata[animal] = ((np.mean(data10_60[animal])  / np.mean(data90_60[animal])), (np.mean(data10_90[animal])  / np.mean(data90_90[animal])), (np.mean(data10_120[animal]) / np.mean(data90_120[animal])))
            
            else:
                if plot == "90%": 
                    ax.scatter((60, 90, 120), (np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist])), marker = 'o', s=3, color = rat_markers[animal][0], label = animal)
                    ydata[animal] = (np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist]))
                    ax.legend()
                    #ax.errorbar((60, 90, 120), (np.nanmean([item for sublist in data90_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data90_90[animal]  for item in sublist]), np.nanmean([item for sublist in data90_120[animal] for item in sublist])), yerr = (stats.sem([item for sublist in data90_60[animal]  for item in sublist]),  stats.sem([item for sublist in data90_90[animal]  for item in sublist]), stats.sem([item for sublist in data90_120[animal] for item in sublist])), color = rat_markers[animal][0])
                if plot == "10%": 
                    ax.scatter((60, 90, 120), (np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist])), marker = 'o', s=3, color = rat_markers[animal][0])
                    ydata[animal] = (np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist]))
                    #ax.errorbar((60, 90, 120), (np.nanmean([item for sublist in data10_60[animal]  for item in sublist]),  np.nanmean([item for sublist in data10_90[animal]  for item in sublist]), np.nanmean([item for sublist in data10_120[animal] for item in sublist])), yerr = (stats.sem([item for sublist in data10_60[animal]  for item in sublist]),  stats.sem([item for sublist in data10_90[animal]  for item in sublist]), stats.sem([item for sublist in data10_120[animal] for item in sublist])), color = rat_markers[animal][0])
                if plot == "%"  : 
                    ax.scatter((60, 90, 120), ((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_60[animal]  for item in sublist])), 
                                            (np.nanmean([item for sublist in data10_90[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_90[animal]  for item in sublist])), 
                                            (np.nanmean([item for sublist in data10_120[animal] for item in sublist]) / np.nanmean([item for sublist in data90_120[animal] for item in sublist]))), marker = 'o', s=3, color = rat_markers[animal][0])
                    ydata[animal] = ((np.nanmean([item for sublist in data10_60[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_60[animal]  for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_90[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_90[animal]  for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_120[animal] for item in sublist]) / np.nanmean([item for sublist in data90_120[animal] for item in sublist])))

    if experiment == 'TM_ON':
        data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = separate_data(animal_list, session_list, dataLeft, dataRight, experiment, datatype, False)
        for animal in animal_list:
            xdata[animal] = (-20, -10, -2, 2, 10, 20)
            if datatype == 'nb_runs': 
                if plot == "90%": 
                    ydata[animal] = (np.mean(data90_rev20[animal]), np.mean(data90_rev10[animal]), np.mean(data90_rev2[animal]), np.mean(data90_2[animal]), np.mean(data90_10[animal]), np.mean(data90_20[animal]))
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.mean(data90_rev20[animal]), np.mean(data90_rev10[animal]), np.mean(data90_rev2[animal]), np.mean(data90_2[animal]), np.mean(data90_10[animal]), np.mean(data90_20[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0], label = animal)
                    ax.legend()
                if plot == "10%": 
                    ydata[animal] = (np.mean(data10_rev20[animal]), np.mean(data10_rev10[animal]), np.mean(data10_rev2[animal]), np.mean(data10_2[animal]), np.mean(data10_10[animal]), np.mean(data10_20[animal]))
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.mean(data10_rev20[animal]), np.mean(data10_rev10[animal]), np.mean(data10_rev2[animal]), np.mean(data10_2[animal]), np.mean(data10_10[animal]), np.mean(data10_20[animal])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "%" : 
                    ydata[animal] = ((np.mean(data10_rev20[animal]) / np.mean(data90_rev20[animal])), (np.mean(data10_rev10[animal]) / np.mean(data90_rev10[animal])), 
                                             (np.mean(data10_rev2[animal])  / np.mean(data90_rev2[animal])),  (np.mean(data10_2[animal])     / np.mean(data90_2[animal])), 
                                             (np.mean(data10_10[animal])    / np.mean(data90_10[animal])),    (np.mean(data10_20[animal])    / np.mean(data90_20[animal])))
                    ax.plot((-20, -10, -2, 2, 10, 20), ((np.mean(data10_rev20[animal]) / np.mean(data90_rev20[animal])), (np.mean(data10_rev10[animal]) / np.mean(data90_rev10[animal])), 
                                                        (np.mean(data10_rev2[animal])  / np.mean(data90_rev2[animal])),  (np.mean(data10_2[animal])     / np.mean(data90_2[animal])), 
                                                        (np.mean(data10_10[animal])    / np.mean(data90_10[animal])),    (np.mean(data10_20[animal])    / np.mean(data90_20[animal]))), marker = 'o', markersize = 6, color = rat_markers[animal][0])   
            else:
                if plot == "90%":
                    ydata[animal] = (np.nanmean([item for sublist in data90_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev10[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data90_2[animal]     for item in sublist]), np.nanmean([item for sublist in data90_10[animal]    for item in sublist]), np.nanmean([item for sublist in data90_20[animal]    for item in sublist]))
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.nanmean([item for sublist in data90_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data90_rev10[animal] for item in sublist]), 
                                                        np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data90_2[animal]     for item in sublist]), 
                                                        np.nanmean([item for sublist in data90_10[animal]    for item in sublist]), np.nanmean([item for sublist in data90_20[animal]    for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "10%": 
                    ydata[animal] = (np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data10_2[animal]     for item in sublist]), np.nanmean([item for sublist in data10_10[animal]    for item in sublist]), np.nanmean([item for sublist in data10_20[animal]    for item in sublist]))
                    ax.plot((-20, -10, -2, 2, 10, 20), (np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]), np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]), 
                                                        np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]), np.nanmean([item for sublist in data10_2[animal]     for item in sublist]), 
                                                        np.nanmean([item for sublist in data10_10[animal]    for item in sublist]), np.nanmean([item for sublist in data10_20[animal]    for item in sublist])), marker = 'o', markersize = 6, color = rat_markers[animal][0])
                if plot == "%" : 
                    ydata[animal] = ((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev20[animal] for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev10[animal] for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_2[animal]     for item in sublist]) / np.nanmean([item for sublist in data90_2[animal]     for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_10[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_10[animal]    for item in sublist])), 
                                             (np.nanmean([item for sublist in data10_20[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_20[animal]    for item in sublist])))
                    ax.plot((-20, -10, -2, 2, 10, 20), ((np.nanmean([item for sublist in data10_rev20[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev20[animal] for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_rev10[animal] for item in sublist]) / np.nanmean([item for sublist in data90_rev10[animal] for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_rev2[animal]  for item in sublist]) / np.nanmean([item for sublist in data90_rev2[animal]  for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_2[animal]     for item in sublist]) / np.nanmean([item for sublist in data90_2[animal]     for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_10[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_10[animal]    for item in sublist])), 
                                                        (np.nanmean([item for sublist in data10_20[animal]    for item in sublist]) / np.nanmean([item for sublist in data90_20[animal]    for item in sublist]))), marker = 'o', markersize = 6, color = rat_markers[animal][0])
    #global gradient
    gradient, intercept, r_value, p_value, std_err = {}, {}, {}, {}, {}
    for animal in animal_list:    
        #print(animal, xdata[animal], ydata[animal], stats.linregress(list(xdata[animal]), list(ydata[animal])))
        ax.scatter(xdata[animal], ydata[animal], color = rat_markers[animal][0])
        gradient[animal], intercept[animal], r_value[animal], p_value[animal], std_err[animal] = stats.linregress(list(xdata[animal]), list(ydata[animal]))
        ax.plot(np.linspace(np.min(list(xdata[animal])), np.max(list(xdata[animal])), 500), 
                gradient[animal] * np.linspace(np.min(list(xdata[animal])), np.max(list(xdata[animal])), 500) + intercept[animal], color = rat_markers[animal][0], lw = 2)
        #print(animal, gradient[animal])

    #ax.annotate(("r=%.2f, \np=%.2f" %(r_value, p_value)), (np.max(list(xdata)), gradient * np.max(list(xdata))+ intercept))
    return ax, gradient

#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(20, 5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 20)

row1 = slice(0, 5)
col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax30 = fig.add_subplot(gs[row1, col1])
ax30 = across_session_plot2("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 3], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')
ax31 = fig.add_subplot(gs[row1, col2])
ax31 = across_session_plot2("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0, 3], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')
ax32 = fig.add_subplot(gs[row1, col3])
ax32 = across_session_plot2("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0.5, 2], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', 'Distance', 'ratio'], datatype = 'runningtime')
#save_plot_as_png('test8', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:

#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(30, 10)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 20)

row1 = slice(0, 5)
col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax30 = fig.add_subplot(gs[row1, col1])
ax30 = across_session_plot2("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 80], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', 'Distance', 'avg. run speed (s)'], datatype = 'runningspeed')
ax31 = fig.add_subplot(gs[row1, col2])
ax31 = across_session_plot2("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 20, 80], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', 'Distance', 'avg. run speed (s)'], datatype = 'runningspeed')
ax32 = fig.add_subplot(gs[row1, col3])
ax32 = across_session_plot2("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.20], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', 'Distance', 'ratio'], datatype = 'runningspeed')
#save_plot_as_png('test8', dpi = 300, transparent = True, background = 'auto') #change to 300 dpi for clean printfigs

In [ ]:
a90 = across_session_plot2("90%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.8], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')[1];
a10 = across_session_plot2("10%", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.8], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')[1];
aratio = across_session_plot2("%",   animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.20], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', 'Distance', 'ratio'], datatype = 'runningtime')[1];
#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(20, 5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 20)

row1 = slice(0, 5)
col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist60+dist90+dist120
experiment = 'Distance'


ax30 = fig.add_subplot(gs[row1, col1])
x = a90
x = {k: 1/v for k, v in a90.items()}
#plt.scatter(sorted_rats, [x[rat] for rat in sorted_rats], color = [rat_markers[rat][0] for rat in sorted_rats])
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])
ax31 = fig.add_subplot(gs[row1, col2])
x = a10
x = {k: 1/v for k, v in a10.items()}
#plt.scatter(sorted_rats, [x[rat] for rat in sorted_rats], color = [rat_markers[rat][0] for rat in sorted_rats])
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])

ax32 = fig.add_subplot(gs[row1, col3])
x = aratio
x = {k: 1/v for k, v in aratio.items()}
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])
#plt.scatter(sorted_rats, [x[rat] for rat in sorted_rats], color = [rat_markers[rat][0] for rat in sorted_rats])

ax30.set_title("90%rwd")
ax31.set_title("10%rwd")
ax32.set_title("ratio")
ax30.set_xlabel("RatID")
ax30.set_ylabel("vigor = 1/slope (a.u.)");

In [ ]:
a90 = across_session_plot2("90%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.8], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['90%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')[1]
a10 = across_session_plot2("10%", animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.8], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['10%', 'Distance', 'avg. run time (s)'], datatype = 'runningtime')[1]
aratio = across_session_plot2("%",   animalList, session_list, timeRunToLeftBin, timeRunToRightBin, experiment, plot_axes = [55, 125, 0.8, 1.20], ticks = [[60, 90, 120],[]], titles_plot_xaxis_yaxis = ['ratio 10% / 90% rwd', 'Distance', 'ratio'], datatype = 'runningtime')[1]
#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(20, 5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 20)

row1 = slice(0, 5)
col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist60+dist90+dist120
experiment = 'Distance'


ax30 = fig.add_subplot(gs[row1, col1])
x = a90
x = {k: 1/v for k, v in a90.items()}
#plt.scatter(sorted_rats, [x[rat] for rat in sorted_rats], color = [rat_markers[rat][0] for rat in sorted_rats])
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])
ax31 = fig.add_subplot(gs[row1, col2])
x = a10
x = {k: 1/v for k, v in a10.items()}
#plt.scatter(sorted_rats, [x[rat] for rat in sorted_rats], color = [rat_markers[rat][0] for rat in sorted_rats])
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])

ax32 = fig.add_subplot(gs[row1, col3])
x = aratio
x = {k: 1/v for k, v in aratio.items()}
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])
#plt.scatter(sorted_rats, [x[rat] for rat in sorted_rats], color = [rat_markers[rat][0] for rat in sorted_rats])

ax30.set_title("90%rwd")
ax31.set_title("10%rwd")
ax32.set_title("ratio")
ax30.set_xlabel("RatID")
ax30.set_ylabel("vigor = 1/slope (a.u.)");

In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(20, 5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 20)

row1 = slice(0, 5)
col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax30 = fig.add_subplot(gs[row1, col1])
x = a90
x = {k: 1/v for k, v in a90.items()}
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])

ax31 = fig.add_subplot(gs[row1, col2])
y = a10
y = {k: 1/v for k, v in a10.items()}
plt.scatter({k: v for k, v in sorted(y.items(), key=lambda item: item[1])}.keys(), {k: v for k, v in sorted(y.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(y.items(), key=lambda item: item[1])}.keys()])


ax32 = fig.add_subplot(gs[row1, col3])
plt.scatter({k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.values(), {k: v for k, v in sorted(y.items(), key=lambda item: item[1])}.values(), color = [rat_markers[rat][0] for rat in {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}.keys()])
gradient, intercept, r_value, p_value, std_err = stats.linregress(list(x.values()), list(y.values()))
#ax32.plot((80, 140), (80, 140), alpha=0.5, color='k')
ax32.plot(np.linspace(np.min(list(x.values())), np.max(list(x.values())), 500), gradient * np.linspace(np.min(list(x.values())), np.max(list(x.values())), 500) + intercept, color = 'k', lw = 2)
ax32.annotate(("r=%.2f, \np=%.2f" %(r_value, p_value)), (np.max(list(x.values())), gradient * np.max(list(x.values()))+ intercept))


ax30.set_title("90%rwd")
ax31.set_title("10%rwd")
ax32.set_title("correlation 10/90")
ax30.set_xlabel("RatID")
ax30.set_ylabel("vigor = 1/slope")
ax32.set_xlabel("1/slope 90%")
ax32.set_ylabel("1/slope 10%");

In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(20, 5)) #10, 14
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 20)

row1 = slice(0, 5)
col1 = slice(0, 5)
col2 = slice(7, 12)
col3 = slice(15, 20)

session_list = dist60+dist90+dist120
experiment = 'Distance'
w = avgweight
ax30 = fig.add_subplot(gs[row1, col1])
x1 = a90
x1 = {k: 1/v for k, v in a90.items()}
plt.scatter(avgweight.values(), x1.values())
gradient, intercept, r_value, p_value, std_err = stats.linregress(list(w.values()), list(x1.values()))
#ax30.plot((80, 140), (80, 140), alpha=0.5, color='k')
ax30.plot(np.linspace(np.min(list(w.values())), np.max(list(w.values())), 500), gradient * np.linspace(np.min(list(w.values())), np.max(list(w.values())), 500) + intercept, color = 'k', lw = 2)
ax30.annotate(("r=%.2f, \np=%.2f" %(r_value, p_value)), (np.max(list(w.values())), gradient * np.max(list(w.values()))+ intercept))

ax31 = fig.add_subplot(gs[row1, col2])
w = avgweight
x2 = a10
x2 = {k: 1/v for k, v in a10.items()}
plt.scatter(avgweight.values(), x2.values())
gradient, intercept, r_value, p_value, std_err = stats.linregress(list(w.values()), list(x2.values()))
#ax31.plot((80, 140), (80, 140), alpha=0.5, color='k')
ax31.plot(np.linspace(np.min(list(w.values())), np.max(list(w.values())), 500), gradient * np.linspace(np.min(list(w.values())), np.max(list(w.values())), 500) + intercept, color = 'k', lw = 2)
ax31.annotate(("r=%.2f, \np=%.2f" %(r_value, p_value)), (np.max(list(w.values())), gradient * np.max(list(w.values()))+ intercept))

ax30.set_title("90%rwd")
ax31.set_title("10%rwd")
ax30.set_ylim(20, 120)
ax31.set_ylim(20, 120)
#ax32.set_title("correlation 10/90")
ax30.set_xlabel("Rat weight")
ax30.set_ylabel("vigor = 1/slope")
#ax32.set_xlabel("1/slope 90%")
#ax32.set_ylabel("1/slope 10%");

In [ ]:
def separate_data2(animal_list, session_list, dataLeft, dataRight, experiment, datatype, bin):

    if experiment == 'Distance':
        if bin == True:
            data60, data90, data120 = ({} for _ in range(3))
            for animal in animal_list:
                data60[animal], data90[animal], data120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(3))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillDist'] == 60:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data60[animal][i] = np.append(data60[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data60[animal][i] = data60[animal][i] + [[[index, d ]for index, d in enumerate(dataRight[animal, session][i])] +  [[index, d] for index, d in enumerate(dataLeft[animal, session][i])]]
                            #print(animal, i, data60[animal][i])

                    if params[animal, session]['treadmillDist'] == 90:                    
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data90[animal][i] = np.append(data90[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data90[animal][i] = np.append(data90[animal][i], [list(enumerate(dataRight[animal, session][i] ))+ list(enumerate(dataLeft[animal, session][i]))])

                    if params[animal, session]['treadmillDist'] == 120:                     
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data120[animal][i] = np.append(data120[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data120[animal][i] = np.append(data120[animal][i], [list(enumerate(dataRight[animal, session][i])) + list(enumerate(dataLeft[animal, session][i]))])
            return data60, data90, data120

    if experiment == 'TM_ON':
        if bin == False:
            data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20 = ({} for _ in range(12))
            for animal in animal_list:
                data90_rev20[animal], data90_rev10[animal], data90_rev2[animal], data90_2[animal], data90_10[animal], data90_20[animal], data10_rev20[animal], data10_rev10[animal], data10_rev2[animal], data10_2[animal], data10_10[animal], data10_20[animal] = ([] for _ in range(12))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:
                        if datatype == 'nb_runs': 
                            data90_rev20[animal] = np.append(data90_rev20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev20[animal] = np.append(data10_rev20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev20[animal] = np.append(data90_rev20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev20[animal] = np.append(data10_rev20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
                        if datatype == 'nb_runs': 
                            data90_rev10[animal] = np.append(data90_rev10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev10[animal] = np.append(data10_rev10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev10[animal] = np.append(data90_rev10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev10[animal] = np.append(data10_rev10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:
                        if datatype == 'nb_runs': 
                            data90_rev2[animal] = np.append(data90_rev2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_rev2[animal] = np.append(data10_rev2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_rev2[animal] = np.append(data90_rev2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_rev2[animal] = np.append(data10_rev2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:
                        if datatype == 'nb_runs':
                            data90_2[animal] = np.append(data90_2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_2[animal] = np.append(data10_2[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_2[animal] = np.append(data90_2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_2[animal] = np.append(data10_2[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:
                        if datatype == 'nb_runs':
                            data90_10[animal] = np.append(data90_10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_10[animal] = np.append(data10_10[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_10[animal] = np.append(data90_10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_10[animal] = np.append(data10_10[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                    if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:
                        if datatype == 'nb_runs': 
                            data90_20[animal] = np.append(data90_20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                            data10_20[animal] = np.append(data10_20[animal], [i/(int((params[animal, session]['blocks'][block][1]-params[animal, session]['blocks'][block][0])/60)) for block, i in enumerate(poolByReward([dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock']))])
                        else:
                            data90_20[animal] = np.append(data90_20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_ON"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
                            data10_20[animal] = np.append(data10_20[animal], [i for i in poolByReward([dataRight[animal, session], dataLeft[animal, session]], params[animal, session]["rewardP_OFF"][0], params[animal, session]['blocks'], params[animal, session]['rewardProbaBlock'])])
            return data90_rev20, data90_rev10, data90_rev2, data90_2, data90_10, data90_20, data10_rev20, data10_rev10, data10_rev2, data10_2, data10_10, data10_20

        if bin == True:
            datarev20, datarev10, datarev2, data2, data10, data20 = ({} for _ in range(6))
            for animal in animal_list:
                datarev20[animal], datarev10[animal], datarev2[animal], data2[animal], data10[animal], data20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
                for session in sorted(matchsession(animal, session_list)):
                    if params[animal, session]['treadmillSpeed'] == [-20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20, -20]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev20[animal][i] = np.append(datarev20[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): datarev20[animal][i] = np.append(datarev20[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev10[animal][i] = np.append(datarev10[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): datarev10[animal][i] = np.append(datarev10[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [- 2,  -2,  -2,  -2,  -2,  -2, - 2,  -2,  -2,  -2,  -2,  -2]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): datarev2[animal][i] = np.append(datarev2[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): datarev2[animal][i] = np.append(datarev2[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data2[animal][i] = np.append(data2[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data2[animal][i] = np.append(data2[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [ 10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data10[animal][i] = np.append(data10[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data10[animal][i] = np.append(data10[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
                    if params[animal, session]['treadmillSpeed'] == [ 20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20,  20]:
                        if datatype == 'nb_runs': 
                            for i in range (0, len(params[animal, session]['blocks'])): data20[animal][i] = np.append(data20[animal][i], [b/(int((params[animal, session]['blocks'][i][1]-params[animal, session]['blocks'][i][0])/60)) for b in dataLeft[animal, session].values()][i])
                        else:
                            for i in range (0, len(params[animal, session]['blocks'])): data20[animal][i] = np.append(data20[animal][i], [dataRight[animal, session][i] + dataLeft[animal, session][i]])
            return datarev20, datarev10, datarev2, data2, data10, data20


def across_Binsession_plot3(plot, animal_list, session_list, dataLeft, dataRight, experiment, plot_axes, ticks, titles_plot_xaxis_yaxis, datatype): 
    ax = plt.gca()
    ax.set_title( titles_plot_xaxis_yaxis[0], fontsize = 16)
    ax.set_xlabel(titles_plot_xaxis_yaxis[1], fontsize = 16)
    ax.set_ylabel(titles_plot_xaxis_yaxis[2], fontsize = 16)
    ax.set_xlim(plot_axes[0], plot_axes[1])
    ax.set_ylim(plot_axes[2], plot_axes[3])
    if ticks[0] != []: ax.set_xticks(ticks[0])
    if ticks[1] != []: ax.set_yticks(ticks[1])
    ax.tick_params(width=1.5, labelsize = 12)
    ax.tick_params(axis = 'x', rotation = 45) 
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.yaxis.label.set_color('dodgerblue' if datatype == 'avgrunspeed' else 
                             'dodgerblue' if datatype == 'runningtime' else 
                             'orange' if datatype == 'idletime' else 
                             'red'if datatype == 'maxspeed' else 'k')

    if experiment == 'Distance':
        data60, data90, data120 = separate_data2(animal_list, session_list, dataLeft, dataRight, experiment, datatype, True)
        for animal in animal_list:
            if plot == "60" : 
                for x in data60[animal].keys():
                    #print([x+1 + run[0]/10 for sess in data60[animal][x] for run in sess[0:10]], [run[1] for sess in data60[animal][x] for run in sess[0:10]])
                    ax.scatter([x+1 + run[0]/10 for sess in data60[animal][x] for run in sess[0:10]], [run[1] for sess in data60[animal][x] for run in sess[0:10]], marker = 'o', s = 1, color = rat_markers[animal][0])
                
                
            if plot == "90" : ax.plot([x+1 for x in data90[animal].keys()], [np.mean(i) for i in data90[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "120": ax.plot([x+1 for x in data120[animal].keys()],[np.mean(i) for i in data120[animal].values()],marker = 'o', markersize = 6, color = rat_markers[animal][0])  
        
    if experiment == 'TM_ON':
        datarev20, datarev10, datarev2, data2, data10, data20 = separate_data2(animal_list, session_list, dataLeft, dataRight, experiment, datatype, True)
        for animal in animal_list:      
            if plot == "-20" : ax.plot([x+1 for x in datarev20[animal].keys()], [np.mean(i) for i in datarev20[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "-10" : ax.plot([x+1 for x in datarev10[animal].keys()], [np.mean(i) for i in datarev10[animal].values()], marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "-2"  : ax.plot([x+1 for x in datarev2[animal].keys()],  [np.mean(i) for i in datarev2[animal].values()],  marker = 'o', markersize = 6, color = rat_markers[animal][0])  
            if plot == "2"   : ax.plot([x+1 for x in data2[animal].keys()],     [np.mean(i) for i in data2[animal].values()],     marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "10"  : ax.plot([x+1 for x in data10[animal].keys()],    [np.mean(i) for i in data10[animal].values()],    marker = 'o', markersize = 6, color = rat_markers[animal][0])
            if plot == "20"  : ax.plot([x+1 for x in data20[animal].keys()],    [np.mean(i) for i in data20[animal].values()],    marker = 'o', markersize = 6, color = rat_markers[animal][0])  
    #print(data120[animal])
    return ax

#Distance plots
fig = plt.figure(constrained_layout = False, figsize=(10, 10))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(5, 5)

row1 = slice(0, 5)
"""row2 = slice(7, 12)
row3 = slice(14, 19)
row4 = slice(21, 26)"""

col1 = slice(0, 5)
"""col2 = slice(7, 12)
col3 = slice(14, 19)"""

session_list = dist60+dist90+dist120
experiment = 'Distance'

ax20 = fig.add_subplot(gs[row1, col1])
ax20 = across_Binsession_plot3("60",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 0, 100], ticks = [[],[]], titles_plot_xaxis_yaxis = ['60',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
#ax21 = fig.add_subplot(gs[row3, col2])
#ax21 = across_Binsession_plot3("90",  animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['90',  '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')
#ax22 = fig.add_subplot(gs[row3, col3])
#ax22 = across_Binsession_plot3("120", animalList, session_list, speedRunToLeftBin, speedRunToRightBin, experiment, plot_axes = [0, 13, 20, 100], ticks = [[*range(1, 13, 1)],[]], titles_plot_xaxis_yaxis = ['120', '', 'avg. run speed (cm/s)'], datatype = 'avgrunspeed')


In [ ]:
fig = plt.figure(constrained_layout = False, figsize=(40, 30))
fig.suptitle("", y=0.9, fontsize = 24)
gs = fig.add_gridspec(19, 37) #row ,col

row1 = slice(0, 5)
row2 = slice(7, 12)
row3 = slice(14, 19)
col1 = slice(0, 5)
col2 = slice(6, 11)
col3 = slice(12, 17)

col4 = slice(20, 25)
col5 = slice(26, 31)
col6 = slice(32, 37)


animal = 'RatM00'
sessionID = 0
session = matchsession(animal, dist60)[sessionID]
experiment = 'Distance'
block0=0
blockN=1

ax00 = fig.add_subplot(gs[row1, col1])
ax00 = plt.plot([i for i in range(0, len(timeRunToRightBin[animal, session][block0]))], timeRunToRightBin[animal, session][block0], label="to right")
ax00 = plt.plot([i for i in range(0, len(timeRunToLeftBin[animal, session][block0]))], timeRunToLeftBin[animal, session][block0], label = "to left")
plt.ylabel("TR")
plt.title("1st session 60cm block %s" %block0)
plt.legend()
ax10 = fig.add_subplot(gs[row2, col1])
ax10 = plt.plot([i for i in range(0, len(timeStayInRightBin[animal, session][block0]))], timeStayInRightBin[animal, session][block0], label="in right")
ax10 = plt.plot([i for i in range(0, len(timeStayInLeftBin[animal, session][block0]))], timeStayInLeftBin[animal, session][block0], label = "in left")
plt.ylabel("TW")
plt.title("1st session 60cm block %s" %block0)
plt.legend()

ax30 = fig.add_subplot(gs[row1, col4])
ax30 = plt.plot([i for i in range(0, len(timeRunToRightBin[animal, session][blockN]))], timeRunToRightBin[animal, session][blockN], label="to right")
ax30 = plt.plot([i for i in range(0, len(timeRunToLeftBin[animal, session][blockN]))], timeRunToLeftBin[animal, session][blockN], label = "to left")
plt.title("1st session 60cm block %s" %blockN)
plt.legend()
ax40 = fig.add_subplot(gs[row2, col4])
ax40 = plt.plot([i for i in range(0, len(timeStayInRightBin[animal, session][blockN]))], timeStayInRightBin[animal, session][blockN], label="in right")
ax40 = plt.plot([i for i in range(0, len(timeStayInLeftBin[animal, session][blockN]))], timeStayInLeftBin[animal, session][blockN], label = "in left")
plt.title("1st session 60cm block %s" %blockN)
plt.legend()



session = matchsession(animal, dist90)[sessionID]
experiment = 'Distance'

ax01 = fig.add_subplot(gs[row1, col2])
ax01 = plt.plot([i for i in range(0, len(timeRunToRightBin[animal, session][block0]))], timeRunToRightBin[animal, session][block0], label="to right")
ax01 = plt.plot([i for i in range(0, len(timeRunToLeftBin[animal, session][block0]))], timeRunToLeftBin[animal, session][block0], label = "to left")
plt.title("1st session 90cm block %s" %block0)
plt.legend()
ax11 = fig.add_subplot(gs[row2, col2])
ax11 = plt.plot([i for i in range(0, len(timeStayInRightBin[animal, session][block0]))], timeStayInRightBin[animal, session][block0], label="in right")
ax11 = plt.plot([i for i in range(0, len(timeStayInLeftBin[animal, session][block0]))], timeStayInLeftBin[animal, session][block0], label = "in left")
plt.title("1st session 90cm block %s" %block0)
plt.legend()

ax31 = fig.add_subplot(gs[row1, col5])
ax31 = plt.plot([i for i in range(0, len(timeRunToRightBin[animal, session][blockN]))], timeRunToRightBin[animal, session][blockN], label="to right")
ax31 = plt.plot([i for i in range(0, len(timeRunToLeftBin[animal, session][blockN]))], timeRunToLeftBin[animal, session][blockN], label = "to left")
plt.title("1st session 90cm block %s" %blockN)
plt.legend()
ax41 = fig.add_subplot(gs[row2, col5])
ax41 = plt.plot([i for i in range(0, len(timeStayInRightBin[animal, session][blockN]))], timeStayInRightBin[animal, session][blockN], label="in right")
ax41 = plt.plot([i for i in range(0, len(timeStayInLeftBin[animal, session][blockN]))], timeStayInLeftBin[animal, session][blockN], label = "in left")
plt.title("1st session 90cm block %s" %blockN)
plt.legend()

session = matchsession(animal, dist120)[sessionID]
experiment = 'Distance'

ax02 = fig.add_subplot(gs[row1, col3])
ax02 = plt.plot([i for i in range(0, len(timeRunToRightBin[animal, session][block0]))], timeRunToRightBin[animal, session][block0], label="to right")
ax02 = plt.plot([i for i in range(0, len(timeRunToLeftBin[animal, session][block0]))], timeRunToLeftBin[animal, session][block0], label = "to left")
plt.title("1st session 120cm block %s" %block0)
plt.legend()
ax12 = fig.add_subplot(gs[row2, col3])
ax12 = plt.plot([i for i in range(0, len(timeStayInRightBin[animal, session][block0]))], timeStayInRightBin[animal, session][block0], label="in right")
ax12 = plt.plot([i for i in range(0, len(timeStayInLeftBin[animal, session][block0]))], timeStayInLeftBin[animal, session][block0], label = "in left")
plt.title("1st session 120cm block %s" %block0)
plt.legend()

ax32 = fig.add_subplot(gs[row1, col6])
ax32 = plt.plot([i for i in range(0, len(timeRunToRightBin[animal, session][blockN]))], timeRunToRightBin[animal, session][blockN], label="to right")
ax32 = plt.plot([i for i in range(0, len(timeRunToLeftBin[animal, session][blockN]))], timeRunToLeftBin[animal, session][blockN], label = "to left")
plt.title("1st session 120cm block %s" %blockN)
plt.legend()
ax42 = fig.add_subplot(gs[row2, col6])
ax42 = plt.plot([i for i in range(0, len(timeStayInRightBin[animal, session][blockN]))], timeStayInRightBin[animal, session][blockN], label="in right")
ax42 = plt.plot([i for i in range(0, len(timeStayInLeftBin[animal, session][blockN]))], timeStayInLeftBin[animal, session][blockN], label = "in left")
plt.title("1st session 120cm block %s" %blockN)
plt.legend()



In [ ]:
endTimeNotebook = datetime.datetime.now()
timeExecutionNotebook = endTimeNotebook-startTimeNotebook
print(timeExecutionNotebook)
<

In [ ]:
def plot_median_trajectory3(posdataRight, timedataRight, animal, alpha, color, label):
    median_plot = plt.gca()
    try : maxL = np.nanmax([len(j) for j in timedataRight])
    except TypeError: maxL = 1
    except ValueError: maxL = 1
    avgPos = np.ones((len(timedataRight), maxL))*np.nan
    for i, p in enumerate(posdataRight): avgPos[i, 0:len(p)] = p
    nanSum = np.sum(np.isnan(avgPos), axis = 0)
    try: end_medTraj = np.where(nanSum > .7 * avgPos.shape[0])[0][0]
    except IndexError: end_medTraj = avgPos.shape[1]
    ci = (np.nanstd(avgPos, axis = 0)[:end_medTraj])/(np.sqrt(np.count_nonzero(~np.isnan(avgPos), axis = 0)[:end_medTraj]))
    plt.plot(np.linspace(0, end_medTraj*0.04, end_medTraj), np.nanmedian(avgPos, axis = 0)[:end_medTraj], color =color, alpha = alpha, label = label if label != "" else None)#color = 'g' if block == 90 else 'r')
    plt.fill_between(np.linspace(0, end_medTraj*0.04, end_medTraj), np.nanmedian(avgPos, axis = 0)[:end_medTraj]-ci, np.nanmedian(avgPos, axis = 0)[:end_medTraj]+ci, color =color, zorder = 3, alpha = alpha/4) #,'g' if block == 90 else 'r'
    return median_plot



median60 = {}
median90 = {}
median120= {}
tmedian60 = {}
tmedian90 = {}
tmedian120= {}

speed60 = {}
speed90 = {}
speed120= {}

for animal in animalList:
    median60[animal], median90[animal], median120[animal], tmedian60[animal], tmedian90[animal], tmedian120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    speed60[animal], speed90[animal], speed120[animal] = ({bin : [] for bin in range(0, (12))} for i in range(3))
    for session in sorted(matchsession(animal, dist60)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median60[animal][i] = np.append(median60[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian60[animal][i] = np.append(tmedian60[animal][i], trackTimeToRightBin[animal, session][i])       
                speed60[animal][i] = np.append(speed60[animal][i], instantSpeedRightBin[animal, session][i])

    for session in sorted(matchsession(animal, dist90)):   
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median90[animal][i] = np.append(median90[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian90[animal][i] = np.append(tmedian90[animal][i], trackTimeToRightBin[animal, session][i])
                speed90[animal][i] = np.append(speed90[animal][i], instantSpeedRightBin[animal, session][i])
    for session in sorted(matchsession(animal, dist120)):
        for i in range(0, 12):
            if len(XtrackRunToRightBin[animal, session][i]) >= 3:
                median120[animal][i]= np.append(median120[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian120[animal][i]= np.append(tmedian120[animal][i], trackTimeToRightBin[animal, session][i])
                speed120[animal][i]= np.append(speed120[animal][i], instantSpeedRightBin[animal, session][i])


med60, med90, med120 = {}, {}, {}
sp60, sp90, sp120 = {}, {}, {}
tmed60, tmed90, tmed120 = {}, {}, {}
mediantrack60, mediantrack90, mediantrack120 = {},{},{}
medianspeed60, medianspeed90, medianspeed120 = {},{},{}
fig = plt.figure(constrained_layout = False, figsize=(15, 20))
gs = fig.add_gridspec(len(animalList), 4)
for index, animal in enumerate(animalList):
    med60[animal], med90[animal], med120[animal], tmed60[animal], tmed90[animal], tmed120[animal] = ([]for i in range(6))
    sp60[animal], sp90[animal], sp120[animal] = ([]for i in range(3))
    mediantrack60[animal], mediantrack90[animal], mediantrack120[animal] = ([]for i in range(3))
    medianspeed60[animal], medianspeed90[animal], medianspeed120[animal] = ([]for i in range(3))
    gs00 = gs[index, 0:12].subgridspec(1,4)
    for i in range(0, 12):
        med60[animal] = np.append(med60[animal], median60[animal][i])
        sp60[animal] = np.append(sp60[animal], speed60[animal][i])
        tmed60[animal] = np.append(tmed60[animal], tmedian60[animal][i])
        med90[animal] = np.append(med90[animal], median90[animal][i])
        sp90[animal] = np.append(sp90[animal], speed90[animal][i])
        tmed90[animal] = np.append(tmed90[animal], tmedian90[animal][i])
        med120[animal] = np.append(med120[animal], median120[animal][i])
        sp120[animal] = np.append(sp120[animal], speed120[animal][i])
        tmed120[animal] = np.append(tmed120[animal], tmedian120[animal][i])


    axs= fig.add_subplot(gs00[0, 1])
    axs.set_xlim(0, 2)
    axs.set_ylim(0, 130)
    axs = plot_median_trajectory3(med60[animal], tmed60[animal], animal, 1, 'g', label = "60cm")    
    axs = plot_median_trajectory3(med90[animal], tmed90[animal], animal, 1, 'k', label = "90cm")
    axs = plot_median_trajectory3(med120[animal], tmed120[animal], animal, 1, 'r', label = "120cm")    
    mediantrack60[animal] = compute_median_trajectory(med60[animal], tmed60[animal])  
    mediantrack90[animal] = compute_median_trajectory(med90[animal], tmed90[animal])
    mediantrack120[animal]= compute_median_trajectory(med120[animal], tmed120[animal])
    if index == 0: 
        axs.set_title("Pos")   
        axs.legend()
    
    axs.set_ylabel(animal)

    axs= fig.add_subplot(gs00[0, 2])
    axs.set_xlim(0, 2)
    axs.set_ylim(0, 130)
    axs = plot_median_trajectory3(sp60[animal], tmed60[animal], animal, 1, 'g', label = "")    
    axs = plot_median_trajectory3(sp90[animal], tmed90[animal], animal, 1, 'k', label = "")
    axs = plot_median_trajectory3(sp120[animal], tmed120[animal], animal, 1, 'r', label = "")    
    medianspeed60[animal] = compute_median_trajectory(sp60[animal], tmed60[animal])  
    medianspeed90[animal] = compute_median_trajectory(sp90[animal], tmed90[animal])
    medianspeed120[animal]= compute_median_trajectory(sp120[animal], tmed120[animal])
    if index == 0: axs.set_title("Speed")   


    axs= fig.add_subplot(gs00[0, 3])
    axs.set_xlim(0, 2)
    axs.set_ylim(-15, 15)
    axs = plot_median_trajectory3([np.diff(i) for i in sp60[animal]], tmed60[animal], animal, 1, 'g', label = "")    
    axs = plot_median_trajectory3([np.diff(i) for i in sp90[animal]], tmed90[animal], animal, 1, 'k', label = "")
    axs = plot_median_trajectory3([np.diff(i) for i in sp120[animal]],tmed120[animal], animal, 1, 'r', label = "")    
    if index == 0: axs.set_title("Acc")   
    plt.plot([0, 2], [0, 0], ":", color='k')


plt.show()
#print(med60[animal],med90[animal], med120[animal])

    


In [ ]:

medianrev20, medianrev10, medianrev2, median2, median10, median20, tmedianrev20, tmedianrev10, tmedianrev2, tmedian2, tmedian10, tmedian20 = ({} for i in range(12))
speedrev20, speedrev10, speedrev2, speed2, speed10, speed20 = ({} for i in range(6))
for animal in animalList:
    medianrev20[animal], medianrev10[animal], medianrev2[animal], median2[animal], median10[animal], median20[animal], tmedianrev20[animal], tmedianrev10[animal], tmedianrev2[animal], tmedian2[animal], tmedian10[animal], tmedian20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(12))
    speedrev20[animal], speedrev10[animal], speedrev2[animal], speed2[animal], speed10[animal], speed20[animal] = ({bin : [] for bin in range(0, (12))} for i in range(6))
    for session in sorted(matchsession(animal, TMrev20)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev20[animal][i] = np.append(medianrev20[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev20[animal][i] = np.append(tmedianrev20[animal][i], trackTimeToRightBin[animal, session][i])
                speedrev20[animal][i] = np.append(speedrev20[animal][i], instantSpeedRightBin[animal, session][i])           
    for session in sorted(matchsession(animal, TMrev10)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev10[animal][i] = np.append(medianrev10[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev10[animal][i] = np.append(tmedianrev10[animal][i], trackTimeToRightBin[animal, session][i])
                speedrev10[animal][i] = np.append(speedrev10[animal][i], instantSpeedRightBin[animal, session][i])  
    for session in sorted(matchsession(animal, TMrev2)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                medianrev2[animal][i]= np.append(medianrev2[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedianrev2[animal][i]= np.append(tmedianrev2[animal][i], trackTimeToRightBin[animal, session][i])
                speedrev2[animal][i] = np.append(speedrev2[animal][i], instantSpeedRightBin[animal, session][i])  

    for session in sorted(matchsession(animal, TM2)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 3:
                median2[animal][i] = np.append(median2[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian2[animal][i] = np.append(tmedian2[animal][i], trackTimeToRightBin[animal, session][i])
                speed2[animal][i] = np.append(speed2[animal][i], instantSpeedRightBin[animal, session][i])             
    for session in sorted(matchsession(animal, TM10)):   
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 5:
                median10[animal][i] = np.append(median10[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian10[animal][i] = np.append(tmedian10[animal][i], trackTimeToRightBin[animal, session][i])
                speed10[animal][i] = np.append(speed10[animal][i], instantSpeedRightBin[animal, session][i])  
    for session in sorted(matchsession(animal, TM20)):
        for i in range(0, 12):
            if len(instantSpeedRightBin[animal, session][i]) >= 5:
                median20[animal][i]= np.append(median20[animal][i], XtrackRunToRightBin[animal, session][i])
                tmedian20[animal][i]= np.append(tmedian20[animal][i], trackTimeToRightBin[animal, session][i])
                speed20[animal][i] = np.append(speed20[animal][i], instantSpeedRightBin[animal, session][i])  




medrev20, medrev10, medrev2, med2, med10, med20, tmedrev20, tmedrev10, tmedrev2, tmed2, tmed10, tmed20 = ({} for i in range(12))
sprev20, sprev10, sprev2, sp2, sp10, sp20 = ({} for i in range(6))


mediantrackrev20, mediantrackrev10, mediantrackrev2, mediantrack2, mediantrack10, mediantrack20 = {},{},{},{},{},{}
medianspeedrev20, medianspeedrev10, medianspeedrev2, medianspeed2, medianspeed10, medianspeed20 = {},{},{},{},{},{}

fig = plt.figure(constrained_layout = False, figsize=(15, 20))
gs = fig.add_gridspec(len(animalList), 12)
for index, animal in enumerate(animalList):
    medrev20[animal], medrev10[animal], medrev2[animal], med2[animal], med10[animal], med20[animal], tmedrev20[animal], tmedrev10[animal], tmedrev2[animal], tmed2[animal], tmed10[animal], tmed20[animal] = ([] for i in range(12))
    sprev20[animal], sprev10[animal], sprev2[animal], sp2[animal], sp10[animal], sp20[animal] = ([] for i in range(6))
    mediantrackrev20[animal], mediantrackrev10[animal], mediantrackrev2[animal], mediantrack2[animal], mediantrack10[animal], mediantrack20[animal] = ([]for i in range(6))
    medianspeedrev20[animal], medianspeedrev10[animal], medianspeedrev2[animal], medianspeed2[animal], medianspeed10[animal], medianspeed20[animal] = ([]for i in range(6))

    gs00 = gs[index, 0:12].subgridspec(1,4)
    for i in range(0, 12):
        medrev20[animal] = np.append(medrev20[animal], medianrev20[animal][i])
        sprev20[animal] = np.append(sprev20[animal], speedrev20[animal][i])
        tmedrev20[animal] = np.append(tmedrev20[animal], tmedianrev20[animal][i])

        medrev10[animal] = np.append(medrev10[animal], medianrev10[animal][i])
        sprev10[animal] = np.append(sprev10[animal], speedrev10[animal][i])
        tmedrev10[animal] = np.append(tmedrev10[animal], tmedianrev10[animal][i])

        medrev2[animal] = np.append(medrev2[animal], medianrev2[animal][i])
        sprev2[animal] = np.append(sprev2[animal], speedrev2[animal][i])
        tmedrev2[animal] = np.append(tmedrev2[animal], tmedianrev2[animal][i])

        med20[animal] = np.append(med20[animal], median20[animal][i])
        sp20[animal] = np.append(sp20[animal], speed20[animal][i])
        tmed20[animal] = np.append(tmed20[animal], tmedian20[animal][i])

        med10[animal] = np.append(med10[animal], median10[animal][i])
        sp10[animal] = np.append(sp10[animal], speed10[animal][i])
        tmed10[animal] = np.append(tmed10[animal], tmedian10[animal][i])

        med2[animal] = np.append(med2[animal], median2[animal][i])
        sp2[animal] = np.append(sp2[animal], speed2[animal][i])
        tmed2[animal] = np.append(tmed2[animal], tmedian2[animal][i])


    axs= fig.add_subplot(gs00[0, 1])
    axs.set_xlim(0, 2)
    axs.set_ylim(0, 130)
    axs = plot_median_trajectory3(medrev20[animal], tmedrev20[animal], animal, 1, 'r', label = "-20")    
    axs = plot_median_trajectory3(medrev10[animal], tmedrev10[animal], animal, 0.2, 'r', label = "-10")
    axs = plot_median_trajectory3(medrev2[animal],  tmedrev2[animal], animal, 0.2, 'k', label = "-2")    
    axs = plot_median_trajectory3(med2[animal],  tmed2[animal], animal, 0.2, 'k', label = "2")    
    axs = plot_median_trajectory3(med10[animal], tmed10[animal], animal, 0.2, 'g', label = "10")
    axs = plot_median_trajectory3(med20[animal], tmed20[animal], animal, 1, 'g', label = "20") 
    axs.set_ylabel(animal)
    if index == 0: 
        axs.set_title("Position")   
        axs.legend()

    mediantrackrev20[animal] = compute_median_trajectory(medrev20[animal], tmedrev20[animal])  
    mediantrackrev10[animal] = compute_median_trajectory(medrev10[animal], tmedrev10[animal])
    mediantrackrev2[animal]= compute_median_trajectory(medrev2[animal],  tmedrev2[animal])
    mediantrack2[animal] = compute_median_trajectory(med2[animal],  tmed2[animal])  
    mediantrack10[animal] = compute_median_trajectory(med10[animal], tmed10[animal])
    mediantrack20[animal]= compute_median_trajectory(med20[animal], tmed20[animal])

    axs= fig.add_subplot(gs00[0, 2])
    axs.set_xlim(0, 2)
    axs.set_ylim(0, 130)
    axs = plot_median_trajectory3(sprev20[animal], tmedrev20[animal], animal, 1, 'r', label = "")    
    axs = plot_median_trajectory3(sprev10[animal], tmedrev10[animal], animal, 0.2, 'r', label = "")
    axs = plot_median_trajectory3(sprev2[animal],  tmedrev2[animal], animal, 0.2, 'k', label = "")    
    axs = plot_median_trajectory3(sp2[animal],  tmed2[animal], animal, 0.2, 'k', label = "")    
    axs = plot_median_trajectory3(sp10[animal], tmed10[animal], animal, 0.2, 'g', label = "")
    axs = plot_median_trajectory3(sp20[animal], tmed20[animal], animal, 1, 'g', label = "")    
    if index == 0: axs.set_title("Speed")   
    
    medianspeedrev20[animal] = compute_median_trajectory(sprev20[animal], tmedrev20[animal])  
    medianspeedrev10[animal] = compute_median_trajectory(sprev10[animal], tmedrev10[animal])
    medianspeedrev2[animal]= compute_median_trajectory(sprev2[animal],  tmedrev2[animal])
    medianspeed2[animal] = compute_median_trajectory(sp2[animal],  tmed2[animal])  
    medianspeed10[animal] = compute_median_trajectory(sp10[animal], tmed10[animal])
    medianspeed20[animal]= compute_median_trajectory(sp20[animal], tmed20[animal])


    axs= fig.add_subplot(gs00[0, 3])
    axs.set_xlim(0, 2)
    axs.set_ylim(-15, 15)
    axs = plot_median_trajectory3([np.diff(i) for i in sprev20[animal]], tmedrev20[animal], animal, 1, 'r', label = "")    
    axs = plot_median_trajectory3([np.diff(i) for i in sprev10[animal]], tmedrev10[animal], animal, 0.2, 'r', label = "")
    axs = plot_median_trajectory3([np.diff(i) for i in sprev2[animal]],  tmedrev2[animal], animal, 0.2, 'k', label = "")    
    axs = plot_median_trajectory3([np.diff(i) for i in sp2[animal]],  tmed2[animal], animal, 0.2, 'k', label = "")    
    axs = plot_median_trajectory3([np.diff(i) for i in sp10[animal]], tmed10[animal], animal, 0.2, 'g', label = "")
    axs = plot_median_trajectory3([np.diff(i) for i in sp20[animal]], tmed20[animal], animal, 1, 'g', label = "")    
    if index == 0: axs.set_title("Acc")   
    plt.plot([0, 2], [0, 0], ":", color='k')

plt.show()
#print(med60[animal],med90[animal], med120[animal])

    


In [ ]:
#plot rat speed along run
def plot_speed(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    for i, j in zip(posdataRight, timedataRight):
        #print(i, j)
        time = np.subtract(j, j[0])
        iabs =  [abs(ele) for ele in i]
        plt.plot(np.subtract(j, j[0]), iabs, color='g' , linewidth=0.3)
        if len(np.where(i == max(i))[0]) == 1:
            maxspeed = max(iabs)
            maxspeedtime = np.where(iabs == maxspeed)[0]
            plt.scatter(time[maxspeedtime], maxspeed, color ='darkgreen', s=20)
        else: print(max(i))
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    xline1 = [bounds[0], bounds[0]]
    xline2 = [bounds[1], bounds[1]]
    yline = [0,20]
    plt.plot(yline, xline1, ":", color='k')
    plt.plot(yline, xline2, ":", color='k')

    return ax

def plot_trackstest(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    n = len(posdataRight)
    colors = plt.cm.RdYlGn_r(np.linspace(0,1,n)) #green early, red late
    for index, (i, j) in enumerate(zip(posdataRight, timedataRight)):
        tracks = ax.plot(np.subtract(j, j[0]), i, color='g' , linewidth=0.3, label = index if index == 0 or index == n-1 else "")#, label="Good Item" if i == posdataRight[0] else "")
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
    xline1 = [bounds[0], bounds[0]]
    xline2 = [bounds[1], bounds[1]]
    yline = [0,20]
    plt.plot(yline, xline1, ":", color='k')
    plt.plot(yline, xline2, ":", color='k')
    return ax    


fig = plt.figure(constrained_layout = False, figsize=(5, 15))
ax1 = plt.subplot(311)
ax1 = plot_trackstest(animal, session, XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0, 0],xylim = [-0.1, 2, 0, 120], xyLabels=["Time (s)","X Position (cm)", 14], title=["Tracking run to Right",  16], linewidth = [1.5]) 
ax1 = plot_median_trajectory2(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], animal, 1, 'k')
aaa = compute_median_trajectory(XtrackRunToRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])

ax2 = plt.subplot(312)
ax2 = plot_speed(animal, session, instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0,0], xylim = [-0.1, 2, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["To Right" + "µL/drop", 12], linewidth = [1.5])
ax2 = plot_median_trajectory2(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], animal, 1, 'k')
bbb = compute_median_trajectory(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])

#bbb = compute_median_trajectory(instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0])

In [ ]:
print(aaa, bbb)

In [ ]:
for animal in animalList:
    mediantrack120[animal] = aaa
    medianspeed120[animal] = bbb

In [ ]:
mediantrack60[animal]
mediantrack90[animal]
mediantrack120[animal]

In [ ]:
from casadi import *
from casadi.tools import *
import numpy as np
from scipy import optimize
import pickle
import os
import csv

In [ ]:
def spatialpenalty(x, k, mu, amp):
    return(amp/(1+np.exp(1+k*(x-mu)))) #why "1"+k(x-mu) shift

def speedpenalty(x, k, mu, amp):
    return(amp/(1+np.exp(1+k*(x-mu))))

def optimal_trajectory(cost, a, b, xb, xi, xf, xdoti, xdotf, tf, dist, vtapis, tau_res, mass, ampxpenalty, kxpenalty, nk):
    """Compute the optimal trajectory
    Args:
        cost: string, cost type (defines the effort and spacial costs)  
        a: scalar, effort sensitivity 
        b: scalar, spacial sensitivity
        xb: scalar, beam location
        xi: scalar, initial position
        xf: scalar, final position
        tf: scalar, final time (entrance time)
        vtapis: scalar, treadmill speed
        tau_res: scalar, resistive force, tau
        mass: scalar, rat's weight (mass)
        ampxpenalty: scalar, height of the spatial penalty function
        kxpenalty: scalar, steepness of the spatial penalty function 
        nk: scalar, number of collocation points
    Returns:
        (x0_opt,x1_opt,tgrid,u_opt,tgrid_u,float(res["f"])): lists; optimal positions, optimal speeds
        time for postion and speed), optimal control, time for optimal control, final total cost."""
    
    #mu=0.1    # Control discretization
    #kT=mu/1
    
    # Declare variables
    t = SX.sym("t")    # time
    u = SX.sym("u")    # control

    states = struct_symSX([
            entry('x', shape=2),    #  states
            entry('L')             #  helper state: Langrange integrand
         ])

    # Create a structure for the right hand side
    rhs = struct_SX(states)
    x = states['x']
    #  rhs["x"] = vertcat(x[1],u)    
    rhs["x"] = vertcat(x[1], u - fabs(x[1]-vtapis)/tau_res)
 
    speedlimit = 0.5
    Ltread = dist

    if cost=='speed_heaviside':  
        rhs["L"] = a*mass*((x[1]-vtapis)**2) + (b*spatialpenalty(x=x[0],k=kxpenalty,mu=xb,amp=ampxpenalty)*
                                                speedpenalty(x=x[1],k=kxpenalty,mu=speedlimit,amp=ampxpenalty))
    elif cost=='force_heaviside':   
        rhs["L"] = a*(mass*u) + (b*spatialpenalty(x=x[0],k=kxpenalty,mu=xb,amp=ampxpenalty)*
                                                speedpenalty(x=x[1],k=kxpenalty,mu=speedlimit,amp=ampxpenalty))
    elif cost=='mixed':
        rhs["L"] = a*((mass*u)**2) + ((x[1]-vtapis)**2)  + (b*spatialpenalty(x=x[0],k=kxpenalty,mu=xb,amp=ampxpenalty)*
                                                speedpenalty(x=x[1],k=kxpenalty,mu=speedlimit,amp=ampxpenalty))

    # ODE right hand side function
    f = Function('f', [t, states, u], [rhs])

    # Objective function (meyer term)
    m = Function('m', [t, states, u], [states["L"]])

    # Control bounds
    u_min = -1.5
    u_max = 2.5
    u_init = 0.0

    u_lb = np.array([u_min])
    u_ub = np.array([u_max])
    u_init = np.array([u_init])

    # State bounds and initial guess
    vmax = 5
    x_min  = [0, 0, -inf]
    x_max  = [Ltread, vmax, inf]
    xi_min = [xi, xdoti, -inf]
    xi_max = [xi, xdoti,  inf]
    xf_min = [xf, xdotf, -inf]
    xf_max = [xf, xdotf,  inf]
    x_init = [0, 0, 0]

    # Dimensions
    nx = 3
    nu = 1

    # Choose collocation points
    tau_root = [0] + collocation_points(3, "radau")

    # Degree of interpolating polynomial
    d = len(tau_root)-1

    # Size of the finite elements
    h = tf/nk

    # Coefficients of the collocation equation
    C = np.zeros((d+1,d+1))

    # Coefficients of the continuity equation
    D = np.zeros(d+1)

    # Dimensionless time inside one control interval
    tau = SX.sym("tau")

    # All collocation time points
    T = np.zeros((nk,d+1))
    for k in range(nk):
        for j in range(d+1):
            T[k,j] = h*(k + tau_root[j])

    # For all collocation points
    for j in range(d+1):
    # Construct Lagrange polynomials to get the polynomial basis at the collocation point
        L = 1
        for r in range(d+1):
            if r != j:
                L *= (tau-tau_root[r])/(tau_root[j]-tau_root[r])

        # Evaluate the polynomial at the final time to get the coefficients of the continuity equation
        lfcn = Function('lfcn', [tau],[L])
        D[j] = lfcn(1.0)

        # Evaluate the time derivative of the polynomial at all collocation points to get the coefficients of the continuity equation
        tfcn = Function('tfcn', [tau], [tangent(L, tau)])
        for r in range(d+1):
            C[j,r] = tfcn(tau_root[r])

    # Structure holding NLP variables
    V = struct_symMX([
      (
       entry("X",repeat=[nk+1,d+1],struct=states),
       entry("U",repeat=[nk],shape=nu)
      )
    ])

    vars_lb   = V()
    vars_ub   = V()
    vars_init = V()

    # Set states and its bounds
    vars_init["X",:,:] = repeated(repeated(x_init))
    vars_lb["X",:,:]   = repeated(repeated(x_min))
    vars_ub["X",:,:]   = repeated(repeated(x_max))

    # Set controls and its bounds
    vars_init["U",:] = repeated(u_init)
    vars_lb["U",:]   = repeated(u_min)
    vars_ub["U",:]   = repeated(u_max)

    # State at initial time
    vars_lb["X",0,0] = xi_min
    vars_ub["X",0,0] = xi_max

    # State at end time
    vars_lb["X",-1,0] = xf_min
    vars_ub["X",-1,0] = xf_max

    # Constraint function for the NLP (Non Linear Program)
    g = []
    lbg = []
    ubg = []

    # For all finite elements
    for k in range(nk):

        # For all collocation points
        for j in range(1,d+1):

        # Get an expression for the state derivative at the collocation point
            xp_jk = 0
            for r in range (d+1):
                xp_jk += C[r,j]*V["X",k,r]

            # Add collocation equations to the NLP
            fk = f(T[k][j], V["X",k,j], V["U",k])
            g.append(h*fk - xp_jk)
            lbg.append(np.zeros(nx)) # equality constraints
            ubg.append(np.zeros(nx)) # equality constraints

        # Get an expression for the state at the end of the finite element
        xf_k = 0
        for r in range(d+1):
            xf_k += D[r]*V["X",k,r]

        # Add continuity equation to NLP
        g.append(V["X",k+1,0] - xf_k)
        lbg.append(np.zeros(nx))
        ubg.append(np.zeros(nx))

    # Concatenate constraints
    g = vertcat(*g)

    # Objective function
    f = m(T[nk-1][d],V["X",nk,0],V["U",nk-1])

    # NLP
    nlp = {'x':V, 'f':f, 'g':g}

    ## ----
    ## SOLVE THE NLP
    ## ----

    # Set options
    opts = {'ipopt.print_level':0, 'print_time':0} # hide ipopt output  # opts = {}
    opts["expand"] = True

    # Allocate an NLP solver
    solver = nlpsol("solver", "ipopt", nlp, opts)
    arg = {}

    # Initial condition
    arg["x0"] = vars_init

    # Bounds on x
    arg["lbx"] = vars_lb
    arg["ubx"] = vars_ub

    # Bounds on g
    arg["lbg"] = np.concatenate(lbg)
    arg["ubg"] = np.concatenate(ubg)

    # Solve the problem
    res = solver(**arg)

    # Print the optimal cost
    #print("optimal cost: ", float(res["f"]))

    # Retrieve the solution
    opt = V(res["x"])

    # Get values at the beginning of each finite element
    x0_opt = opt["X",:,0,"x",0]
    x1_opt = opt["X",:,0,"x",1]
    x2_opt = opt["X",:,0,"L"]

    u_opt = opt["U",:,0]

    tgrid = np.linspace(0,tf,nk+1)
    tgrid_u = np.linspace(0,tf,nk)
    
    return(x0_opt,x1_opt,tgrid,u_opt,tgrid_u,float(res["f"]))


In [ ]:
#dist = 1.2 

ampxpenalty = 10
kxpenalty = -10
xb = 1.
vtapis = 0.0
tau_res = 1.8
#mass = 1
shift_time = 0.1 # s 
b = 1
dt = 0.04
bounds = optimize.Bounds(0,100)
selected_rats = animalList
trajectories_before = mediantrack120
speed_before = medianspeed120

# Parameter fit- Before Lesion
ig_before = 1
a_fit_before_dict = dict()
xk_fit_before_dict = dict()

#for rat in ['Rat302']:
#for rat in [selected_rats[2]]:
fig = plt.figure(constrained_layout = False, figsize=(15, 5))
gs = fig.add_gridspec(1, 6) #row ,col




for index, rat in enumerate(selected_rats):    
    a_fit_list = []
    xk_fit_list = []
    mass = int(avgweight[rat])/1000
    print(mass)
    
    #for i_sess in range(len(trajectories_before[rat])):

    #real_trajectory = trajectories_before[rat][i_sess][int(shift_time/dt):]*0.01
    #real_trajectory = trajectories_before[rat][int(shift_time/dt):]*0.01
    real_trajectory = trajectories_before[rat][:]*0.01
    xi = real_trajectory[0]
    xf = real_trajectory[-1]
    xdoti = (real_trajectory[1]*100 - real_trajectory[0]*100)/0.04
    xdotf = (real_trajectory[-2]*100 - real_trajectory[-1]*100)/0.04
    #dist = xf-xi
    print(rat, xi, xf, xdoti, xdotf, dist)
    #tf = tf_before[rat][i_sess]-shift_time
    tf = len(trajectories_before[rat])*dt
    nk = len(real_trajectory)-1
    ax00 = fig.add_subplot(gs[0, index])
    ax00.set_xlim(-5, 50)
    ax00.set_ylim(0, 120)
    ax00.plot([i for i in range(0, len(real_trajectory))], [1+i*100 for i in real_trajectory], '--', color = 'k', zorder =2 )
    ax00.plot([i for i in range(0, len(trajectories_before[rat]))], trajectories_before[rat], color = rat_markers[rat][0])
    ax00.plot([i for i in range(0, len(speed_before[rat]))], speed_before[rat], color = rat_markers[rat][0])

    #ax00.plot([i for i in range(0, len(trajectories_before[rat])-1)], np.diff(trajectories_before[rat])/0.04, color = rat_markers[rat][0], lw = 2)



In [ ]:
avgweight = {}
for animal in animalList:
    avgweight[animal] = []
    for session in matchsession(animal, dist60+dist90+dist120):
        avgweight[animal].extend([params[animal, session]['weight']])
        plt.scatter(params[animal, session]['daysSinceadLib'], params[animal, session]['weight'], color = rat_markers[animal][0])
        #plt.plot((0, 7, 14, 21, 28), (250, 275, 300, 325, 350), color='g', lw=2, marker="$\u2642$")
        #plt.plot((0, 7, 14, 21, 28), (200, 215, 225, 240, 250), color='r', lw=2, marker="$\u2640$")
    avgweight[animal] = sum(avgweight[animal]) / len(avgweight[animal])
    print(avgweight[animal])
    plt.scatter(40, avgweight[animal], color = rat_markers[animal][0], marker ='x')
    plt.annotate("  AVG\nweight", (38, 300))

In [ ]:
# Parameter fit- Before Lesion
ig_before = 1. #initial guess
a_fit_before_dict = dict()
xk_fit_before_dict = dict()
xdotk_fit_before_dict = dict()
acck_fit_before_dict = dict()

fig = plt.figure(constrained_layout = False, figsize=(20, 20))
gs = fig.add_gridspec(6, 3) #row ,col

#for rat in ['Rat302']:
#for rat in [selected_rats[2]]:
for index, rat in enumerate(selected_rats):    
    mass = int(avgweight[rat])/1000
    print((rat))
    a_fit_list = []
    xk_fit_list = []
    xdotk_fit_list = []
    acck_fit_list = []
    #for i_sess in range(len(trajectories_before[rat])):
    #real_trajectory = trajectories_before[rat][i_sess][int(shift_time/dt):]*0.01
    #real_trajectory = trajectories_before[rat][:]*0.01
    real_trajectory = trajectories_before[rat][:]*0.01
    xi = real_trajectory[0]
    xf = real_trajectory[-1]
    dist = 1.2#xf-xi
    xdoti = speed_before[rat][0]*0.01
    xdotf = speed_before[rat][-1]*0.01
    #xdoti = abs((real_trajectory[1] - real_trajectory[0])/0.04)
    #xdotf = abs((real_trajectory[-2] - real_trajectory[-1])/0.04)
    #tf = tf_before[rat][i_sess]-shift_time
    tf = len(trajectories_before[rat])*dt
    nk = len(real_trajectory)-1
    cost = 'speed_heaviside'
    b=1
    def E(a,cost=cost,b=b,
                        xb=xb,xi=xi,xf=xf,xdoti=xdoti,xdotf=xdotf,tf=tf,dist=dist, 
                        vtapis=vtapis,tau_res=tau_res, mass=mass,
                        ampxpenalty=ampxpenalty,kxpenalty=kxpenalty,nk=nk, 
                        real_trajectory=real_trajectory):

        xk,xdotk,tk,acck,tak,oc = optimal_trajectory(cost=cost,a=a,b=b,
                                xb=xb,xi=xi,xf=xf,xdoti=xdoti,xdotf=xdotf,tf=tf,dist=dist,
                                vtapis=vtapis,tau_res=tau_res, mass=mass,
                                ampxpenalty=ampxpenalty,kxpenalty=kxpenalty,nk=nk)    

        #return np.abs((np.max(xk)-np.max(real_trajectory)))
        return np.sum((xk-real_trajectory)**2) 
    res = optimize.minimize(E, x0=np.array([ig_before]), method='trust-constr', 
                            bounds=bounds, 
                            options={'xtol': 1e-1, 'disp': True}) #1e-1,
    
    a_fit = res.x[0]
    print(a_fit)
    xk_fit,xdotk_fit,tk,acck_fit,tak,oc = optimal_trajectory(cost=cost,a=a_fit,b=b,
                            xb=xb,xi=xi,xf=xf,xdoti=xdoti,xdotf=xdotf,tf=tf,dist=dist,
                            vtapis=vtapis,tau_res=tau_res, mass=mass,
                            ampxpenalty=ampxpenalty,kxpenalty=kxpenalty,nk=nk)
    
    a_fit_list.append(a_fit)
    xk_fit_list.append(xk_fit)
    xdotk_fit_list.append(xdotk_fit)
    acck_fit_list.append(acck_fit)
    
    a_fit_before_dict[rat] = a_fit_list
    xk_fit_before_dict[rat] = xk_fit_list
    xdotk_fit_before_dict[rat] = xdotk_fit_list
    acck_fit_before_dict[rat] = acck_fit_list


    ax00 = fig.add_subplot(gs[index, 0])
    ax00.plot([i*0.04 for i in range(0, len(np.array(trajectories_before[rat])))], trajectories_before[rat], color = rat_markers[rat][0])
    ax00.plot([i*0.04 for i in range(0, len(np.array(xk_fit_before_dict[rat][0])))], [i*100 for i in np.array(xk_fit_before_dict[rat][0])], '--', color = rat_markers[rat][0])

    ax01 = fig.add_subplot(gs[index, 1])
    ax01.plot([i*0.04 for i in range(0, len(np.array(speed_before[rat])))], speed_before[rat], color = rat_markers[rat][0])
    ax01.plot([i*0.04 for i in range(0, len(np.array(xdotk_fit_before_dict[rat][0])))], [i*100 for i in np.array(xdotk_fit_before_dict[rat][0])], '--', color = rat_markers[rat][0])

    ax02 = fig.add_subplot(gs[index, 2])
    ax02.plot([i*0.04 for i in range(1, len(np.array(speed_before[rat])))], np.diff(speed_before[rat])/0.04, color = rat_markers[rat][0])
    ax02.plot([i*0.04 for i in range(0, len(np.array(acck_fit_before_dict[rat][0])))], [i*100 for i in np.array(acck_fit_before_dict[rat][0])], '--', color = rat_markers[rat][0])

In [ ]:
#weight, xk
fig = plt.figure(constrained_layout = False, figsize=(20, 20))
gs = fig.add_gridspec(6, 4) #row ,col
for index, rat in enumerate(animalList):
    
    ax00 = fig.add_subplot(gs[index, 0])
    ax00.plot([i*0.04 for i in range(0, len(np.array(trajectories_before[rat])))], trajectories_before[rat], color = rat_markers[rat][0])
    ax00.plot([i*0.04 for i in range(0, len(np.array(xk_fit_before_dict[rat][0])))], [i*100 for i in np.array(xk_fit_before_dict[rat][0])], '--', color = rat_markers[rat][0])

    ax01 = fig.add_subplot(gs[index, 1])
    ax01.plot([i*0.04 for i in range(0, len(np.array(speed_before[rat])))], speed_before[rat], color = rat_markers[rat][0])
    ax01.plot([i*0.04 for i in range(0, len(np.array(xdotk_fit_before_dict[rat][0])))], [i*100 for i in np.array(xdotk_fit_before_dict[rat][0])], '--', color = rat_markers[rat][0])

    ax02 = fig.add_subplot(gs[index, 2])
    ax02.plot([i*0.04 for i in range(1, len(np.array(speed_before[rat])))], np.diff(speed_before[rat])/0.04, color = rat_markers[rat][0])
    ax02.plot([i*0.04 for i in range(0, len(np.array(acck_fit_before_dict[rat][0])))], [i*100 for i in np.array(acck_fit_before_dict[rat][0])], '--', color = rat_markers[rat][0])

    ax03 = fig.add_subplot(gs[index, 3])
    ax03.plot([i*0.04 for i in range(2, len(np.array(speed_before[rat])))], np.diff(np.diff(speed_before[rat])/0.04)/0.04, color = rat_markers[rat][0])
    ax03.plot([i*0.04 for i in range(1, len(np.array(acck_fit_before_dict[rat][0])))], np.diff([i*100 for i in np.array(acck_fit_before_dict[rat][0])])/0.04, '--', color = rat_markers[rat][0])

In [ ]:
a_fit_before_dict

In [ ]:
#plot rat speed along run
def plot_speedcenter(animal, session, posdataRight, timedataRight, bounds, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1 ): 
    ax = plt.gca()
    for i, j in zip(posdataRight, timedataRight):
        time = np.subtract(j, j[0])
        iabs =  [abs(ele) for ele in i]
        #plt.plot(np.subtract(j, j[0]), iabs, color='g' , linewidth=0.3, label="Good Item" if i == posdataRight[0] else "")
        if len(np.where(i == max(i))[0]) == 1:
            maxspeed = max(iabs)
            maxspeedtime = np.where(iabs == maxspeed)[0]
            plt.scatter(time[maxspeedtime]-time[maxspeedtime], maxspeed, color ='darkgreen', s=2)
        else: print(max(i))
        plt.plot([_-j[0]-time[maxspeedtime] for _ in j], iabs, color='g' , linewidth=0.3, label="Good Item" if i == posdataRight[0] else "")
    ax.set_title(title[0], fontsize = title[1])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[2])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[2])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")
 
    return ax


animal = 'RatF02'
session = matchsession(animal, dist120)[5]
fig = plt.figure(constrained_layout = False, figsize=(7,15))
ax = plt.subplot(211)
ax = plot_speedcenter(animal, XtrackRunToRightBin[animal, session][0], instantSpeedRightBin[animal, session][0], trackTimeToRightBin[animal, session][0], [0,0], xylim = [-1, 1, 0, 200], xyLabels=["Time (s)","X Speed (cm/s)", 14], title=["", 12], linewidth = [1.5])

